# Sample Notebook - AIDC with WML

### Import AIDC library and requirements

In [1]:
## Import AIDC libraries
!pip install /project_data/data_asset/aidc-1.4.zip;
import aidc

Processing /project_data/data_asset/aidc-1.4.zip
  Preparing metadata (setup.py) ... done
  Using cached javascript-1%211.0.3-py3-none-any.whl (33 kB)
  Created wheel for aidc: filename=aidc-1.4-py3-none-any.whl size=138109 sha256=dbed5aa3f2ec947c7174c6ef04a7a170df6b663bc21d2334bbc55e1b1eb7825e
  Stored in directory: /tmp/1000710000/.cache/pip/wheels/36/35/e2/ffafef2711229494b6f5514e8d265d330c9b11bfa3285f4fce
Successfully built aidc
  Attempting uninstall: javascript
    Found existing installation: javascript 1!1.0.3
    Uninstalling javascript-1!1.0.3:
      Successfully uninstalled javascript-1!1.0.3
  Attempting uninstall: aidc
    Found existing installation: aidc 1.4
    Uninstalling aidc-1.4:
      Successfully uninstalled aidc-1.4


### Load the dataset from the JSON file and create AIDC model

In [2]:
## Load the AIDC model from JSON
import json
read_file = open('/project_data/data_asset/AIDC_model.json')
model_json = json.load(read_file)
load_model =aidc.load_task_model(model_json)
print(aidc.dispatch_description(load_model))
model_export=aidc.export_model(load_model)
#print(model_export.valueOf())

if confidence <42.659%: use ml else 
if confidence <67.579%: use human else 
use ml 


### Test if the dispatch rules is applied based on different confidence results

In [3]:
#Test the dispatch on the model
case_for_ml={'case': {'caseId': 'CaseDataModel0', 'CheckingStatus': 'no_checking', 'CreditHistory': 'prior_payments_delayed', 'LoanPurpose': 'education', 'ExistingSavings': '100_to_500', 'EmploymentDuration': '1_to_4', 'InstallmentPercent': '3', 'Sex': 'male', 'OthersOnLoan': 'co-applicant', 
               'CurrentResidenceDuration': '2', 'OwnsProperty': 'real_estate', 'InstallmentPlans': 'bank', 'Housing': 'rent', 'ExistingCreditsCount': '1', 'Job': 'skilled', 'Dependents': '1', 'Telephone': 'yes', 'ForeignWorker': 'no', 'Age': '31', 'LoanAmount': '2000', 'LoanDuration': '34'},
      'decisionTools': {'ml': {'outcome': 'Risk', 'confidence': 0.27}}}
dispatch_result = aidc.dispatch(load_model, case_for_ml)
print(dispatch_result)

case_for_human={'case': {'caseId': 'CaseDataModel0', 'CheckingStatus': 'no_checking', 'CreditHistory': 'prior_payments_delayed', 'LoanPurpose': 'education', 'ExistingSavings': '100_to_500', 'EmploymentDuration': '1_to_4', 'InstallmentPercent': '3', 'Sex': 'male', 'OthersOnLoan': 'co-applicant', 
               'CurrentResidenceDuration': '2', 'OwnsProperty': 'real_estate', 'InstallmentPlans': 'bank', 'Housing': 'rent', 'ExistingCreditsCount': '1', 'Job': 'skilled', 'Dependents': '1', 'Telephone': 'yes', 'ForeignWorker': 'no', 'Age': '31', 'LoanAmount': '2000', 'LoanDuration': '34'},
      'decisionTools': {'ml': {'outcome': 'Risk', 'confidence': 0.48}}}
dispatch_result = aidc.dispatch(load_model, case_for_human)
print(dispatch_result)

{
  toolToUse: 'ml',
  caseId: undefined,
  type: 'ml',
  confidence: 0.27,
  outcome: 'Risk'
}
{ toolToUse: 'human', caseId: undefined, type: 'h' }


### Configure WML client to deploy the function

In [4]:
#Configure the WML client
import os
from ibm_watson_machine_learning import APIClient

wml_credentials = {
    "instance_id": "openshift",
    "url": os.environ.get("RUNTIME_ENV_APSX_URL"),
    "version": "4.7"
}
client = APIClient(wml_credentials)

metadata = {            
     client.spaces.ConfigurationMetaNames.NAME: 'AIDC_space',         
     client.spaces.ConfigurationMetaNames.DESCRIPTION:  'AIDC space',            
     }
SPACE_NAME=metadata[client.spaces.ConfigurationMetaNames.NAME]

def guid_from_space_name(client, space_name):
 space = client.spaces.get_details()
 try:
     return_item=next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id']
 except:
     return_item="0"
 return(return_item)

space_uid = guid_from_space_name(client, SPACE_NAME)
if(space_uid=="0"):
    space_details = client.spaces.store(meta_props=metadata,background_mode=False)
    space_uid = guid_from_space_name(client, SPACE_NAME)
client.set.default_space(space_uid)

'SUCCESS'

### Collect the model endpoint to be able to collect the results

In [5]:
#Configure the model endpoint
model_serving_url='https://internal-nginx-svc.cpd-instance.svc.cluster.local:12443/ml/v4/deployments/cd7cded4-d757-477a-a716-d56c7ba3072c/predictions'

### Define the custom function which includes the AIDC model

In [6]:
#AIDC custom_function
def my_deployable_function():
    import aidc
    import json
    import requests
    from ibm_watson_studio_lib import access_project_or_space    
    wslib = access_project_or_space()
    token = wslib.auth.get_current_token()
    
    default_model={'id': '0', 'name': 'taskModel', 'caseDataModel': {'id': 'CaseDataModel0', 'comments': [], 'attributes': {'CheckingStatus': {'name': 'CheckingStatus', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'0_to_200': {'name': '0_to_200', '_className': 'AttributeValue', '_version': 1}, 'less_0': {'name': 'less_0', '_className': 'AttributeValue', '_version': 1}, 'no_checking': {'name': 'no_checking', '_className': 'AttributeValue', '_version': 1}, 'greater_200': {'name': 'greater_200', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'LoanDuration': {'name': 'LoanDuration', 'isUsable': True, 'type': 'continuous', 'range': [4, 64], 'values': {}, '_className': 'Attribute', '_version': 1}, 'CreditHistory': {'name': 'CreditHistory', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'credits_paid_to_date': {'name': 'credits_paid_to_date', '_className': 'AttributeValue', '_version': 1}, 'prior_payments_delayed': {'name': 'prior_payments_delayed', '_className': 'AttributeValue', '_version': 1}, 'outstanding_credit': {'name': 'outstanding_credit', '_className': 'AttributeValue', '_version': 1}, 'all_credits_paid_back': {'name': 'all_credits_paid_back', '_className': 'AttributeValue', '_version': 1}, 'no_credits': {'name': 'no_credits', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'LoanPurpose': {'name': 'LoanPurpose', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'other': {'name': 'other', '_className': 'AttributeValue', '_version': 1}, 'car_new': {'name': 'car_new', '_className': 'AttributeValue', '_version': 1}, 'furniture': {'name': 'furniture', '_className': 'AttributeValue', '_version': 1}, 'retraining': {'name': 'retraining', '_className': 'AttributeValue', '_version': 1}, 'education': {'name': 'education', '_className': 'AttributeValue', '_version': 1}, 'vacation': {'name': 'vacation', '_className': 'AttributeValue', '_version': 1}, 'appliances': {'name': 'appliances', '_className': 'AttributeValue', '_version': 1}, 'car_used': {'name': 'car_used', '_className': 'AttributeValue', '_version': 1}, 'repairs': {'name': 'repairs', '_className': 'AttributeValue', '_version': 1}, 'radio_tv': {'name': 'radio_tv', '_className': 'AttributeValue', '_version': 1}, 'business': {'name': 'business', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'LoanAmount': {'name': 'LoanAmount', 'isUsable': True, 'type': 'continuous', 'range': [250, 11676], 'values': {}, '_className': 'Attribute', '_version': 1}, 'ExistingSavings': {'name': 'ExistingSavings', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'100_to_500': {'name': '100_to_500', '_className': 'AttributeValue', '_version': 1}, 'less_100': {'name': 'less_100', '_className': 'AttributeValue', '_version': 1}, '500_to_1000': {'name': '500_to_1000', '_className': 'AttributeValue', '_version': 1}, 'unknown': {'name': 'unknown', '_className': 'AttributeValue', '_version': 1}, 'greater_1000': {'name': 'greater_1000', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'EmploymentDuration': {'name': 'EmploymentDuration', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'less_1': {'name': 'less_1', '_className': 'AttributeValue', '_version': 1}, '1_to_4': {'name': '1_to_4', '_className': 'AttributeValue', '_version': 1}, 'greater_7': {'name': 'greater_7', '_className': 'AttributeValue', '_version': 1}, '4_to_7': {'name': '4_to_7', '_className': 'AttributeValue', '_version': 1}, 'unemployed': {'name': 'unemployed', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'InstallmentPercent': {'name': 'InstallmentPercent', 'isUsable': True, 'type': 'discrete', 'range': [1, 6], 'values': {'1': {'name': '1', '_className': 'AttributeValue', '_version': 1}, '2': {'name': '2', '_className': 'AttributeValue', '_version': 1}, '3': {'name': '3', '_className': 'AttributeValue', '_version': 1}, '4': {'name': '4', '_className': 'AttributeValue', '_version': 1}, '5': {'name': '5', '_className': 'AttributeValue', '_version': 1}, '6': {'name': '6', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'Sex': {'name': 'Sex', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'female': {'name': 'female', '_className': 'AttributeValue', '_version': 1}, 'male': {'name': 'male', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'OthersOnLoan': {'name': 'OthersOnLoan', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'none': {'name': 'none', '_className': 'AttributeValue', '_version': 1}, 'co-applicant': {'name': 'co-applicant', '_className': 'AttributeValue', '_version': 1}, 'guarantor': {'name': 'guarantor', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'CurrentResidenceDuration': {'name': 'CurrentResidenceDuration', 'isUsable': True, 'type': 'discrete', 'range': [1, 6], 'values': {'1': {'name': '1', '_className': 'AttributeValue', '_version': 1}, '2': {'name': '2', '_className': 'AttributeValue', '_version': 1}, '3': {'name': '3', '_className': 'AttributeValue', '_version': 1}, '4': {'name': '4', '_className': 'AttributeValue', '_version': 1}, '5': {'name': '5', '_className': 'AttributeValue', '_version': 1}, '6': {'name': '6', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'OwnsProperty': {'name': 'OwnsProperty', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'savings_insurance': {'name': 'savings_insurance', '_className': 'AttributeValue', '_version': 1}, 'real_estate': {'name': 'real_estate', '_className': 'AttributeValue', '_version': 1}, 'unknown': {'name': 'unknown', '_className': 'AttributeValue', '_version': 1}, 'car_other': {'name': 'car_other', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'Age': {'name': 'Age', 'isUsable': True, 'type': 'continuous', 'range': [19, 74], 'values': {}, '_className': 'Attribute', '_version': 1}, 'InstallmentPlans': {'name': 'InstallmentPlans', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'none': {'name': 'none', '_className': 'AttributeValue', '_version': 1}, 'stores': {'name': 'stores', '_className': 'AttributeValue', '_version': 1}, 'bank': {'name': 'bank', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'Housing': {'name': 'Housing', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'own': {'name': 'own', '_className': 'AttributeValue', '_version': 1}, 'free': {'name': 'free', '_className': 'AttributeValue', '_version': 1}, 'rent': {'name': 'rent', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'ExistingCreditsCount': {'name': 'ExistingCreditsCount', 'isUsable': True, 'type': 'discrete', 'range': [1, 4], 'values': {'1': {'name': '1', '_className': 'AttributeValue', '_version': 1}, '2': {'name': '2', '_className': 'AttributeValue', '_version': 1}, '3': {'name': '3', '_className': 'AttributeValue', '_version': 1}, '4': {'name': '4', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'Job': {'name': 'Job', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'skilled': {'name': 'skilled', '_className': 'AttributeValue', '_version': 1}, 'management_self-employed': {'name': 'management_self-employed', '_className': 'AttributeValue', '_version': 1}, 'unskilled': {'name': 'unskilled', '_className': 'AttributeValue', '_version': 1}, 'unemployed': {'name': 'unemployed', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'Dependents': {'name': 'Dependents', 'isUsable': True, 'type': 'discrete', 'range': [1, 2], 'values': {'1': {'name': '1', '_className': 'AttributeValue', '_version': 1}, '2': {'name': '2', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'Telephone': {'name': 'Telephone', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'none': {'name': 'none', '_className': 'AttributeValue', '_version': 1}, 'yes': {'name': 'yes', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}, 'ForeignWorker': {'name': 'ForeignWorker', 'isUsable': True, 'type': 'categorical', 'range': [], 'values': {'yes': {'name': 'yes', '_className': 'AttributeValue', '_version': 1}, 'no': {'name': 'no', '_className': 'AttributeValue', '_version': 1}}, '_className': 'Attribute', '_version': 1}}, 'outcomes': ['No Risk', 'Risk'], 'targetOutcome': 'No Risk', '_className': 'CaseDataModel', '_version': 1}, 'decisionMetrics': {'Scenario0': {'id': 'DecisionMetrics2', 'comments': [], 'costMatrix': {'No Risk': {'No Risk': {'value': '0', '_className': 'Expression', '_version': 1}, 'Risk': {'value': '-LoanAmount*0.3', '_className': 'Expression', '_version': 1}}, 'Risk': {'No Risk': {'value': '-LoanAmount*0.5', '_className': 'Expression', '_version': 1}, 'Risk': {'value': 'LoanAmount*0.3', '_className': 'Expression', '_version': 1}}}, 'toolCosts': {'DecisionTool1': {'value': '-0.05', '_className': 'Expression', '_version': 1}, 'DecisionTool2': {'value': '-0.02', '_className': 'Expression', '_version': 1}}, 'indicators': {'Performance': {'name': 'Performance', 'expression': '0', 'aggregation': 'Average', '_className': 'PerformanceIndicator', '_version': 1}, 'Impact': {'name': 'Impact', 'expression': '0', 'aggregation': 'Average', '_className': 'ImpactIndicator', '_version': 1}, 'Decision Cost': {'name': 'Decision Cost', 'expression': '0', 'aggregation': 'Average', '_className': 'DecisionCostIndicator', '_version': 1}, 'ml volume': {'name': 'ml volume', 'expression': '0', 'aggregation': '%', 'tool': 'DecisionTool1', '_className': 'ToolUsageIndicator', '_version': 1}, 'human volume': {'name': 'human volume', 'expression': '0', 'aggregation': '%', 'tool': 'DecisionTool2', '_className': 'ToolUsageIndicator', '_version': 1}}, 'commentMatrix': {'No Risk': {'No Risk': '', 'Risk': ''}, 'Risk': {'No Risk': '', 'Risk': ''}}, 'windowWidth': 0.1, 'samples': 100, 'confidenceIntervalWidth': 1.96, 'robustness': 0.5, '_className': 'DecisionMetrics', '_version': 1}}, 'defaultMetrics': 'DecisionMetrics2', 'decisionTools': {'DecisionTool1': {'id': 'DecisionTool1', 'name': 'ml', 'comments': [], 'type': 'ml', 'hasHuman': False, '_className': 'DecisionTool', '_version': 1}, 'DecisionTool2': {'id': 'DecisionTool2', 'name': 'human', 'comments': [], 'type': 'h', 'hasHuman': True, '_className': 'DecisionTool', '_version': 1}}, 'caseSources': {}, 'analyses': {}, 'scenarios': {'Scenario0': {'id': 'Scenario0', 'name': 'initial', 'description': 'simple scenario: uniform costs', 'comments': [], 'decisionTools': ['DecisionTool1', 'DecisionTool2'], 'decisionMetrics': {'id': 'DecisionMetrics2', 'comments': [], 'costMatrix': {'No Risk': {'No Risk': {'value': '0', '_className': 'Expression', '_version': 1}, 'Risk': {'value': '-LoanAmount*0.3', '_className': 'Expression', '_version': 1}}, 'Risk': {'No Risk': {'value': '-LoanAmount*0.5', '_className': 'Expression', '_version': 1}, 'Risk': {'value': 'LoanAmount*0.3', '_className': 'Expression', '_version': 1}}}, 'toolCosts': {'DecisionTool1': {'value': '-0.05', '_className': 'Expression', '_version': 1}, 'DecisionTool2': {'value': '-0.02', '_className': 'Expression', '_version': 1}}, 'indicators': {'Performance': {'name': 'Performance', 'expression': '0', 'aggregation': 'Average', '_className': 'PerformanceIndicator', '_version': 1}, 'Impact': {'name': 'Impact', 'expression': '0', 'aggregation': 'Average', '_className': 'ImpactIndicator', '_version': 1}, 'Decision Cost': {'name': 'Decision Cost', 'expression': '0', 'aggregation': 'Average', '_className': 'DecisionCostIndicator', '_version': 1}, 'ml volume': {'name': 'ml volume', 'expression': '0', 'aggregation': '%', 'tool': 'DecisionTool1', '_className': 'ToolUsageIndicator', '_version': 1}, 'human volume': {'name': 'human volume', 'expression': '0', 'aggregation': '%', 'tool': 'DecisionTool2', '_className': 'ToolUsageIndicator', '_version': 1}}, 'commentMatrix': {'No Risk': {'No Risk': '', 'Risk': ''}, 'Risk': {'No Risk': '', 'Risk': ''}}, 'windowWidth': 0.1, 'samples': 100, 'confidenceIntervalWidth': 1.96, 'robustness': 0.5, '_className': 'DecisionMetrics', '_version': 1}, 'caseSources': ['TableCaseSource0'], 'selectedAttributes': [], 'computedTools': {'CompositeDecisionTool0': {'id': 'CompositeDecisionTool0', 'name': 'initial scenario', 'comments': [], 'type': 'o', 'hasHuman': False, 'tools': ['DecisionTool1', 'DecisionTool2'], 'confidenceProvider': 'DecisionTool1', 'attributes': [], 'useDeciles': False, 'dispatchConfiguration': {'tools': ['DecisionTool1', 'DecisionTool2'], 'useDeciles': False, 'decisionTree': {'bestTool': 'DecisionTool1', 'basisCount': 4961, 'basisPerf': 0.7698044749042532, 'basisStdDev': 0.6382797744072625, 'basisConfidence95': 0.017761618624811116, 'secondTool': 'DecisionTool2', 'secondPerf': 0.5895988711953235, 'secondStdDev': 0.8076962121275549, 'secondConfidence95': 0.022476024871439102, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[0.7698044749042532, 0.7698044749042532, 0, 4961, 0.6382797744072625, 0.017761618624811116]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[0.5895988711953235, 0.5895988711953235, 0, 4961, 0.8076962121275549, 0.022476024871439102]], '_className': 'AnalysisResult', '_version': 1}}, 'slots': [{'bestTool': 'DecisionTool1', 'basisCount': 1113, 'basisPerf': 0.8885893980233602, 'basisStdDev': 0.458703479080421, 'basisConfidence95': 0.02694886775934558, 'secondTool': 'DecisionTool2', 'secondPerf': 0.5884995507637018, 'secondStdDev': 0.8084975440599192, 'secondConfidence95': 0.04749929833169306, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[0.8885893980233602, 0.8885893980233602, 0, 1113, 0.458703479080421, 0.02694886775934558]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[0.5884995507637018, 0.5884995507637018, 0, 1113, 0.8084975440599192, 0.04749929833169306]], '_className': 'AnalysisResult', '_version': 1}}, 'param': 0, 'lowBound': 0, 'highBound': 0.4293314876469639, '_className': 'DecisionNode', '_version': 1}, {'bestTool': 'DecisionTool2', 'basisCount': 984, 'basisPerf': 0.6158536585365854, 'basisStdDev': 0.7878605658789521, 'basisConfidence95': 0.04922751189464175, 'secondTool': 'DecisionTool1', 'secondPerf': 0.20528455284552846, 'secondStdDev': 0.9787023308253696, 'secondConfidence95': 0.06115178588519657, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[0.20528455284552846, 0.20528455284552846, 0, 984, 0.9787023308253696, 0.06115178588519657]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[0.6158536585365854, 0.6158536585365854, 0, 984, 0.7878605658789521, 0.04922751189464175]], '_className': 'AnalysisResult', '_version': 1}}, 'param': 1, 'lowBound': 0.4293314876469639, 'highBound': 0.6784962888641944, '_className': 'DecisionNode', '_version': 1}, {'bestTool': 'DecisionTool1', 'basisCount': 2864, 'basisPerf': 0.9175977653631285, 'basisStdDev': 0.3975101772289527, 'basisConfidence95': 0.014558539753161076, 'secondTool': 'DecisionTool2', 'secondPerf': 0.5810055865921788, 'secondStdDev': 0.8138995689571773, 'secondConfidence95': 0.029808517890899153, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[0.9175977653631285, 0.9175977653631285, 0, 2864, 0.3975101772289527, 0.014558539753161076]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[0.5810055865921788, 0.5810055865921788, 0, 2864, 0.8138995689571773, 0.029808517890899153]], '_className': 'AnalysisResult', '_version': 1}}, 'param': 2, 'lowBound': 0.6784962888641944, 'highBound': 1, '_className': 'DecisionNode', '_version': 1}], 'confidenceProvider': 'DecisionTool1', 'curvesPerTool': {'DecisionTool1': {'name': 'ScalarAxisPerformanceCurve', 'description': 'average performance over ml', 'headerLabels': ['count', 'totalCount', 'weightedCount', 'weightedSum', 'weightedAverage', 'weightedStdDev', 'confidence95'], 'rowLabels': [0.005, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095, 0.105, 0.115, 0.125, 0.135, 0.145, 0.155, 0.165, 0.175, 0.185, 0.195, 0.205, 0.215, 0.225, 0.235, 0.245, 0.255, 0.265, 0.275, 0.285, 0.295, 0.305, 0.315, 0.325, 0.335, 0.345, 0.355, 0.365, 0.375, 0.385, 0.395, 0.405, 0.415, 0.425, 0.435, 0.445, 0.455, 0.465, 0.475, 0.485, 0.495, 0.505, 0.515, 0.525, 0.535, 0.545, 0.555, 0.565, 0.575, 0.585, 0.595, 0.605, 0.615, 0.625, 0.635, 0.645, 0.655, 0.665, 0.675, 0.685, 0.695, 0.705, 0.715, 0.725, 0.735, 0.745, 0.755, 0.765, 0.775, 0.785, 0.795, 0.805, 0.815, 0.825, 0.835, 0.845, 0.855, 0.865, 0.875, 0.885, 0.895, 0.905, 0.915, 0.925, 0.935, 0.945, 0.955, 0.965, 0.975, 0.985, 0.995], 'content': [[79, 362, 489.9275345206288, 241.61504637599123, 0.9863297298136152, 0.16478368877349603, 0.03594572584199289], [87, 388, 530.9121080279357, 261.9073331296448, 0.9866316068868548, 0.16296647597547023, 0.03433949416493374], [30, 422, 546.8421381860971, 269.67234820872545, 0.986289568332251, 0.1650238994781725, 0.034048541054768004], [28, 443, 554.7489493340258, 273.4156592473389, 0.9857275424336484, 0.16834848406718517, 0.0341438454693685], [26, 472, 557.8916530102493, 274.58701108545074, 0.9843739715546744, 0.17609055660561973, 0.034821636097482726], [24, 505, 556.0899019777773, 273.2861355692148, 0.9828847263625944, 0.18422164553365716, 0.03567417148120009], [18, 525, 549.6626098096364, 269.6997746914622, 0.9813284363106555, 0.19233954374512838, 0.036664204309253066], [18, 553, 540.8611569106579, 265.0990482419729, 0.9802850319523437, 0.19758860323964178, 0.037462272670385356], [15, 581, 530.0420757234103, 259.736701729894, 0.9800606918814925, 0.1986983649373336, 0.03794879979244316], [29, 609, 518.6310228511693, 253.65686661228565, 0.9781785332385609, 0.2077661115563823, 0.039229627251474344], [20, 587, 506.1339526101943, 246.73440327867837, 0.9749766914716512, 0.22230710984354957, 0.04107712236823064], [34, 535, 503.6596155658367, 244.04314388111237, 0.9690796575260137, 0.24674808483403338, 0.04338248265249042], [18, 504, 509.2044422701001, 244.92055702582002, 0.9619733713788202, 0.2731432458584077, 0.04539470649578341], [37, 502, 516.0882146239277, 246.19568364471186, 0.9540837270392393, 0.29954005040881465, 0.0472195180175456], [24, 507, 521.8740411460395, 246.95524228364226, 0.9464170386452891, 0.32294703739449726, 0.0487572133682356], [27, 492, 525.7403756707907, 246.48840954601752, 0.9376811101164662, 0.34749695787265616, 0.05039017994841065], [24, 496, 527.9288654267792, 244.7956073775887, 0.9273810295623643, 0.37411819791056355, 0.05217624058390236], [30, 504, 528.647108361125, 242.1415526419878, 0.9160801177657368, 0.400995284054955, 0.05398122967178537], [31, 505, 526.9675245165822, 237.74919580668225, 0.9023295924157123, 0.4310467569195905, 0.05605654623889823], [25, 504, 520.1420621842151, 230.94407788217086, 0.8880038538409109, 0.4598360094247618, 0.05827691755649791], [27, 494, 510.1043381586664, 222.62037299200847, 0.8728425004014103, 0.4880020179189978, 0.060622992692727286], [24, 481, 497.3211818471546, 213.81223781630368, 0.8598557456256355, 0.5105370669349899, 0.06279878370303994], [25, 462, 483.73101902753115, 205.14386024698615, 0.8481732705890856, 0.5297188905997349, 0.06485998339678045], [25, 462, 469.59355347305564, 196.4707230709495, 0.8367692512721967, 0.5475556776487375, 0.06692820099182197], [19, 452, 453.5758971214299, 187.1438356235625, 0.8251930352174818, 0.5648508251109838, 0.0691668457826484], [22, 435, 439.04738725721785, 178.6786908179519, 0.8139380668413917, 0.580951653191983, 0.07129685431458213], [24, 433, 424.69398812651576, 170.5758557364342, 0.8032882993654239, 0.5955903861737573, 0.07339928919231208], [24, 413, 409.28343667089933, 162.283869639039, 0.7930145962370319, 0.6092026347242905, 0.0756179454030248], [18, 413, 392.48070008605765, 154.02781770303864, 0.7848937166554462, 0.6196304169057547, 0.07787772978185734], [17, 410, 380.1750897437334, 148.6040327340364, 0.7817662795014092, 0.6235715550235787, 0.07937932303742552], [17, 407, 372.5680188298226, 145.24834966808552, 0.7797145344052218, 0.6261351649901539, 0.08035026858527253], [13, 410, 369.03065403029325, 143.0001410778612, 0.7750041332128604, 0.6319561642257971, 0.08110886555475127], [20, 419, 370.25391985848546, 142.88209294714034, 0.7718059703554319, 0.6358581163464933, 0.08122436941837312], [13, 411, 374.1286192186176, 143.93409005589783, 0.7694364058890222, 0.6387234278563646, 0.08098452305118117], [20, 410, 381.0866052798922, 145.63392781428993, 0.7643088253250355, 0.6448503853842895, 0.08062573952563568], [12, 413, 389.28726483807003, 147.70651754401624, 0.7588561501258306, 0.6512582770423737, 0.08016736352101365], [19, 409, 399.6015967302028, 150.89622004143897, 0.7552333187663356, 0.6554560505672258, 0.07938057717153063], [13, 408, 412.4199087359011, 154.52112149633467, 0.74933880844867, 0.6621867940035713, 0.07853739970886], [28, 422, 430.01258087716934, 159.08583817910394, 0.739912482814292, 0.6727031423856976, 0.0775224016266612], [28, 437, 446.0132111277429, 161.58788046520198, 0.7245878661604106, 0.6891824317356787, 0.0770458603115016], [28, 463, 461.919537695125, 162.96709458697592, 0.7056081472550185, 0.7086022456409097, 0.07676692904095223], [26, 486, 478.0187630884355, 161.08101063817725, 0.6739526691272434, 0.7387745256681918, 0.07705300053697492], [28, 501, 493.2074170149863, 154.65605549141765, 0.627144078357274, 0.7789032706192763, 0.07789023884274023], [21, 515, 507.0566467501217, 143.32853297963766, 0.5653353876663417, 0.8248611394969151, 0.07905298134485933], [25, 531, 523.5474086277183, 130.62247381657335, 0.4989900500470466, 0.8666077139940804, 0.07974240679565908], [17, 569, 542.0604638542986, 115.37009288091225, 0.4256724132233439, 0.9048773378865304, 0.08008055253691054], [24, 586, 565.8530931267888, 97.93049549441767, 0.3461339937306827, 0.9381850874875636, 0.07980837847334103], [26, 617, 591.2417580429465, 79.3060391647741, 0.26826941123807935, 0.9633439276779471, 0.07911597847144045], [34, 637, 616.3925462942572, 59.66697278041371, 0.1936005655458706, 0.9810804355506831, 0.07819512707742302], [35, 656, 644.7653608489765, 39.92841162886466, 0.1238540841471076, 0.9923004413180938, 0.0768912311748369], [42, 680, 671.3958571318558, 20.338109989277918, 0.06058455611019269, 0.9981630686220205, 0.07557314422112892], [33, 693, 697.5982532478866, 6.110402605682534, 0.01751839995938535, 0.9998465410566082, 0.07420275898301724], [31, 715, 724.8603003658354, -1.5990875698626665, -0.0044121262236478135, 0.9999902665237228, 0.07279923245118869], [37, 740, 755.4656619962318, -2.465607574768378, -0.006527384893320741, 0.9999786963963048, 0.07130895640568517], [41, 776, 785.6776376809897, 1.1107178417965728, 0.0028274136580365796, 0.9999960028580162, 0.06992508569304007], [50, 806, 815.1952161300952, 9.673824203945758, 0.023733760975363557, 0.9997183146216541, 0.06863791527967196], [32, 841, 839.8752498216907, 22.955187398567688, 0.054663326258134595, 0.9985048426334244, 0.06758086830571432], [43, 866, 864.6347760818899, 40.01236262731253, 0.09255321144640831, 0.9957077397765652, 0.06651286623878065], [50, 881, 889.0974369395549, 59.591487261094315, 0.13404939612967443, 0.9909746512385001, 0.06543538544986666], [37, 911, 909.818261859937, 82.72002556230916, 0.18183856937143691, 0.9833283961571285, 0.06443592047531649], [49, 951, 936.4902138557305, 109.90325056519305, 0.23471307855465537, 0.9720646947376467, 0.06314689992773344], [50, 984, 967.3467242900265, 138.4479255741469, 0.2862426100129904, 0.9581571730217093, 0.061685539338492815], [50, 1033, 1000.0107291210443, 167.67131369840354, 0.33533902950376865, 0.9420975189923122, 0.060159147259183594], [46, 1053, 1033.7052764605755, 199.5093286575739, 0.38600814603693856, 0.9224953719087812, 0.05855173961596633], [59, 1072, 1063.8112291172135, 230.88664428628854, 0.43407446352655266, 0.9008769949966174, 0.057036981611155446], [57, 1081, 1092.8242930419751, 262.4158067006632, 0.48025251336645364, 0.8771302773278359, 0.05552811774468626], [50, 1108, 1120.1225627571314, 293.19922421798117, 0.5235127547047788, 0.8520178376427436, 0.05405646505773237], [53, 1128, 1149.296166934073, 325.66694352030754, 0.5667241445502685, 0.8239076064606795, 0.05247825051646886], [47, 1149, 1177.532691988352, 359.1953733421878, 0.6100813604345192, 0.7923387745468262, 0.05084228249828954], [74, 1170, 1209.2702154651274, 391.41655239872415, 0.6473599488236328, 0.7621844243088803, 0.04920672124311637], [74, 1195, 1234.538417451084, 420.90935294889107, 0.6818894365684148, 0.7314552592581545, 0.047708707406724754], [70, 1213, 1251.2971808176467, 447.36381395030565, 0.7150400733069351, 0.6990834668086567, 0.04632765885904326], [79, 1219, 1260.773050757125, 469.43411736842245, 0.744676636427969, 0.6674254319085593, 0.045096100367743926], [52, 1241, 1262.8039138142037, 485.6976759171134, 0.7692368872220237, 0.6389637011106143, 0.04408858953329172], [57, 1243, 1269.3849869515752, 502.14556728321935, 0.7911635515543959, 0.6116046392007103, 0.043022414290948656], [51, 1270, 1275.8706639830027, 517.3716590775182, 0.8110095696728249, 0.5850328861688885, 0.04197037844201896], [76, 1298, 1288.6035830840453, 533.5592366721472, 0.8281200575201992, 0.5605507740898605, 0.040879340650138846], [60, 1310, 1296.0470336526632, 545.3773929905146, 0.8416012364203661, 0.5400993971999142, 0.040011288414458224], [67, 1347, 1305.2835923705268, 556.3458248073248, 0.8524520312048735, 0.5228054461218689, 0.03922596923885498], [68, 1366, 1313.2979664422573, 568.3602235276248, 0.8655464914292386, 0.5008285846220577, 0.03827533403311521], [69, 1353, 1323.3561954919246, 581.1868788139897, 0.8783529042200886, 0.47801273586394777, 0.037250956424000115], [50, 1353, 1335.6022728465473, 593.6967872209833, 0.889032310428234, 0.45784446159655146, 0.036289123537079096], [65, 1349, 1354.0705972202122, 608.9988714084029, 0.899508301352417, 0.43690366878533876, 0.035206938632476786], [75, 1353, 1376.0786884970978, 625.3224413577477, 0.9088469236315283, 0.41712979922981713, 0.034124794528731926], [64, 1373, 1393.1938667841257, 638.7799591064457, 0.9170008199661767, 0.39888531707918323, 0.03316456608837965], [88, 1397, 1412.0869614254686, 652.3854782870034, 0.9240018442326463, 0.3823880121743704, 0.03225354716590866], [63, 1417, 1425.814054697005, 663.6117048721741, 0.9308530838029871, 0.3653936731450976, 0.031376546252019015], [79, 1464, 1444.0282679852098, 677.4854028476402, 0.9383270644596262, 0.3457489263939665, 0.030328340266897933], [75, 1544, 1469.1220184586969, 694.687562087553, 0.9457179912344817, 0.32498843218708423, 0.029151512243808426], [82, 1777, 1517.2967131979808, 722.4499407343583, 0.9522856465057025, 0.3052082034598922, 0.027798343305251258], [64, 2041, 1628.5344078075004, 781.1165326649259, 0.9592877238824138, 0.2824306336226629, 0.02581151535890055], [79, 1980, 1748.1958761658364, 843.5172747151844, 0.9650146030148479, 0.2621961402616266, 0.024003477949933835], [61, 1923, 1849.7212775517257, 896.5488830162212, 0.9693880844609034, 0.24553358569698794, 0.022581790724225836], [68, 1858, 1936.6869325205805, 942.6786286026306, 0.9734961420695321, 0.2287034354261839, 0.02129917120194863], [81, 1792, 2011.5537233241027, 983.0984506178714, 0.9774518465192132, 0.21115844225647537, 0.020081422713857554], [90, 1717, 2067.1792015753595, 1013.9861293192897, 0.9810336022600744, 0.19383774461291153, 0.018979561831458693], [81, 1636, 2104.682952595497, 1035.768424446506, 0.9842512604277954, 0.1767751576050678, 0.01796277153465826], [133, 1573, 2120.6448727175593, 1046.4309627376497, 0.9868988213916945, 0.1613403741649442, 0.01709596530705885], [194, 1488, 2101.3805918458793, 1039.3361941231385, 0.9891936740599402, 0.14661471685951744, 0.016371651673724754], [422, 1407, 2015.6070777639507, 998.7075674671568, 0.9909744597395348, 0.13405081179887965, 0.015984087864021973]], 'totalCount': 4961, 'weightedCount': 89487.2330131929, 'minAxis': 0, 'maxAxis': 1}, 'DecisionTool2': {'name': 'ScalarAxisPerformanceCurve', 'description': 'average performance over ml', 'headerLabels': ['count', 'totalCount', 'weightedCount', 'weightedSum', 'weightedAverage', 'weightedStdDev', 'confidence95'], 'rowLabels': [0.005, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095, 0.105, 0.115, 0.125, 0.135, 0.145, 0.155, 0.165, 0.175, 0.185, 0.195, 0.205, 0.215, 0.225, 0.235, 0.245, 0.255, 0.265, 0.275, 0.285, 0.295, 0.305, 0.315, 0.325, 0.335, 0.345, 0.355, 0.365, 0.375, 0.385, 0.395, 0.405, 0.415, 0.425, 0.435, 0.445, 0.455, 0.465, 0.475, 0.485, 0.495, 0.505, 0.515, 0.525, 0.535, 0.545, 0.555, 0.565, 0.575, 0.585, 0.595, 0.605, 0.615, 0.625, 0.635, 0.645, 0.655, 0.665, 0.675, 0.685, 0.695, 0.705, 0.715, 0.725, 0.735, 0.745, 0.755, 0.765, 0.775, 0.785, 0.795, 0.805, 0.815, 0.825, 0.835, 0.845, 0.855, 0.865, 0.875, 0.885, 0.895, 0.905, 0.915, 0.925, 0.935, 0.945, 0.955, 0.965, 0.975, 0.985, 0.995], 'content': [[79, 362, 489.9275345206288, 148.1678810536863, 0.6048563128776244, 0.7963346286405478, 0.07902016789047377], [87, 388, 530.9121080279357, 159.1045597255232, 0.5993630859786357, 0.8004772896005017, 0.076106064329513], [30, 422, 546.8421381860971, 164.0264195278287, 0.5999040969004787, 0.8000719183436079, 0.07497036041395001], [28, 443, 554.7489493340258, 166.28920543044805, 0.5995115651145537, 0.8003660932934984, 0.07444785136496639], [26, 472, 557.8916530102493, 167.00051197856666, 0.5986843899795669, 0.8009850193323178, 0.07426656484114247], [24, 505, 556.0899019777773, 166.4980591505767, 0.5988170565889195, 0.8008858425132654, 0.0743821753229064], [18, 525, 549.6626098096364, 164.19292170107357, 0.5974316563316475, 0.8019198314126079, 0.07486407284157405], [18, 553, 540.8611569106579, 160.32137282192707, 0.5928374436710002, 0.8053221500627142, 0.07563067859969748], [15, 581, 530.0420757234103, 155.5509178310632, 0.5869379996626294, 0.8096318821242343, 0.07660280967339962], [29, 609, 518.6310228511693, 150.35049314722397, 0.5797975305089677, 0.8147605928220278, 0.0776858341715802], [20, 587, 506.1339526101943, 145.0126339904964, 0.5730207714485409, 0.8195408443076648, 0.07886941815367608], [34, 535, 503.6596155658367, 142.9725192643703, 0.5677346955989224, 0.8232115860537911, 0.0792397768077082], [18, 504, 509.2044422701001, 142.6385049410164, 0.560240614968382, 0.8283299181726139, 0.07905178000534215], [37, 502, 516.0882146239277, 142.50689470171932, 0.5522578918240315, 0.8336733298589305, 0.078775660749677], [24, 507, 521.8740411460395, 143.2850566834211, 0.5491173938016383, 0.8357452290168916, 0.07843504985452932], [27, 492, 525.7403756707907, 143.9432636484504, 0.5475830668884564, 0.8367513279684903, 0.07819313234744232], [24, 496, 527.9288654267792, 144.83933045566084, 0.5487077518997652, 0.8360142361258601, 0.07799651596705069], [30, 504, 528.647108361125, 146.8213930055499, 0.5554608762004407, 0.8315426717917963, 0.07773478668469941], [31, 505, 526.9675245165822, 149.60463769137866, 0.5677945252076763, 0.8231703208596566, 0.07746561932463569], [25, 504, 520.1420621842151, 150.69026546627308, 0.5794196486763039, 0.8150293680155508, 0.07758570535134572], [27, 494, 510.1043381586664, 151.1899147786198, 0.5927803528367275, 0.8053641743278411, 0.0778794213783042], [24, 481, 497.3211818471546, 150.78963886871932, 0.6064074661314652, 0.7951540637008762, 0.0783724158521079], [25, 462, 483.73101902753115, 149.59767850115895, 0.6185159628667299, 0.7857722339705328, 0.07899551631961815], [25, 462, 469.59355347305564, 148.5985958851874, 0.6328817539600815, 0.7742484649674223, 0.07958572642444017], [19, 452, 453.5758971214299, 145.34520131051556, 0.640885912293546, 0.7676361425986067, 0.0806322529204644], [22, 435, 439.04738725721785, 141.69793803244818, 0.6454790172771661, 0.7637780032541556, 0.08174928184206795], [24, 433, 424.69398812651576, 137.77638399302947, 0.6488266273832257, 0.7609362703922778, 0.08296447273393764], [24, 413, 409.28343667089933, 132.72827414423224, 0.6485885440360867, 0.7611392123292222, 0.08452322024347231], [18, 413, 392.48070008605765, 126.08692007288343, 0.6425127148684604, 0.7662750232340596, 0.0866042596789703], [17, 410, 380.1750897437334, 121.02026496082532, 0.6366554160210869, 0.7711484171357781, 0.08827408579872578], [17, 407, 372.5680188298226, 116.39914897680289, 0.6248477759438089, 0.780746602232809, 0.08972394006646871], [13, 410, 369.03065403029325, 112.93680653758341, 0.6120727657942073, 0.7908014475031195, 0.09073160171549284], [20, 419, 370.25391985848546, 111.30680208466947, 0.6012457727778384, 0.7990641530670611, 0.0910535877689523], [13, 411, 374.1286192186176, 110.35765837691714, 0.5899450226898091, 0.8074434161001751, 0.09105455046822379], [20, 410, 381.0866052798922, 111.1984538961202, 0.5835862628362368, 0.8120511522243141, 0.09047652724688443], [12, 413, 389.28726483807003, 112.60545901097356, 0.5785211548485334, 0.8156673791397561, 0.08971757370427048], [19, 409, 399.6015967302028, 114.74832975454635, 0.574313669882658, 0.81863533309155, 0.08871308999032158], [13, 408, 412.4199087359011, 118.01882639341056, 0.5723236143237963, 0.8200278534826403, 0.08739781245800435], [28, 422, 430.01258087716934, 123.27619893159694, 0.5733608941400256, 0.8193029263166043, 0.08555349254508228], [28, 437, 446.0132111277429, 128.12795895840983, 0.5745478194891973, 0.8184710154429481, 0.08396221015030943], [28, 463, 461.919537695125, 133.86531969774504, 0.579604492876413, 0.8148979272506922, 0.08232362998432714], [26, 486, 478.0187630884355, 140.19089486189196, 0.5865497578217701, 0.8099131938666155, 0.08067757512273749], [28, 501, 493.2074170149863, 146.85702034644783, 0.5955182962789285, 0.803341744712076, 0.07910272403520095], [21, 515, 507.0566467501217, 153.8215354632584, 0.6067232781550024, 0.7949131171045352, 0.07760463543746698], [25, 531, 523.5474086277183, 161.47481735236937, 0.6168488839458287, 0.7870816059182083, 0.07599551207531745], [17, 569, 542.0604638542986, 169.97594461347924, 0.627147545146061, 0.7789004792765685, 0.07429732722169809], [24, 586, 565.8530931267888, 180.54540387336192, 0.638135254773301, 0.7699242797934189, 0.0722983243760059], [26, 617, 591.2417580429465, 191.0182807231322, 0.6461596398583778, 0.763202279751634, 0.0704195671089873], [34, 637, 616.3925462942572, 200.19888348709765, 0.6495824282454107, 0.7602911737714665, 0.06883627430853646], [35, 656, 644.7653608489765, 209.5318293297662, 0.649947537671289, 0.7599790775245272, 0.0672908540976504], [42, 680, 671.3958571318558, 218.31285762805524, 0.6503253045399137, 0.7596558419936392, 0.06592880194467056], [33, 693, 697.5982532478866, 226.43712491206853, 0.6491906304461622, 0.7606257459098503, 0.06472005681496815], [31, 715, 724.8603003658354, 233.70891833864152, 0.6448385108708233, 0.7643188437412098, 0.06364527941907487], [37, 740, 755.4656619962318, 241.04938659127777, 0.6381478304502478, 0.7699138565395733, 0.06257052150534788], [41, 776, 785.6776376809897, 248.63344289269236, 0.632914648370444, 0.7742215754408557, 0.06152710913122554], [50, 806, 815.1952161300952, 256.04705881942107, 0.6281858719312187, 0.7780633073895797, 0.06055259052823077], [32, 841, 839.8752498216907, 259.58140424564476, 0.6181427641801686, 0.786065851625488, 0.05996228091940512], [43, 866, 864.6347760818899, 263.43896502256393, 0.6093647220999899, 0.7928900525671905, 0.05935348259204715], [50, 881, 889.0974369395549, 268.06845737453557, 0.6030125523638421, 0.797731697810511, 0.05870969399204847], [37, 911, 909.818261859937, 272.1450852917504, 0.5982405425351773, 0.8013165749360345, 0.05816755504775177], [49, 951, 936.4902138557305, 277.1928242782135, 0.5919823190398357, 0.8059509500858079, 0.05749879660696526], [50, 984, 967.3467242900265, 283.2556769056243, 0.5856342297815019, 0.810575443070063, 0.056736391389392155], [50, 1033, 1000.0107291210443, 290.45124765392393, 0.5808962628015301, 0.8139775991163493, 0.05591907362803516], [46, 1053, 1033.7052764605755, 299.2935648368669, 0.5790694343007576, 0.8152782287407152, 0.05504408126148415], [59, 1072, 1063.8112291172135, 306.146507791057, 0.5755654751738383, 0.8177556993307438, 0.054341994735028425], [57, 1081, 1092.8242930419751, 313.4538641214375, 0.5736583019195343, 0.8190947153039114, 0.05365966573897827], [50, 1108, 1120.1225627571314, 323.0672680001706, 0.5768427112207347, 0.8168552420787404, 0.052929263253276945], [53, 1128, 1149.296166934073, 333.51426347345114, 0.5803800152978018, 0.8143457729017344, 0.052172844500788115], [47, 1149, 1177.532691988352, 344.2393438816075, 0.584679043263476, 0.8112647017888204, 0.05144591250486983], [74, 1170, 1209.2702154651274, 357.10856229960814, 0.5906183047140572, 0.8069510630370916, 0.05063117337434862], [74, 1195, 1234.538417451084, 369.5928407043227, 0.5987547013197215, 0.8009324613520993, 0.04992311946841011], [70, 1213, 1251.2971808176467, 380.0908800622447, 0.6075149626947588, 0.7943082336863871, 0.049382192775437775], [79, 1219, 1260.773050757125, 388.1573619702831, 0.6157450172926604, 0.7879454763365686, 0.04899882842608863], [52, 1241, 1262.8039138142037, 391.7292228812354, 0.6204117972647818, 0.7842762280055932, 0.048845283819575175], [57, 1243, 1269.3849869515752, 396.47403398137504, 0.6246710620605438, 0.7808879972340153, 0.04861315035964239], [51, 1270, 1275.8706639830027, 399.1933622194459, 0.6257583522976338, 0.7800169770779034, 0.048462383839201026], [76, 1298, 1288.6035830840453, 402.9455626726154, 0.6253987928672933, 0.7803052927413312, 0.04823126808125301], [60, 1310, 1296.0470336526632, 405.4407792389393, 0.6256575088888267, 0.7800978666622723, 0.04808617529299053], [67, 1347, 1305.2835923705268, 407.7692213634026, 0.624797896406332, 0.7807865192523628, 0.047936882294557635], [68, 1366, 1313.2979664422573, 408.00886263214034, 0.6213500257484478, 0.7835331170425422, 0.047874374643938185], [69, 1353, 1323.3561954919246, 407.3600750004874, 0.6156469080481562, 0.7880221345944199, 0.047828515573033074], [50, 1353, 1335.6022728465473, 408.9514260843397, 0.612385040664461, 0.7905596511145694, 0.04768533292300888], [65, 1349, 1354.0705972202122, 413.54966126941144, 0.6108243722570929, 0.7917661184066468, 0.04739514694738575], [75, 1353, 1376.0786884970978, 420.55953491181134, 0.6112434389506183, 0.7914426437461026, 0.04700501117719202], [64, 1373, 1393.1938667841257, 428.3606017049395, 0.6149332292047961, 0.7885791803108676, 0.04663080875463468], [88, 1397, 1412.0869614254686, 438.5360522998502, 0.6211176284173864, 0.7837173544519495, 0.0461748062453867], [63, 1417, 1425.814054697005, 447.56049424894144, 0.6277964406011568, 0.7783775620921506, 0.0457951809933961], [79, 1464, 1444.0282679852098, 456.09104154538363, 0.6316926775702912, 0.7752189117301481, 0.04541302204602391], [75, 1544, 1469.1220184586969, 464.2550499569622, 0.6320170062443514, 0.7749545172575777, 0.04501582818422867], [82, 1777, 1517.2967131979808, 478.90687393993045, 0.6312633116176024, 0.7755685858810762, 0.044312976010300154], [64, 2041, 1628.5344078075004, 507.33896942231985, 0.623062020661082, 0.7821724352147307, 0.0429545185178125], [79, 1980, 1748.1958761658364, 533.8253935316565, 0.6107157679635403, 0.7918498915581879, 0.04171405951481975], [61, 1923, 1849.7212775517257, 553.4638306228444, 0.5984294361963594, 0.8011755175326487, 0.040791226144321506], [68, 1858, 1936.6869325205805, 568.4527247291061, 0.5870362578315846, 0.809560641330272, 0.04007292588088232], [81, 1792, 2011.5537233241027, 577.1882084183393, 0.5738730233508582, 0.8189442917989829, 0.039547354187059], [90, 1717, 2067.1792015753595, 579.0369228955358, 0.5602193776468555, 0.8283442816299127, 0.03923489071032605], [81, 1636, 2104.682952595497, 574.1445355836322, 0.5455876714120733, 0.8380537529318491, 0.03911097721933484], [133, 1573, 2120.6448727175593, 564.2893543466926, 0.5321865641969259, 0.8466271085243325, 0.039162299376758414], [194, 1488, 2101.3805918458793, 548.5198262819994, 0.5220566216424152, 0.8529108299226291, 0.03948712617020338], [422, 1407, 2015.6070777639507, 515.4856468744614, 0.511494182136257, 0.859286740058743, 0.04046897354210361]], 'totalCount': 4961, 'weightedCount': 89487.2330131929, 'minAxis': 0, 'maxAxis': 1}}, 'bounds': [0.4293314876469639, 0.6784962888641944], 'bestTools': ['DecisionTool1', 'DecisionTool2', 'DecisionTool1'], '_className': 'ConfidenceDecisionNode', '_version': 1}, 'state': {}, 'minCasesPerPartition': 70, 'curvesPerTool': {'DecisionTool1': {'name': 'ScalarAxisPerformanceCurve', 'description': 'average performance over ml', 'headerLabels': ['count', 'totalCount', 'weightedCount', 'weightedSum', 'weightedAverage', 'weightedStdDev', 'confidence95'], 'rowLabels': [0.005, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095, 0.105, 0.115, 0.125, 0.135, 0.145, 0.155, 0.165, 0.175, 0.185, 0.195, 0.205, 0.215, 0.225, 0.235, 0.245, 0.255, 0.265, 0.275, 0.285, 0.295, 0.305, 0.315, 0.325, 0.335, 0.345, 0.355, 0.365, 0.375, 0.385, 0.395, 0.405, 0.415, 0.425, 0.435, 0.445, 0.455, 0.465, 0.475, 0.485, 0.495, 0.505, 0.515, 0.525, 0.535, 0.545, 0.555, 0.565, 0.575, 0.585, 0.595, 0.605, 0.615, 0.625, 0.635, 0.645, 0.655, 0.665, 0.675, 0.685, 0.695, 0.705, 0.715, 0.725, 0.735, 0.745, 0.755, 0.765, 0.775, 0.785, 0.795, 0.805, 0.815, 0.825, 0.835, 0.845, 0.855, 0.865, 0.875, 0.885, 0.895, 0.905, 0.915, 0.925, 0.935, 0.945, 0.955, 0.965, 0.975, 0.985, 0.995], 'content': [[79, 362, 489.9275345206288, 241.61504637599123, 0.9863297298136152, 0.16478368877349603, 0.03594572584199289], [87, 388, 530.9121080279357, 261.9073331296448, 0.9866316068868548, 0.16296647597547023, 0.03433949416493374], [30, 422, 546.8421381860971, 269.67234820872545, 0.986289568332251, 0.1650238994781725, 0.034048541054768004], [28, 443, 554.7489493340258, 273.4156592473389, 0.9857275424336484, 0.16834848406718517, 0.0341438454693685], [26, 472, 557.8916530102493, 274.58701108545074, 0.9843739715546744, 0.17609055660561973, 0.034821636097482726], [24, 505, 556.0899019777773, 273.2861355692148, 0.9828847263625944, 0.18422164553365716, 0.03567417148120009], [18, 525, 549.6626098096364, 269.6997746914622, 0.9813284363106555, 0.19233954374512838, 0.036664204309253066], [18, 553, 540.8611569106579, 265.0990482419729, 0.9802850319523437, 0.19758860323964178, 0.037462272670385356], [15, 581, 530.0420757234103, 259.736701729894, 0.9800606918814925, 0.1986983649373336, 0.03794879979244316], [29, 609, 518.6310228511693, 253.65686661228565, 0.9781785332385609, 0.2077661115563823, 0.039229627251474344], [20, 587, 506.1339526101943, 246.73440327867837, 0.9749766914716512, 0.22230710984354957, 0.04107712236823064], [34, 535, 503.6596155658367, 244.04314388111237, 0.9690796575260137, 0.24674808483403338, 0.04338248265249042], [18, 504, 509.2044422701001, 244.92055702582002, 0.9619733713788202, 0.2731432458584077, 0.04539470649578341], [37, 502, 516.0882146239277, 246.19568364471186, 0.9540837270392393, 0.29954005040881465, 0.0472195180175456], [24, 507, 521.8740411460395, 246.95524228364226, 0.9464170386452891, 0.32294703739449726, 0.0487572133682356], [27, 492, 525.7403756707907, 246.48840954601752, 0.9376811101164662, 0.34749695787265616, 0.05039017994841065], [24, 496, 527.9288654267792, 244.7956073775887, 0.9273810295623643, 0.37411819791056355, 0.05217624058390236], [30, 504, 528.647108361125, 242.1415526419878, 0.9160801177657368, 0.400995284054955, 0.05398122967178537], [31, 505, 526.9675245165822, 237.74919580668225, 0.9023295924157123, 0.4310467569195905, 0.05605654623889823], [25, 504, 520.1420621842151, 230.94407788217086, 0.8880038538409109, 0.4598360094247618, 0.05827691755649791], [27, 494, 510.1043381586664, 222.62037299200847, 0.8728425004014103, 0.4880020179189978, 0.060622992692727286], [24, 481, 497.3211818471546, 213.81223781630368, 0.8598557456256355, 0.5105370669349899, 0.06279878370303994], [25, 462, 483.73101902753115, 205.14386024698615, 0.8481732705890856, 0.5297188905997349, 0.06485998339678045], [25, 462, 469.59355347305564, 196.4707230709495, 0.8367692512721967, 0.5475556776487375, 0.06692820099182197], [19, 452, 453.5758971214299, 187.1438356235625, 0.8251930352174818, 0.5648508251109838, 0.0691668457826484], [22, 435, 439.04738725721785, 178.6786908179519, 0.8139380668413917, 0.580951653191983, 0.07129685431458213], [24, 433, 424.69398812651576, 170.5758557364342, 0.8032882993654239, 0.5955903861737573, 0.07339928919231208], [24, 413, 409.28343667089933, 162.283869639039, 0.7930145962370319, 0.6092026347242905, 0.0756179454030248], [18, 413, 392.48070008605765, 154.02781770303864, 0.7848937166554462, 0.6196304169057547, 0.07787772978185734], [17, 410, 380.1750897437334, 148.6040327340364, 0.7817662795014092, 0.6235715550235787, 0.07937932303742552], [17, 407, 372.5680188298226, 145.24834966808552, 0.7797145344052218, 0.6261351649901539, 0.08035026858527253], [13, 410, 369.03065403029325, 143.0001410778612, 0.7750041332128604, 0.6319561642257971, 0.08110886555475127], [20, 419, 370.25391985848546, 142.88209294714034, 0.7718059703554319, 0.6358581163464933, 0.08122436941837312], [13, 411, 374.1286192186176, 143.93409005589783, 0.7694364058890222, 0.6387234278563646, 0.08098452305118117], [20, 410, 381.0866052798922, 145.63392781428993, 0.7643088253250355, 0.6448503853842895, 0.08062573952563568], [12, 413, 389.28726483807003, 147.70651754401624, 0.7588561501258306, 0.6512582770423737, 0.08016736352101365], [19, 409, 399.6015967302028, 150.89622004143897, 0.7552333187663356, 0.6554560505672258, 0.07938057717153063], [13, 408, 412.4199087359011, 154.52112149633467, 0.74933880844867, 0.6621867940035713, 0.07853739970886], [28, 422, 430.01258087716934, 159.08583817910394, 0.739912482814292, 0.6727031423856976, 0.0775224016266612], [28, 437, 446.0132111277429, 161.58788046520198, 0.7245878661604106, 0.6891824317356787, 0.0770458603115016], [28, 463, 461.919537695125, 162.96709458697592, 0.7056081472550185, 0.7086022456409097, 0.07676692904095223], [26, 486, 478.0187630884355, 161.08101063817725, 0.6739526691272434, 0.7387745256681918, 0.07705300053697492], [28, 501, 493.2074170149863, 154.65605549141765, 0.627144078357274, 0.7789032706192763, 0.07789023884274023], [21, 515, 507.0566467501217, 143.32853297963766, 0.5653353876663417, 0.8248611394969151, 0.07905298134485933], [25, 531, 523.5474086277183, 130.62247381657335, 0.4989900500470466, 0.8666077139940804, 0.07974240679565908], [17, 569, 542.0604638542986, 115.37009288091225, 0.4256724132233439, 0.9048773378865304, 0.08008055253691054], [24, 586, 565.8530931267888, 97.93049549441767, 0.3461339937306827, 0.9381850874875636, 0.07980837847334103], [26, 617, 591.2417580429465, 79.3060391647741, 0.26826941123807935, 0.9633439276779471, 0.07911597847144045], [34, 637, 616.3925462942572, 59.66697278041371, 0.1936005655458706, 0.9810804355506831, 0.07819512707742302], [35, 656, 644.7653608489765, 39.92841162886466, 0.1238540841471076, 0.9923004413180938, 0.0768912311748369], [42, 680, 671.3958571318558, 20.338109989277918, 0.06058455611019269, 0.9981630686220205, 0.07557314422112892], [33, 693, 697.5982532478866, 6.110402605682534, 0.01751839995938535, 0.9998465410566082, 0.07420275898301724], [31, 715, 724.8603003658354, -1.5990875698626665, -0.0044121262236478135, 0.9999902665237228, 0.07279923245118869], [37, 740, 755.4656619962318, -2.465607574768378, -0.006527384893320741, 0.9999786963963048, 0.07130895640568517], [41, 776, 785.6776376809897, 1.1107178417965728, 0.0028274136580365796, 0.9999960028580162, 0.06992508569304007], [50, 806, 815.1952161300952, 9.673824203945758, 0.023733760975363557, 0.9997183146216541, 0.06863791527967196], [32, 841, 839.8752498216907, 22.955187398567688, 0.054663326258134595, 0.9985048426334244, 0.06758086830571432], [43, 866, 864.6347760818899, 40.01236262731253, 0.09255321144640831, 0.9957077397765652, 0.06651286623878065], [50, 881, 889.0974369395549, 59.591487261094315, 0.13404939612967443, 0.9909746512385001, 0.06543538544986666], [37, 911, 909.818261859937, 82.72002556230916, 0.18183856937143691, 0.9833283961571285, 0.06443592047531649], [49, 951, 936.4902138557305, 109.90325056519305, 0.23471307855465537, 0.9720646947376467, 0.06314689992773344], [50, 984, 967.3467242900265, 138.4479255741469, 0.2862426100129904, 0.9581571730217093, 0.061685539338492815], [50, 1033, 1000.0107291210443, 167.67131369840354, 0.33533902950376865, 0.9420975189923122, 0.060159147259183594], [46, 1053, 1033.7052764605755, 199.5093286575739, 0.38600814603693856, 0.9224953719087812, 0.05855173961596633], [59, 1072, 1063.8112291172135, 230.88664428628854, 0.43407446352655266, 0.9008769949966174, 0.057036981611155446], [57, 1081, 1092.8242930419751, 262.4158067006632, 0.48025251336645364, 0.8771302773278359, 0.05552811774468626], [50, 1108, 1120.1225627571314, 293.19922421798117, 0.5235127547047788, 0.8520178376427436, 0.05405646505773237], [53, 1128, 1149.296166934073, 325.66694352030754, 0.5667241445502685, 0.8239076064606795, 0.05247825051646886], [47, 1149, 1177.532691988352, 359.1953733421878, 0.6100813604345192, 0.7923387745468262, 0.05084228249828954], [74, 1170, 1209.2702154651274, 391.41655239872415, 0.6473599488236328, 0.7621844243088803, 0.04920672124311637], [74, 1195, 1234.538417451084, 420.90935294889107, 0.6818894365684148, 0.7314552592581545, 0.047708707406724754], [70, 1213, 1251.2971808176467, 447.36381395030565, 0.7150400733069351, 0.6990834668086567, 0.04632765885904326], [79, 1219, 1260.773050757125, 469.43411736842245, 0.744676636427969, 0.6674254319085593, 0.045096100367743926], [52, 1241, 1262.8039138142037, 485.6976759171134, 0.7692368872220237, 0.6389637011106143, 0.04408858953329172], [57, 1243, 1269.3849869515752, 502.14556728321935, 0.7911635515543959, 0.6116046392007103, 0.043022414290948656], [51, 1270, 1275.8706639830027, 517.3716590775182, 0.8110095696728249, 0.5850328861688885, 0.04197037844201896], [76, 1298, 1288.6035830840453, 533.5592366721472, 0.8281200575201992, 0.5605507740898605, 0.040879340650138846], [60, 1310, 1296.0470336526632, 545.3773929905146, 0.8416012364203661, 0.5400993971999142, 0.040011288414458224], [67, 1347, 1305.2835923705268, 556.3458248073248, 0.8524520312048735, 0.5228054461218689, 0.03922596923885498], [68, 1366, 1313.2979664422573, 568.3602235276248, 0.8655464914292386, 0.5008285846220577, 0.03827533403311521], [69, 1353, 1323.3561954919246, 581.1868788139897, 0.8783529042200886, 0.47801273586394777, 0.037250956424000115], [50, 1353, 1335.6022728465473, 593.6967872209833, 0.889032310428234, 0.45784446159655146, 0.036289123537079096], [65, 1349, 1354.0705972202122, 608.9988714084029, 0.899508301352417, 0.43690366878533876, 0.035206938632476786], [75, 1353, 1376.0786884970978, 625.3224413577477, 0.9088469236315283, 0.41712979922981713, 0.034124794528731926], [64, 1373, 1393.1938667841257, 638.7799591064457, 0.9170008199661767, 0.39888531707918323, 0.03316456608837965], [88, 1397, 1412.0869614254686, 652.3854782870034, 0.9240018442326463, 0.3823880121743704, 0.03225354716590866], [63, 1417, 1425.814054697005, 663.6117048721741, 0.9308530838029871, 0.3653936731450976, 0.031376546252019015], [79, 1464, 1444.0282679852098, 677.4854028476402, 0.9383270644596262, 0.3457489263939665, 0.030328340266897933], [75, 1544, 1469.1220184586969, 694.687562087553, 0.9457179912344817, 0.32498843218708423, 0.029151512243808426], [82, 1777, 1517.2967131979808, 722.4499407343583, 0.9522856465057025, 0.3052082034598922, 0.027798343305251258], [64, 2041, 1628.5344078075004, 781.1165326649259, 0.9592877238824138, 0.2824306336226629, 0.02581151535890055], [79, 1980, 1748.1958761658364, 843.5172747151844, 0.9650146030148479, 0.2621961402616266, 0.024003477949933835], [61, 1923, 1849.7212775517257, 896.5488830162212, 0.9693880844609034, 0.24553358569698794, 0.022581790724225836], [68, 1858, 1936.6869325205805, 942.6786286026306, 0.9734961420695321, 0.2287034354261839, 0.02129917120194863], [81, 1792, 2011.5537233241027, 983.0984506178714, 0.9774518465192132, 0.21115844225647537, 0.020081422713857554], [90, 1717, 2067.1792015753595, 1013.9861293192897, 0.9810336022600744, 0.19383774461291153, 0.018979561831458693], [81, 1636, 2104.682952595497, 1035.768424446506, 0.9842512604277954, 0.1767751576050678, 0.01796277153465826], [133, 1573, 2120.6448727175593, 1046.4309627376497, 0.9868988213916945, 0.1613403741649442, 0.01709596530705885], [194, 1488, 2101.3805918458793, 1039.3361941231385, 0.9891936740599402, 0.14661471685951744, 0.016371651673724754], [422, 1407, 2015.6070777639507, 998.7075674671568, 0.9909744597395348, 0.13405081179887965, 0.015984087864021973]], 'totalCount': 4961, 'weightedCount': 89487.2330131929, 'minAxis': 0, 'maxAxis': 1}, 'DecisionTool2': {'name': 'ScalarAxisPerformanceCurve', 'description': 'average performance over ml', 'headerLabels': ['count', 'totalCount', 'weightedCount', 'weightedSum', 'weightedAverage', 'weightedStdDev', 'confidence95'], 'rowLabels': [0.005, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095, 0.105, 0.115, 0.125, 0.135, 0.145, 0.155, 0.165, 0.175, 0.185, 0.195, 0.205, 0.215, 0.225, 0.235, 0.245, 0.255, 0.265, 0.275, 0.285, 0.295, 0.305, 0.315, 0.325, 0.335, 0.345, 0.355, 0.365, 0.375, 0.385, 0.395, 0.405, 0.415, 0.425, 0.435, 0.445, 0.455, 0.465, 0.475, 0.485, 0.495, 0.505, 0.515, 0.525, 0.535, 0.545, 0.555, 0.565, 0.575, 0.585, 0.595, 0.605, 0.615, 0.625, 0.635, 0.645, 0.655, 0.665, 0.675, 0.685, 0.695, 0.705, 0.715, 0.725, 0.735, 0.745, 0.755, 0.765, 0.775, 0.785, 0.795, 0.805, 0.815, 0.825, 0.835, 0.845, 0.855, 0.865, 0.875, 0.885, 0.895, 0.905, 0.915, 0.925, 0.935, 0.945, 0.955, 0.965, 0.975, 0.985, 0.995], 'content': [[79, 362, 489.9275345206288, 148.1678810536863, 0.6048563128776244, 0.7963346286405478, 0.07902016789047377], [87, 388, 530.9121080279357, 159.1045597255232, 0.5993630859786357, 0.8004772896005017, 0.076106064329513], [30, 422, 546.8421381860971, 164.0264195278287, 0.5999040969004787, 0.8000719183436079, 0.07497036041395001], [28, 443, 554.7489493340258, 166.28920543044805, 0.5995115651145537, 0.8003660932934984, 0.07444785136496639], [26, 472, 557.8916530102493, 167.00051197856666, 0.5986843899795669, 0.8009850193323178, 0.07426656484114247], [24, 505, 556.0899019777773, 166.4980591505767, 0.5988170565889195, 0.8008858425132654, 0.0743821753229064], [18, 525, 549.6626098096364, 164.19292170107357, 0.5974316563316475, 0.8019198314126079, 0.07486407284157405], [18, 553, 540.8611569106579, 160.32137282192707, 0.5928374436710002, 0.8053221500627142, 0.07563067859969748], [15, 581, 530.0420757234103, 155.5509178310632, 0.5869379996626294, 0.8096318821242343, 0.07660280967339962], [29, 609, 518.6310228511693, 150.35049314722397, 0.5797975305089677, 0.8147605928220278, 0.0776858341715802], [20, 587, 506.1339526101943, 145.0126339904964, 0.5730207714485409, 0.8195408443076648, 0.07886941815367608], [34, 535, 503.6596155658367, 142.9725192643703, 0.5677346955989224, 0.8232115860537911, 0.0792397768077082], [18, 504, 509.2044422701001, 142.6385049410164, 0.560240614968382, 0.8283299181726139, 0.07905178000534215], [37, 502, 516.0882146239277, 142.50689470171932, 0.5522578918240315, 0.8336733298589305, 0.078775660749677], [24, 507, 521.8740411460395, 143.2850566834211, 0.5491173938016383, 0.8357452290168916, 0.07843504985452932], [27, 492, 525.7403756707907, 143.9432636484504, 0.5475830668884564, 0.8367513279684903, 0.07819313234744232], [24, 496, 527.9288654267792, 144.83933045566084, 0.5487077518997652, 0.8360142361258601, 0.07799651596705069], [30, 504, 528.647108361125, 146.8213930055499, 0.5554608762004407, 0.8315426717917963, 0.07773478668469941], [31, 505, 526.9675245165822, 149.60463769137866, 0.5677945252076763, 0.8231703208596566, 0.07746561932463569], [25, 504, 520.1420621842151, 150.69026546627308, 0.5794196486763039, 0.8150293680155508, 0.07758570535134572], [27, 494, 510.1043381586664, 151.1899147786198, 0.5927803528367275, 0.8053641743278411, 0.0778794213783042], [24, 481, 497.3211818471546, 150.78963886871932, 0.6064074661314652, 0.7951540637008762, 0.0783724158521079], [25, 462, 483.73101902753115, 149.59767850115895, 0.6185159628667299, 0.7857722339705328, 0.07899551631961815], [25, 462, 469.59355347305564, 148.5985958851874, 0.6328817539600815, 0.7742484649674223, 0.07958572642444017], [19, 452, 453.5758971214299, 145.34520131051556, 0.640885912293546, 0.7676361425986067, 0.0806322529204644], [22, 435, 439.04738725721785, 141.69793803244818, 0.6454790172771661, 0.7637780032541556, 0.08174928184206795], [24, 433, 424.69398812651576, 137.77638399302947, 0.6488266273832257, 0.7609362703922778, 0.08296447273393764], [24, 413, 409.28343667089933, 132.72827414423224, 0.6485885440360867, 0.7611392123292222, 0.08452322024347231], [18, 413, 392.48070008605765, 126.08692007288343, 0.6425127148684604, 0.7662750232340596, 0.0866042596789703], [17, 410, 380.1750897437334, 121.02026496082532, 0.6366554160210869, 0.7711484171357781, 0.08827408579872578], [17, 407, 372.5680188298226, 116.39914897680289, 0.6248477759438089, 0.780746602232809, 0.08972394006646871], [13, 410, 369.03065403029325, 112.93680653758341, 0.6120727657942073, 0.7908014475031195, 0.09073160171549284], [20, 419, 370.25391985848546, 111.30680208466947, 0.6012457727778384, 0.7990641530670611, 0.0910535877689523], [13, 411, 374.1286192186176, 110.35765837691714, 0.5899450226898091, 0.8074434161001751, 0.09105455046822379], [20, 410, 381.0866052798922, 111.1984538961202, 0.5835862628362368, 0.8120511522243141, 0.09047652724688443], [12, 413, 389.28726483807003, 112.60545901097356, 0.5785211548485334, 0.8156673791397561, 0.08971757370427048], [19, 409, 399.6015967302028, 114.74832975454635, 0.574313669882658, 0.81863533309155, 0.08871308999032158], [13, 408, 412.4199087359011, 118.01882639341056, 0.5723236143237963, 0.8200278534826403, 0.08739781245800435], [28, 422, 430.01258087716934, 123.27619893159694, 0.5733608941400256, 0.8193029263166043, 0.08555349254508228], [28, 437, 446.0132111277429, 128.12795895840983, 0.5745478194891973, 0.8184710154429481, 0.08396221015030943], [28, 463, 461.919537695125, 133.86531969774504, 0.579604492876413, 0.8148979272506922, 0.08232362998432714], [26, 486, 478.0187630884355, 140.19089486189196, 0.5865497578217701, 0.8099131938666155, 0.08067757512273749], [28, 501, 493.2074170149863, 146.85702034644783, 0.5955182962789285, 0.803341744712076, 0.07910272403520095], [21, 515, 507.0566467501217, 153.8215354632584, 0.6067232781550024, 0.7949131171045352, 0.07760463543746698], [25, 531, 523.5474086277183, 161.47481735236937, 0.6168488839458287, 0.7870816059182083, 0.07599551207531745], [17, 569, 542.0604638542986, 169.97594461347924, 0.627147545146061, 0.7789004792765685, 0.07429732722169809], [24, 586, 565.8530931267888, 180.54540387336192, 0.638135254773301, 0.7699242797934189, 0.0722983243760059], [26, 617, 591.2417580429465, 191.0182807231322, 0.6461596398583778, 0.763202279751634, 0.0704195671089873], [34, 637, 616.3925462942572, 200.19888348709765, 0.6495824282454107, 0.7602911737714665, 0.06883627430853646], [35, 656, 644.7653608489765, 209.5318293297662, 0.649947537671289, 0.7599790775245272, 0.0672908540976504], [42, 680, 671.3958571318558, 218.31285762805524, 0.6503253045399137, 0.7596558419936392, 0.06592880194467056], [33, 693, 697.5982532478866, 226.43712491206853, 0.6491906304461622, 0.7606257459098503, 0.06472005681496815], [31, 715, 724.8603003658354, 233.70891833864152, 0.6448385108708233, 0.7643188437412098, 0.06364527941907487], [37, 740, 755.4656619962318, 241.04938659127777, 0.6381478304502478, 0.7699138565395733, 0.06257052150534788], [41, 776, 785.6776376809897, 248.63344289269236, 0.632914648370444, 0.7742215754408557, 0.06152710913122554], [50, 806, 815.1952161300952, 256.04705881942107, 0.6281858719312187, 0.7780633073895797, 0.06055259052823077], [32, 841, 839.8752498216907, 259.58140424564476, 0.6181427641801686, 0.786065851625488, 0.05996228091940512], [43, 866, 864.6347760818899, 263.43896502256393, 0.6093647220999899, 0.7928900525671905, 0.05935348259204715], [50, 881, 889.0974369395549, 268.06845737453557, 0.6030125523638421, 0.797731697810511, 0.05870969399204847], [37, 911, 909.818261859937, 272.1450852917504, 0.5982405425351773, 0.8013165749360345, 0.05816755504775177], [49, 951, 936.4902138557305, 277.1928242782135, 0.5919823190398357, 0.8059509500858079, 0.05749879660696526], [50, 984, 967.3467242900265, 283.2556769056243, 0.5856342297815019, 0.810575443070063, 0.056736391389392155], [50, 1033, 1000.0107291210443, 290.45124765392393, 0.5808962628015301, 0.8139775991163493, 0.05591907362803516], [46, 1053, 1033.7052764605755, 299.2935648368669, 0.5790694343007576, 0.8152782287407152, 0.05504408126148415], [59, 1072, 1063.8112291172135, 306.146507791057, 0.5755654751738383, 0.8177556993307438, 0.054341994735028425], [57, 1081, 1092.8242930419751, 313.4538641214375, 0.5736583019195343, 0.8190947153039114, 0.05365966573897827], [50, 1108, 1120.1225627571314, 323.0672680001706, 0.5768427112207347, 0.8168552420787404, 0.052929263253276945], [53, 1128, 1149.296166934073, 333.51426347345114, 0.5803800152978018, 0.8143457729017344, 0.052172844500788115], [47, 1149, 1177.532691988352, 344.2393438816075, 0.584679043263476, 0.8112647017888204, 0.05144591250486983], [74, 1170, 1209.2702154651274, 357.10856229960814, 0.5906183047140572, 0.8069510630370916, 0.05063117337434862], [74, 1195, 1234.538417451084, 369.5928407043227, 0.5987547013197215, 0.8009324613520993, 0.04992311946841011], [70, 1213, 1251.2971808176467, 380.0908800622447, 0.6075149626947588, 0.7943082336863871, 0.049382192775437775], [79, 1219, 1260.773050757125, 388.1573619702831, 0.6157450172926604, 0.7879454763365686, 0.04899882842608863], [52, 1241, 1262.8039138142037, 391.7292228812354, 0.6204117972647818, 0.7842762280055932, 0.048845283819575175], [57, 1243, 1269.3849869515752, 396.47403398137504, 0.6246710620605438, 0.7808879972340153, 0.04861315035964239], [51, 1270, 1275.8706639830027, 399.1933622194459, 0.6257583522976338, 0.7800169770779034, 0.048462383839201026], [76, 1298, 1288.6035830840453, 402.9455626726154, 0.6253987928672933, 0.7803052927413312, 0.04823126808125301], [60, 1310, 1296.0470336526632, 405.4407792389393, 0.6256575088888267, 0.7800978666622723, 0.04808617529299053], [67, 1347, 1305.2835923705268, 407.7692213634026, 0.624797896406332, 0.7807865192523628, 0.047936882294557635], [68, 1366, 1313.2979664422573, 408.00886263214034, 0.6213500257484478, 0.7835331170425422, 0.047874374643938185], [69, 1353, 1323.3561954919246, 407.3600750004874, 0.6156469080481562, 0.7880221345944199, 0.047828515573033074], [50, 1353, 1335.6022728465473, 408.9514260843397, 0.612385040664461, 0.7905596511145694, 0.04768533292300888], [65, 1349, 1354.0705972202122, 413.54966126941144, 0.6108243722570929, 0.7917661184066468, 0.04739514694738575], [75, 1353, 1376.0786884970978, 420.55953491181134, 0.6112434389506183, 0.7914426437461026, 0.04700501117719202], [64, 1373, 1393.1938667841257, 428.3606017049395, 0.6149332292047961, 0.7885791803108676, 0.04663080875463468], [88, 1397, 1412.0869614254686, 438.5360522998502, 0.6211176284173864, 0.7837173544519495, 0.0461748062453867], [63, 1417, 1425.814054697005, 447.56049424894144, 0.6277964406011568, 0.7783775620921506, 0.0457951809933961], [79, 1464, 1444.0282679852098, 456.09104154538363, 0.6316926775702912, 0.7752189117301481, 0.04541302204602391], [75, 1544, 1469.1220184586969, 464.2550499569622, 0.6320170062443514, 0.7749545172575777, 0.04501582818422867], [82, 1777, 1517.2967131979808, 478.90687393993045, 0.6312633116176024, 0.7755685858810762, 0.044312976010300154], [64, 2041, 1628.5344078075004, 507.33896942231985, 0.623062020661082, 0.7821724352147307, 0.0429545185178125], [79, 1980, 1748.1958761658364, 533.8253935316565, 0.6107157679635403, 0.7918498915581879, 0.04171405951481975], [61, 1923, 1849.7212775517257, 553.4638306228444, 0.5984294361963594, 0.8011755175326487, 0.040791226144321506], [68, 1858, 1936.6869325205805, 568.4527247291061, 0.5870362578315846, 0.809560641330272, 0.04007292588088232], [81, 1792, 2011.5537233241027, 577.1882084183393, 0.5738730233508582, 0.8189442917989829, 0.039547354187059], [90, 1717, 2067.1792015753595, 579.0369228955358, 0.5602193776468555, 0.8283442816299127, 0.03923489071032605], [81, 1636, 2104.682952595497, 574.1445355836322, 0.5455876714120733, 0.8380537529318491, 0.03911097721933484], [133, 1573, 2120.6448727175593, 564.2893543466926, 0.5321865641969259, 0.8466271085243325, 0.039162299376758414], [194, 1488, 2101.3805918458793, 548.5198262819994, 0.5220566216424152, 0.8529108299226291, 0.03948712617020338], [422, 1407, 2015.6070777639507, 515.4856468744614, 0.511494182136257, 0.859286740058743, 0.04046897354210361]], 'totalCount': 4961, 'weightedCount': 89487.2330131929, 'minAxis': 0, 'maxAxis': 1}, 'CompositeDecisionTool0': {'name': 'ScalarAxisPerformanceCurve', 'description': 'average performance over ml', 'headerLabels': ['count', 'totalCount', 'weightedCount', 'weightedSum', 'weightedAverage', 'weightedStdDev', 'confidence95'], 'rowLabels': [0.005, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095, 0.105, 0.115, 0.125, 0.135, 0.145, 0.155, 0.165, 0.175, 0.185, 0.195, 0.205, 0.215, 0.225, 0.235, 0.245, 0.255, 0.265, 0.275, 0.285, 0.295, 0.305, 0.315, 0.325, 0.335, 0.345, 0.355, 0.365, 0.375, 0.385, 0.395, 0.405, 0.415, 0.425, 0.435, 0.445, 0.455, 0.465, 0.475, 0.485, 0.495, 0.505, 0.515, 0.525, 0.535, 0.545, 0.555, 0.565, 0.575, 0.585, 0.595, 0.605, 0.615, 0.625, 0.635, 0.645, 0.655, 0.665, 0.675, 0.685, 0.695, 0.705, 0.715, 0.725, 0.735, 0.745, 0.755, 0.765, 0.775, 0.785, 0.795, 0.805, 0.815, 0.825, 0.835, 0.845, 0.855, 0.865, 0.875, 0.885, 0.895, 0.905, 0.915, 0.925, 0.935, 0.945, 0.955, 0.965, 0.975, 0.985, 0.995], 'content': [[79, 362, 489.9275345206288, 241.61504637599123, 0.9863297298136152, 0.16478368877349603, 0.03594572584199289], [87, 388, 530.9121080279357, 261.9073331296448, 0.9866316068868548, 0.16296647597547023, 0.03433949416493374], [30, 422, 546.8421381860971, 269.67234820872545, 0.986289568332251, 0.1650238994781725, 0.034048541054768004], [28, 443, 554.7489493340258, 273.4156592473389, 0.9857275424336484, 0.16834848406718517, 0.0341438454693685], [26, 472, 557.8916530102493, 274.58701108545074, 0.9843739715546744, 0.17609055660561973, 0.034821636097482726], [24, 505, 556.0899019777773, 273.2861355692148, 0.9828847263625944, 0.18422164553365716, 0.03567417148120009], [18, 525, 549.6626098096364, 269.6997746914622, 0.9813284363106555, 0.19233954374512838, 0.036664204309253066], [18, 553, 540.8611569106579, 265.0990482419729, 0.9802850319523437, 0.19758860323964178, 0.037462272670385356], [15, 581, 530.0420757234103, 259.736701729894, 0.9800606918814925, 0.1986983649373336, 0.03794879979244316], [29, 609, 518.6310228511693, 253.65686661228565, 0.9781785332385609, 0.2077661115563823, 0.039229627251474344], [20, 587, 506.1339526101943, 246.73440327867837, 0.9749766914716512, 0.22230710984354957, 0.04107712236823064], [34, 535, 503.6596155658367, 244.04314388111237, 0.9690796575260137, 0.24674808483403338, 0.04338248265249042], [18, 504, 509.2044422701001, 244.92055702582002, 0.9619733713788202, 0.2731432458584077, 0.04539470649578341], [37, 502, 516.0882146239277, 246.19568364471186, 0.9540837270392393, 0.29954005040881465, 0.0472195180175456], [24, 507, 521.8740411460395, 246.95524228364226, 0.9464170386452891, 0.32294703739449726, 0.0487572133682356], [27, 492, 525.7403756707907, 246.48840954601752, 0.9376811101164662, 0.34749695787265616, 0.05039017994841065], [24, 496, 527.9288654267792, 244.7956073775887, 0.9273810295623643, 0.37411819791056355, 0.05217624058390236], [30, 504, 528.647108361125, 242.1415526419878, 0.9160801177657368, 0.400995284054955, 0.05398122967178537], [31, 505, 526.9675245165822, 237.74919580668225, 0.9023295924157123, 0.4310467569195905, 0.05605654623889823], [25, 504, 520.1420621842151, 230.94407788217086, 0.8880038538409109, 0.4598360094247618, 0.05827691755649791], [27, 494, 510.1043381586664, 222.62037299200847, 0.8728425004014103, 0.4880020179189978, 0.060622992692727286], [24, 481, 497.3211818471546, 213.81223781630368, 0.8598557456256355, 0.5105370669349899, 0.06279878370303994], [25, 462, 483.73101902753115, 205.14386024698615, 0.8481732705890856, 0.5297188905997349, 0.06485998339678045], [25, 462, 469.59355347305564, 196.4707230709495, 0.8367692512721967, 0.5475556776487375, 0.06692820099182197], [19, 452, 453.5758971214299, 187.1438356235625, 0.8251930352174818, 0.5648508251109838, 0.0691668457826484], [22, 435, 439.04738725721785, 178.6786908179519, 0.8139380668413917, 0.580951653191983, 0.07129685431458213], [24, 433, 424.69398812651576, 170.5758557364342, 0.8032882993654239, 0.5955903861737573, 0.07339928919231208], [24, 413, 409.28343667089933, 162.283869639039, 0.7930145962370319, 0.6092026347242905, 0.0756179454030248], [18, 413, 392.48070008605765, 154.02781770303864, 0.7848937166554462, 0.6196304169057547, 0.07787772978185734], [17, 410, 380.1750897437334, 148.6040327340364, 0.7817662795014092, 0.6235715550235787, 0.07937932303742552], [17, 407, 372.5680188298226, 145.24834966808552, 0.7797145344052218, 0.6261351649901539, 0.08035026858527253], [13, 410, 369.03065403029325, 143.0001410778612, 0.7750041332128604, 0.6319561642257971, 0.08110886555475127], [20, 419, 370.25391985848546, 142.88209294714034, 0.7718059703554319, 0.6358581163464933, 0.08122436941837312], [13, 411, 374.1286192186176, 143.96890824027358, 0.7696225353781185, 0.6384991409846665, 0.08097030299961869], [20, 410, 381.0866052798922, 146.05761223845187, 0.7665323851053681, 0.6422056544321907, 0.08046023410203769], [12, 413, 389.28726483807003, 148.04167903028423, 0.7605780738389396, 0.6492464813885761, 0.08004344563440681], [19, 409, 399.6015967302028, 151.2525695573539, 0.7570168427503777, 0.6533953625426555, 0.07925569665833772], [13, 408, 412.4199087359011, 155.2280631456524, 0.7527670699575993, 0.658286972670317, 0.07830579277457875], [28, 422, 430.01258087716934, 161.03150731157498, 0.7489618419214238, 0.6626131294697306, 0.0769388178048718], [28, 437, 446.0132111277429, 165.22077328097063, 0.7408783827869603, 0.6716392051682047, 0.07605893314405547], [28, 463, 461.919537695125, 169.21027061436334, 0.7326395911230977, 0.6806168007910032, 0.07523574712453841], [26, 486, 478.0187630884355, 173.40314903818074, 0.725507709855734, 0.6882140386099999, 0.0743695858782852], [28, 501, 493.2074170149863, 176.8190134484321, 0.7170168466588952, 0.6970558382277094, 0.07368431599238254], [21, 515, 507.0566467501217, 179.3153247881679, 0.7072792593784292, 0.7069342609133483, 0.07318420277741297], [25, 531, 523.5474086277183, 182.8498078968376, 0.6985033442381438, 0.7156067901355678, 0.07246282989902537], [17, 569, 542.0604638542986, 187.70971515160045, 0.6925785135366569, 0.7213425001948471, 0.07149949354436214], [24, 586, 565.8530931267888, 194.54531948585085, 0.6876177645714829, 0.7260728681377082, 0.07020924913609594], [26, 617, 591.2417580429465, 201.7208069534972, 0.6823631930911235, 0.7310133191088152, 0.0689185555312819], [34, 637, 616.3925462942572, 207.74199004676169, 0.6740574372474276, 0.738678936542412, 0.06785084174403039], [35, 656, 644.7653608489765, 214.47138941604626, 0.6652695769315125, 0.7466032346630767, 0.06669605646701358], [42, 680, 671.3958571318558, 221.1346448699015, 0.6587310377951073, 0.7523785083622323, 0.06561225038369023], [33, 693, 697.5982532478866, 227.52178004793848, 0.6523003146542863, 0.7579606187012101, 0.06460657226400882], [31, 715, 724.8603003658354, 233.7880270536989, 0.645056783867751, 0.7641346383892003, 0.06363760951295087], [37, 740, 755.4656619962318, 241.04938659127777, 0.6381478304502478, 0.7699138565395733, 0.06257052150534788], [41, 776, 785.6776376809897, 248.63344289269236, 0.632914648370444, 0.7742215754408557, 0.06152710913122554], [50, 806, 815.1952161300952, 256.04705881942107, 0.6281858719312187, 0.7780633073895797, 0.06055259052823077], [32, 841, 839.8752498216907, 259.58140424564476, 0.6181427641801686, 0.786065851625488, 0.05996228091940512], [43, 866, 864.6347760818899, 263.43896502256393, 0.6093647220999899, 0.7928900525671905, 0.05935348259204715], [50, 881, 889.0974369395549, 268.15637753512715, 0.6032103263241275, 0.7975821601665474, 0.05870419106367778], [37, 911, 909.818261859937, 273.68732374478094, 0.6016307546636473, 0.7987743329895194, 0.058075211099945075], [49, 951, 936.4902138557305, 280.9325849259276, 0.5999690776677058, 0.8000231908154665, 0.057286954895827], [50, 984, 967.3467242900265, 290.4106876937673, 0.6004272933407844, 0.7996793516225507, 0.05635376412253526], [50, 1033, 1000.0107291210443, 302.5601427843794, 0.6051137932296258, 0.7961389936708628, 0.05530293612444094], [46, 1053, 1033.7052764605755, 316.9316243665288, 0.613195330591149, 0.7899313176100861, 0.05418166801381816], [59, 1072, 1063.8112291172135, 330.4163864511993, 0.6211936430214032, 0.7836571047784853, 0.05319696112131574], [57, 1081, 1092.8242930419751, 344.66175691671754, 0.630772502242461, 0.775967815321477, 0.05222792301437146], [50, 1108, 1120.1225627571314, 362.4288526900108, 0.6471235644033604, 0.7623851339014236, 0.051134087265929835], [53, 1128, 1149.296166934073, 382.89360097050667, 0.6663097154355524, 0.745675105603099, 0.04992463683905899], [47, 1149, 1177.532691988352, 404.6097966693358, 0.6872162436290782, 0.7264529127839866, 0.048682550094345294], [74, 1170, 1209.2702154651274, 426.2733836133016, 0.7050093158034857, 0.7091980433068736, 0.047465510656369135], [74, 1195, 1234.538417451084, 446.84030768014657, 0.72389858648988, 0.6899063968959533, 0.04633389953268394], [70, 1213, 1251.2971808176467, 465.667535988426, 0.7442956687301742, 0.6678502508118872, 0.04528093490095946], [79, 1219, 1260.773050757125, 480.9952157018706, 0.763016334165806, 0.6463792027874777, 0.04437938709609405], [52, 1241, 1262.8039138142037, 492.738776049206, 0.7803884208133721, 0.6252950604797816, 0.04361447114526356], [57, 1243, 1269.3849869515752, 506.4420815629872, 0.7979329939598648, 0.6027461631153216, 0.042709709727859695], [51, 1270, 1275.8706639830027, 520.077690489032, 0.8152514281744792, 0.5791071652634537, 0.041757281247810134], [76, 1298, 1288.6035830840453, 534.7322178456922, 0.8299406037129043, 0.5578517673259261, 0.04078080653458931], [60, 1310, 1296.0470336526632, 545.6024237070233, 0.8419484934421648, 0.5395579064293922, 0.03999122620311459], [67, 1347, 1305.2835923705268, 556.3458248073248, 0.8524520312048735, 0.5228054461218689, 0.03922596923885498], [68, 1366, 1313.2979664422573, 568.3602235276248, 0.8655464914292386, 0.5008285846220577, 0.03827533403311521], [69, 1353, 1323.3561954919246, 581.1868788139897, 0.8783529042200886, 0.47801273586394777, 0.037250956424000115], [50, 1353, 1335.6022728465473, 593.6967872209833, 0.889032310428234, 0.45784446159655146, 0.036289123537079096], [65, 1349, 1354.0705972202122, 608.9988714084029, 0.899508301352417, 0.43690366878533876, 0.035206938632476786], [75, 1353, 1376.0786884970978, 625.3224413577477, 0.9088469236315283, 0.41712979922981713, 0.034124794528731926], [64, 1373, 1393.1938667841257, 638.7799591064457, 0.9170008199661767, 0.39888531707918323, 0.03316456608837965], [88, 1397, 1412.0869614254686, 652.3854782870034, 0.9240018442326463, 0.3823880121743704, 0.03225354716590866], [63, 1417, 1425.814054697005, 663.6117048721741, 0.9308530838029871, 0.3653936731450976, 0.031376546252019015], [79, 1464, 1444.0282679852098, 677.4854028476402, 0.9383270644596262, 0.3457489263939665, 0.030328340266897933], [75, 1544, 1469.1220184586969, 694.687562087553, 0.9457179912344817, 0.32498843218708423, 0.029151512243808426], [82, 1777, 1517.2967131979808, 722.4499407343583, 0.9522856465057025, 0.3052082034598922, 0.027798343305251258], [64, 2041, 1628.5344078075004, 781.1165326649259, 0.9592877238824138, 0.2824306336226629, 0.02581151535890055], [79, 1980, 1748.1958761658364, 843.5172747151844, 0.9650146030148479, 0.2621961402616266, 0.024003477949933835], [61, 1923, 1849.7212775517257, 896.5488830162212, 0.9693880844609034, 0.24553358569698794, 0.022581790724225836], [68, 1858, 1936.6869325205805, 942.6786286026306, 0.9734961420695321, 0.2287034354261839, 0.02129917120194863], [81, 1792, 2011.5537233241027, 983.0984506178714, 0.9774518465192132, 0.21115844225647537, 0.020081422713857554], [90, 1717, 2067.1792015753595, 1013.9861293192897, 0.9810336022600744, 0.19383774461291153, 0.018979561831458693], [81, 1636, 2104.682952595497, 1035.768424446506, 0.9842512604277954, 0.1767751576050678, 0.01796277153465826], [133, 1573, 2120.6448727175593, 1046.4309627376497, 0.9868988213916945, 0.1613403741649442, 0.01709596530705885], [194, 1488, 2101.3805918458793, 1039.3361941231385, 0.9891936740599402, 0.14661471685951744, 0.016371651673724754], [422, 1407, 2015.6070777639507, 998.7075674671568, 0.9909744597395348, 0.13405081179887965, 0.015984087864021973]], 'totalCount': 4961, 'weightedCount': 89487.2330131929, 'minAxis': 0, 'maxAxis': 1}}, '_className': 'DefaultDispatchConfiguration', '_version': 1}, 'isEditable': True, '_className': 'CompositeDecisionTool', '_version': 1}, 'CompositeDecisionTool1': {'id': 'CompositeDecisionTool1', 'name': 'initial scenario', 'comments': [], 'type': 'o', 'hasHuman': False, 'tools': ['DecisionTool1', 'DecisionTool2'], 'confidenceProvider': 'DecisionTool1', 'attributes': [], 'useDeciles': False, 'dispatchConfiguration': {'tools': ['DecisionTool1', 'DecisionTool2'], 'useDeciles': False, 'decisionTree': {'bestTool': 'DecisionTool1', 'basisCount': 4961, 'basisPerf': 259.1315964523127, 'basisStdDev': 1030.1811987507722, 'basisConfidence95': 28.667186867473713, 'secondTool': 'DecisionTool2', 'secondPerf': 152.032408788545, 'secondStdDev': 1129.5266064833627, 'secondConfidence95': 31.431703800367703, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[259.1315964523127, 259.18159645232817, -0.050000000000004464, 4961, 1030.1811987507722, 28.667186867473713]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[152.032408788545, 152.05240878855065, -0.019999999999999477, 4961, 1129.5266064833627, 31.431703800367703]], '_className': 'AnalysisResult', '_version': 1}}, 'slots': [{'bestTool': 'DecisionTool1', 'basisCount': 1109, 'basisPerf': 1518.6667718665442, 'basisStdDev': 1269.0681026377545, 'basisConfidence95': 74.69219575798107, 'secondTool': 'DecisionTool2', 'secondPerf': 956.928872858433, 'secondStdDev': 1612.5622458124085, 'secondConfidence95': 94.90886634515807, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[1518.6667718665442, 1518.7167718665467, -0.04999999999999909, 1109, 1269.0681026377545, 74.69219575798107]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[956.928872858433, 956.9488728584308, -0.019999999999999653, 1109, 1612.5622458124085, 94.90886634515807]], '_className': 'AnalysisResult', '_version': 1}}, 'param': 0, 'lowBound': 0, 'highBound': 0.4265894850170295, '_className': 'DecisionNode', '_version': 1}, {'bestTool': 'DecisionTool2', 'basisCount': 978, 'basisPerf': 124.60566462167593, 'basisStdDev': 1203.0887353893202, 'basisConfidence95': 75.40224983527078, 'secondTool': 'DecisionTool1', 'secondPerf': -315.8386503067458, 'secondStdDev': 1042.6191101052505, 'secondConfidence95': 65.34499435550252, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[-315.8386503067458, -315.78865030674825, -0.049999999999999344, 978, 1042.6191101052505, 65.34499435550252]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[124.60566462167593, 124.62566462167683, -0.019999999999999664, 978, 1203.0887353893202, 75.40224983527078]], '_className': 'AnalysisResult', '_version': 1}}, 'param': 1, 'lowBound': 0.4265894850170295, 'highBound': 0.6757876453869698, '_className': 'DecisionNode', '_version': 1}, {'bestTool': 'DecisionTool1', 'basisCount': 2874, 'basisPerf': -31.231524008352157, 'basisStdDev': 177.72526915852063, 'basisConfidence95': 6.49773312584119, 'secondTool': 'DecisionTool2', 'secondPerf': -149.22257480863553, 'secondStdDev': 604.4315866818647, 'secondConfidence95': 22.098349670155564, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[-31.231524008352157, -31.181524008350735, -0.049999999999999455, 2874, 177.72526915852063, 6.49773312584119]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[-149.22257480863553, -149.20257480862907, -0.020000000000001177, 2874, 604.4315866818647, 22.098349670155564]], '_className': 'AnalysisResult', '_version': 1}}, 'param': 2, 'lowBound': 0.6757876453869698, 'highBound': 1, '_className': 'DecisionNode', '_version': 1}], 'confidenceProvider': 'DecisionTool1', 'curvesPerTool': {'DecisionTool1': {'name': 'ScalarAxisPerformanceCurve', 'description': 'average performance over ml', 'headerLabels': ['count', 'totalCount', 'weightedCount', 'weightedSum', 'weightedAverage', 'weightedStdDev', 'confidence95'], 'rowLabels': [0.005, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095, 0.105, 0.115, 0.125, 0.135, 0.145, 0.155, 0.165, 0.175, 0.185, 0.195, 0.205, 0.215, 0.225, 0.235, 0.245, 0.255, 0.265, 0.275, 0.285, 0.295, 0.305, 0.315, 0.325, 0.335, 0.345, 0.355, 0.365, 0.375, 0.385, 0.395, 0.405, 0.415, 0.425, 0.435, 0.445, 0.455, 0.465, 0.475, 0.485, 0.495, 0.505, 0.515, 0.525, 0.535, 0.545, 0.555, 0.565, 0.575, 0.585, 0.595, 0.605, 0.615, 0.625, 0.635, 0.645, 0.655, 0.665, 0.675, 0.685, 0.695, 0.705, 0.715, 0.725, 0.735, 0.745, 0.755, 0.765, 0.775, 0.785, 0.795, 0.805, 0.815, 0.825, 0.835, 0.845, 0.855, 0.865, 0.875, 0.885, 0.895, 0.905, 0.915, 0.925, 0.935, 0.945, 0.955, 0.965, 0.975, 0.985, 0.995], 'content': [[79, 362, 489.9275345206288, 528373.39815652, 2156.945102803861, 658.785048928632, 2.2728041542043456], [87, 388, 530.9121080279357, 569138.53995202, 2144.0028635477, 662.9110460432803, 2.190142557446295], [30, 422, 546.8421381860971, 580378.5895550667, 2122.6549639360696, 676.9707102782401, 2.180770805190957], [28, 443, 554.7489493340258, 582221.6918374808, 2099.0456765585077, 693.5734354500629, 2.1915634690318844], [26, 472, 557.8916530102493, 577639.1883111263, 2070.793442398086, 716.40539552667, 2.221061394804876], [24, 505, 556.0899019777773, 565974.4117626875, 2035.5500423573783, 738.2898064130303, 2.258379916499804], [18, 525, 549.6626098096364, 548535.25676137, 1995.898018063638, 757.9904402981722, 2.3016529379910637], [18, 553, 540.8611569106579, 529494.6160377273, 1957.9687292100803, 764.2543063427773, 2.329872322383725], [15, 581, 530.0420757234103, 508414.0736477782, 1918.3913765860423, 762.1691509069773, 2.3503177561564637], [29, 609, 518.6310228511693, 486293.60343946645, 1875.297010834685, 775.1727596720058, 2.396216625558494], [20, 587, 506.1339526101943, 463352.2768840791, 1830.9472205708196, 795.2519593711303, 2.456833410168547], [34, 535, 503.6596155658367, 449667.23729545856, 1785.5997320344202, 834.5962090928206, 2.523049147460784], [18, 504, 509.2044422701001, 445168.22728227155, 1748.4852461131456, 879.9830874470028, 2.576600829083228], [37, 502, 516.0882146239277, 441558.81601715524, 1711.1757389729125, 924.3813281791207, 2.62312920509842], [24, 507, 521.8740411460395, 436429.3628947749, 1672.5467391954294, 963.7891377552237, 2.663570753317751], [27, 492, 525.7403756707907, 429770.4956558691, 1634.9153138848278, 1009.1900205830974, 2.7155442212771774], [24, 496, 527.9288654267792, 420709.3165990254, 1593.8106216598812, 1061.5442928153236, 2.779312796560511], [30, 504, 528.647108361125, 408948.83627011464, 1547.1524569117928, 1118.4339229647235, 2.8508758587975382], [31, 505, 526.9675245165822, 393987.0387958554, 1495.2991236311289, 1182.9219147760334, 2.936582350720672], [25, 504, 520.1420621842151, 374597.21251441934, 1440.3650069805385, 1239.8114453354342, 3.0260277361461285], [27, 494, 510.1043381586664, 352216.7903412083, 1380.9597919226178, 1295.6937595631246, 3.1237606255920087], [24, 481, 497.3211818471546, 330068.3230498974, 1327.3849379346955, 1338.545945115802, 3.215542353206572], [25, 462, 483.73101902753115, 308791.20447355456, 1276.7062368434965, 1374.616018905163, 3.304036246512993], [25, 462, 469.59355347305564, 288081.2890982294, 1226.9388579447734, 1409.5919451327707, 3.3957969418055014], [19, 452, 453.5758971214299, 267861.55097135017, 1181.1101633543774, 1446.1565955575104, 3.499763911766855], [22, 435, 439.04738725721785, 249490.75053481138, 1136.5094419233892, 1477.6440068164616, 3.5957151603470576], [24, 433, 424.69398812651576, 232786.4749704444, 1096.2550988647272, 1507.0433961241615, 3.6921632382625518], [24, 413, 409.28343667089933, 216745.38649668783, 1059.145653484974, 1533.9323269484678, 3.7944347296061847], [18, 413, 392.48070008605765, 201976.85479687096, 1029.2320348622713, 1553.0004188528424, 3.8988156890348833], [17, 410, 380.1750897437334, 192209.40845049458, 1011.1625597566541, 1558.0836211661028, 3.967889999967082], [17, 407, 372.5680188298226, 185584.6177343058, 996.2455624462765, 1556.1893708339444, 4.005756144006946], [13, 410, 369.03065403029325, 180241.2597214069, 976.8362478994015, 1556.896190809806, 4.0258230113662945], [20, 419, 370.25391985848546, 178568.44827816729, 964.572898223024, 1551.1456145491998, 4.01173764224286], [13, 411, 374.1286192186176, 177950.9004535111, 951.2819458996138, 1545.475550758689, 3.9836087284778947], [20, 410, 381.0866052798922, 177665.24565131846, 932.4140139789524, 1547.5858945560117, 3.949768215474291], [12, 413, 389.28726483807003, 178129.79302731622, 915.1585942654046, 1555.6972229393011, 3.9181721319815876], [19, 409, 399.6015967302028, 180446.47889838862, 903.1319212681718, 1558.3295686577894, 3.8705449866746617], [13, 408, 412.4199087359011, 182449.2839041892, 884.774376985875, 1569.0372123391921, 3.82298753791692], [28, 422, 430.01258087716934, 184331.57416022065, 857.3310752174199, 1586.2322410261206, 3.764426975338503], [28, 437, 446.0132111277429, 181632.7339815276, 814.4724391560951, 1617.2026749655554, 3.7321959340386877], [28, 463, 461.919537695125, 176918.5191514876, 766.0144450017048, 1649.0650364280868, 3.7033247474222355], [26, 486, 478.0187630884355, 167497.3768053623, 700.7983357104105, 1674.341949812775, 3.668222569871048], [28, 501, 493.2074170149863, 150626.44979568172, 610.8036683929466, 1699.9449868221423, 3.638804351370104], [21, 515, 507.0566467501217, 128006.89097588649, 504.90173749351715, 1719.4491861987228, 3.609296033979238], [25, 531, 523.5474086277183, 103179.33207989857, 394.1546854384943, 1719.187709577509, 3.5517281145822626], [17, 569, 542.0604638542986, 74333.75186660636, 274.26369131612836, 1701.0571716300822, 3.472095486824801], [24, 586, 565.8530931267888, 41025.23590019717, 145.0031338469852, 1665.5795264397334, 3.362690609853125], [26, 617, 591.2417580429465, 6493.8706857647585, 21.96688781678732, 1610.4343161865759, 3.2347820879718974], [34, 637, 616.3925462942572, -29701.094528910784, -96.37071281109196, 1535.5368460401207, 3.093552588655884], [35, 656, 644.7653608489765, -66171.51819894271, -205.25767113733667, 1440.7250605726038, 2.929852641671837], [42, 680, 671.3958571318558, -103183.91411540253, -307.3713161001474, 1329.2123724918335, 2.757807385757026], [33, 693, 697.5982532478866, -135505.83556340562, -388.49247380570654, 1220.5099535208813, 2.5925314999430826], [31, 715, 724.8603003658354, -160969.04719347548, -444.1381245799632, 1113.676373256834, 2.429452527007756], [37, 740, 755.4656619962318, -179948.87133719213, -476.39192722988304, 1012.8245618647669, 2.269424918486337], [41, 776, 785.6776376809897, -194521.96007833135, -495.1699036579007, 920.283277837597, 2.1212636042219333], [50, 806, 815.1952161300952, -204278.8462404887, -501.1777355864372, 839.2730788884588, 1.988734951677985], [32, 841, 839.8752498216907, -208882.80163082902, -497.41387587067436, 767.3540609300209, 1.8734692861356317], [43, 866, 864.6347760818899, -210343.26264705067, -486.548236239642, 705.7028718532548, 1.77072314598558], [50, 881, 889.0974369395549, -208978.72114848054, -470.0918312571582, 659.757124817531, 1.6883925869793015], [37, 911, 909.818261859937, -203137.2544882157, -446.5446848097843, 629.1451982529911, 1.629874592147328], [49, 951, 936.4902138557305, -194582.39520166066, -415.55670806323394, 614.598196487518, 1.5878156946713144], [50, 984, 967.3467242900265, -185096.28764315546, -382.6886120465335, 596.5000158931228, 1.5391119794869674], [50, 1033, 1000.0107291210443, -175647.0345820629, -351.29030012797404, 577.7082159363395, 1.4897315433332088], [46, 1053, 1033.7052764605755, -165720.5644176394, -320.63406890031445, 558.1006932083352, 1.4401707488968587], [59, 1072, 1063.8112291172135, -155666.80120397452, -292.6586915860102, 538.5539416570883, 1.3945638097244366], [57, 1081, 1092.8242930419751, -145105.10281584447, -265.55980451703044, 517.0760588283993, 1.348211775473957], [50, 1108, 1120.1225627571314, -134336.0151403179, -239.85949324983838, 494.51239140178416, 1.3023025410254596], [53, 1128, 1149.296166934073, -123365.95810146579, -214.68088322362328, 470.6366891346173, 1.2542467440956997], [47, 1149, 1177.532691988352, -111631.02615723347, -189.60157440509974, 444.2661597035806, 1.2039021819417512], [74, 1170, 1209.2702154651274, -101506.24624048946, -167.8801725906176, 418.8546817970011, 1.153522527255387], [74, 1195, 1234.538417451084, -91179.83430510962, -147.71485928054958, 392.9544273593543, 1.1057956059582097], [70, 1213, 1251.2971808176467, -80619.355782661, -128.85724833165716, 367.3306790138121, 1.061950904387979], [79, 1219, 1260.773050757125, -71003.67284553041, -112.63513731181195, 343.46884907864427, 1.023013324212357], [52, 1241, 1262.8039138142037, -62341.073521201084, -98.73436855751355, 320.37053242844934, 0.9872210186599369], [57, 1243, 1269.3849869515752, -54510.90734107568, -85.88553969270345, 296.29117004924404, 0.9469318996938341], [51, 1270, 1275.8706639830027, -47744.1368813903, -74.84165633582802, 274.33005448412746, 0.908844200625387], [76, 1298, 1288.6035830840453, -42475.56594492333, -65.92495396181587, 255.3885890794162, 0.8725637135715629], [60, 1310, 1296.0470336526632, -37822.89467202044, -58.36654641371119, 237.29015825471149, 0.8386592726873059], [67, 1347, 1305.2835923705268, -33938.886410576844, -52.00231828386106, 220.66014498338728, 0.805871100079928], [68, 1366, 1313.2979664422573, -29678.760445998745, -45.19729902026564, 202.74988370570935, 0.7701135102188484], [69, 1353, 1323.3561954919246, -25913.674951351622, -39.16356766171916, 186.38405879237288, 0.7355667277360556], [50, 1353, 1335.6022728465473, -23207.87364784284, -34.7526716892751, 174.88151367038213, 0.7092338389058843], [65, 1349, 1354.0705972202122, -20491.329672716583, -30.266264867996515, 161.71154184916142, 0.6773387570091182], [75, 1353, 1376.0786884970978, -18235.018968537406, -26.502872431594763, 150.49194886044586, 0.6481731996173935], [64, 1373, 1393.1938667841257, -16384.895803266514, -23.521343574512507, 141.65716294629786, 0.6249849569516799], [88, 1397, 1412.0869614254686, -14656.141619441627, -20.758128953540645, 131.89973399813903, 0.599028257795916], [63, 1417, 1425.814054697005, -12682.216362065636, -17.78943940170472, 119.69409614259874, 0.5678857753811023], [79, 1464, 1444.0282679852098, -11012.190251958522, -15.252042492663048, 109.11707535089707, 0.5387838662926596], [75, 1544, 1469.1220184586969, -9571.557911037904, -13.030310336073695, 99.62451900000809, 0.5103995187945511], [82, 1777, 1517.2967131979808, -8531.488769990236, -11.245643249313524, 91.0081649198034, 0.48002183358919515], [64, 2041, 1628.5344078075004, -7685.332903712331, -9.438342680225114, 81.74875237837368, 0.4391351172251292], [79, 1980, 1748.1958761658364, -6991.652568614631, -7.998706167810903, 73.49692931859745, 0.40187928694254926], [61, 1923, 1849.7212775517257, -6472.956543712446, -6.998845309580903, 67.75010089333598, 0.37510935760394054], [68, 1858, 1936.6869325205805, -5843.628473241459, -6.034665051037474, 61.912162870862716, 0.3504405460001962], [81, 1792, 2011.5537233241027, -5021.949222769537, -4.9931047473797925, 54.95447793078099, 0.32396030308080165], [90, 1717, 2067.1792015753595, -4283.750630297966, -4.14453727769068, 49.30804544628199, 0.30270939616331516], [81, 1636, 2104.682952595497, -3714.6864940316605, -3.5299250078979405, 45.705346500872544, 0.28883261520344355], [133, 1573, 2120.6448727175593, -3102.9772614027374, -2.926446857107518, 41.48135961018739, 0.27412496499829425], [194, 1488, 2101.3805918458793, -2535.797035917185, -2.413458129152805, 37.681220684406995, 0.26246185538723726], [422, 1407, 2015.6070777639507, -1995.5167036367493, -1.9800651879537068, 34.063555527271824, 0.2547992367044842]], 'totalCount': 4961, 'weightedCount': 89487.2330131929, 'minAxis': 0, 'maxAxis': 1}, 'DecisionTool2': {'name': 'ScalarAxisPerformanceCurve', 'description': 'average performance over ml', 'headerLabels': ['count', 'totalCount', 'weightedCount', 'weightedSum', 'weightedAverage', 'weightedStdDev', 'confidence95'], 'rowLabels': [0.005, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095, 0.105, 0.115, 0.125, 0.135, 0.145, 0.155, 0.165, 0.175, 0.185, 0.195, 0.205, 0.215, 0.225, 0.235, 0.245, 0.255, 0.265, 0.275, 0.285, 0.295, 0.305, 0.315, 0.325, 0.335, 0.345, 0.355, 0.365, 0.375, 0.385, 0.395, 0.405, 0.415, 0.425, 0.435, 0.445, 0.455, 0.465, 0.475, 0.485, 0.495, 0.505, 0.515, 0.525, 0.535, 0.545, 0.555, 0.565, 0.575, 0.585, 0.595, 0.605, 0.615, 0.625, 0.635, 0.645, 0.655, 0.665, 0.675, 0.685, 0.695, 0.705, 0.715, 0.725, 0.735, 0.745, 0.755, 0.765, 0.775, 0.785, 0.795, 0.805, 0.815, 0.825, 0.835, 0.845, 0.855, 0.865, 0.875, 0.885, 0.895, 0.905, 0.915, 0.925, 0.935, 0.945, 0.955, 0.965, 0.975, 0.985, 0.995], 'content': [[79, 362, 489.9275345206288, 318753.40353122336, 1301.2267369014426, 1814.3532309799205, 3.7718209514318786], [87, 388, 530.9121080279357, 339352.7818411534, 1278.3765022865036, 1817.2180729767817, 3.626170997536704], [30, 422, 546.8421381860971, 345142.81425655755, 1262.312430426132, 1807.6975153524047, 3.5635919197663184], [28, 443, 554.7489493340258, 345240.7674118468, 1244.6738937543084, 1797.3592819860105, 3.52797319339893], [26, 472, 557.8916530102493, 341538.3506844893, 1224.3895345686942, 1785.2162400918642, 3.5061182067416397], [24, 505, 556.0899019777773, 334473.2211103046, 1202.946573641148, 1765.5537121066943, 3.4924004387631635], [18, 525, 549.6626098096364, 323175.3452755852, 1175.9044166657432, 1744.4735364407343, 3.491726077389597], [18, 553, 540.8611569106579, 309474.2259533937, 1144.3758606037748, 1725.3424119697556, 3.5006671891602865], [15, 581, 530.0420757234103, 294517.38826554193, 1111.2981469011856, 1703.2743207198682, 3.513526211144509], [29, 609, 518.6310228511693, 279125.3432295607, 1076.3927761014822, 1683.629302184823, 3.531425620821245], [20, 587, 506.1339526101943, 264205.48894858314, 1044.0140898908019, 1663.5335987889434, 3.5533593021075185], [34, 535, 503.6596155658367, 256204.39565661715, 1017.3712076112522, 1645.366229275817, 3.542572901061437], [18, 504, 509.2044422701001, 253036.55743951074, 993.850549737707, 1637.239873520991, 3.5145209457429263], [37, 502, 516.0882146239277, 249868.29287710573, 968.3162133790797, 1630.9989538398588, 3.484343330727361], [24, 507, 521.8740411460395, 247536.68952945876, 948.6453435617002, 1619.3104244734993, 3.4525365091796507], [27, 492, 525.7403756707907, 245081.13271632013, 932.327605250488, 1610.390556007328, 3.4303308832763726], [24, 496, 527.9288654267792, 242683.27251125607, 919.3786830165849, 1600.4906980023409, 3.4126751047537893], [30, 504, 528.647108361125, 241119.06306000534, 912.2117921253969, 1585.1536207436884, 3.393976409697896], [31, 505, 526.9675245165822, 239810.0452873643, 910.1511350527945, 1568.1687855304249, 3.3811197142840608], [25, 504, 520.1420621842151, 235219.0150748724, 904.4414292784749, 1547.3867206848652, 3.3806056777699744], [27, 494, 510.1043381586664, 229108.74673925876, 898.2818988220217, 1523.9173001371985, 3.3877180848304027], [24, 481, 497.3211818471546, 222415.21828926954, 894.4530271691748, 1500.124762280971, 3.4040919518500337], [25, 462, 483.73101902753115, 214690.32476572096, 887.6434064423809, 1479.2796495714824, 3.4275140530561217], [25, 462, 469.59355347305564, 207887.66302328324, 885.3940241971887, 1457.1865178578198, 3.452650197658323], [19, 452, 453.5758971214299, 198594.42118849012, 875.6833087862385, 1446.1087962751822, 3.4997060730839102], [22, 435, 439.04738725721785, 188788.7705042139, 859.9926840863361, 1439.0086397049074, 3.548395939039697], [24, 433, 424.69398812651576, 178861.42016222273, 842.307285541043, 1435.1956179203214, 3.6030771758653], [24, 413, 409.28343667089933, 168230.00443232918, 822.0709139891298, 1434.3750971951522, 3.669233542077721], [18, 413, 392.48070008605765, 156254.00782268998, 796.2379183915479, 1434.7037319806195, 3.7473825712395192], [17, 410, 380.1750897437334, 147454.3887442784, 775.7183083387907, 1434.9679687166113, 3.8078983468613017], [17, 407, 372.5680188298226, 139756.0542778182, 750.2310837992479, 1434.6897372248777, 3.8462036971302553], [13, 410, 369.03065403029325, 133642.43689333685, 724.2890824043377, 1431.6152645142809, 3.8604506949717177], [20, 419, 370.25391985848546, 130713.67615577232, 706.075853056369, 1425.9547331008505, 3.84644129324641], [13, 411, 374.1286192186176, 128417.2324999202, 686.487084404955, 1421.775751456342, 3.820860284129973], [20, 410, 381.0866052798922, 128366.9776545019, 673.689265778427, 1413.3693359323645, 3.774609787305215], [12, 413, 389.28726483807003, 129618.41477416016, 665.9268179660432, 1406.824357156498, 3.7259833734040955], [19, 409, 399.6015967302028, 132830.64633075582, 664.8153932199549, 1401.5002592853004, 3.6706168602479696], [13, 408, 412.4199087359011, 136379.54767642223, 661.3625811345339, 1399.2318234320837, 3.6101986330391407], [28, 422, 430.01258087716934, 141224.47172888304, 656.8387903479643, 1395.921573584683, 3.5313925787033194], [28, 437, 446.0132111277429, 143912.18384000094, 645.3270004093353, 1391.6687354115795, 3.462184001832032], [28, 463, 461.919537695125, 146206.216319214, 633.0375937279046, 1387.1515541685912, 3.396525361855946], [26, 486, 478.0187630884355, 147496.4247230944, 617.1156285587348, 1382.1041701974216, 3.3327596005225972], [28, 501, 493.2074170149863, 147245.13488017127, 597.0921352778325, 1374.7539722337312, 3.272304905838548], [21, 515, 507.0566467501217, 146400.99556683903, 577.4542016367086, 1362.8193491570314, 3.213268205690713], [25, 531, 523.5474086277183, 145190.52695144256, 554.6413736704559, 1348.0584944414793, 3.1450853298976598], [17, 569, 542.0604638542986, 143017.35379610592, 527.680446491843, 1329.6099087890952, 3.0696886374705854], [24, 586, 565.8530931267888, 140339.31037094447, 496.0273685011, 1303.84427359178, 2.9752063085559364], [26, 617, 591.2417580429465, 137402.1231021723, 464.7916735684685, 1274.998302013928, 2.8782487499123044], [34, 637, 616.3925462942572, 131671.17328572072, 427.23155585616945, 1252.3969617828777, 2.7938197610953766], [35, 656, 644.7653608489765, 124701.13900792014, 386.8109131784731, 1237.788156066643, 2.7156786874644894], [42, 680, 671.3958571318558, 116879.31532732684, 348.16811598041437, 1222.7947762668675, 2.6451087380431897], [33, 693, 697.5982532478866, 108449.17789308644, 310.921586710309, 1211.0681823039138, 2.5824842187312353], [31, 715, 724.8603003658354, 98246.69970028783, 271.0776121983862, 1199.7946676579872, 2.5216358937624905], [37, 740, 755.4656619962318, 86696.72212670733, 229.51863066183878, 1190.71992776003, 2.460670811804124], [41, 776, 785.6776376809897, 74717.72710443629, 190.19944954771407, 1182.2865292737313, 2.404336480360633], [50, 806, 815.1952161300952, 62516.20342157206, 153.3772578262904, 1173.8942229734967, 2.3520131353765548], [32, 841, 839.8752498216907, 46454.46299579605, 110.62229302663353, 1171.467674648572, 2.314801922205992], [43, 866, 864.6347760818899, 29277.37979147474, 67.72195752788429, 1170.9257641507115, 2.280890323741416], [50, 881, 889.0974369395549, 13799.560620240107, 31.041728492078125, 1166.4016639201586, 2.2449437180262968], [37, 911, 909.818261859937, -405.7415502734907, -0.89191779783368, 1154.998972298582, 2.208358338348539], [49, 951, 936.4902138557305, -16045.477488141098, -34.26726142087152, 1146.1498633956455, 2.1683288745280525], [50, 984, 967.3467242900265, -32902.56366077775, -68.02641252530466, 1137.6619177393218, 2.1255513388467775], [50, 1033, 1000.0107291210443, -49199.774247240995, -98.39849276514254, 1128.1710932292553, 2.081810647592656], [46, 1053, 1033.7052764605755, -64630.63670139185, -125.04654503203902, 1114.220600708587, 2.034901053935979], [59, 1072, 1063.8112291172135, -81225.10526322071, -152.70586179208513, 1101.468407297035, 1.9943887181253204], [57, 1081, 1092.8242930419751, -97520.28705854093, -178.47386387629507, 1088.832202324472, 1.9564167814227995], [50, 1108, 1120.1225627571314, -109892.82068875077, -196.2157077137247, 1073.0638632941902, 1.9183863850431708], [53, 1128, 1149.296166934073, -121562.13282411652, -211.54187462123448, 1053.639004543391, 1.8766617913566994], [47, 1149, 1177.532691988352, -133247.02939606033, -226.31563489088825, 1035.1344576326012, 1.8376718702312365], [74, 1170, 1209.2702154651274, -144108.50956168087, -238.3396328111029, 1019.4256489399991, 1.7995841995842998], [74, 1195, 1234.538417451084, -151700.88076638116, -245.7612960795398, 1000.2598566637229, 1.7642502456085107], [70, 1213, 1251.2971808176467, -155803.76422068168, -249.0275956969285, 973.802548675195, 1.7290648881748207], [79, 1219, 1260.773050757125, -157962.03119450936, -250.57964413127215, 949.3532460359693, 1.7007932854405359], [52, 1241, 1262.8039138142037, -159719.11820600333, -252.9594919033531, 929.0906455946342, 1.6811913834908785], [57, 1243, 1269.3849869515752, -159914.8959812941, -251.9564948776168, 906.4700902640192, 1.6562890548098426], [51, 1270, 1275.8706639830027, -160149.26933960884, -251.0431094005346, 883.9829626429846, 1.6314534731821777], [76, 1298, 1288.6035830840453, -161672.72154531284, -250.92700915572144, 863.9685285671559, 1.6048903492542628], [60, 1310, 1296.0470336526632, -161388.46506818145, -249.0472349808766, 843.9642153728931, 1.581640242689987], [67, 1347, 1305.2835923705268, -159623.2523235626, -244.58018664537207, 819.4361732662906, 1.5529633185904668], [68, 1366, 1313.2979664422573, -157364.70032600465, -239.6481291329611, 791.546278349176, 1.5216423645917616], [69, 1353, 1323.3561954919246, -155178.7869937261, -234.52308233013903, 764.0810484592214, 1.489317895180292], [50, 1353, 1335.6022728465473, -152332.02419575836, -228.10986068643865, 740.6757357151247, 1.4595922724681347], [65, 1349, 1354.0705972202122, -149124.53086920854, -220.26108708858766, 713.9117036082287, 1.4231729097611814], [75, 1353, 1376.0786884970978, -145142.6058435924, -210.95102635752872, 686.1516188506038, 1.3840267466573115], [64, 1373, 1393.1938667841257, -139093.02580742005, -199.6750475631722, 656.8122145328541, 1.3457701920108236], [88, 1397, 1412.0869614254686, -130929.11592940718, -185.4405847600736, 622.6904961762785, 1.3015518342393444], [63, 1417, 1425.814054697005, -120457.10843661502, -168.9660836766165, 581.6534802725506, 1.2518629640656904], [79, 1464, 1444.0282679852098, -110863.52072190335, -153.54757684430433, 540.9490190812529, 1.1996274442091504], [75, 1544, 1469.1220184586969, -102217.72985937215, -139.15485381753663, 502.19677116234135, 1.1459458483289577], [82, 1777, 1517.2967131979808, -94071.03950322726, -123.99821166811245, 460.6091441161893, 1.0799088257719134], [64, 2041, 1628.5344078075004, -87960.27279846894, -108.02384325043529, 412.88117866212633, 0.9868931734594801], [79, 1980, 1748.1958761658364, -83600.45850841807, -95.64198113974685, 370.44081509423626, 0.9022366011470442], [61, 1923, 1849.7212775517257, -79215.5161931164, -85.65130017638694, 333.14831312793893, 0.8318053981862149], [68, 1858, 1936.6869325205805, -74476.91519563725, -76.91167213970533, 297.1212680010481, 0.7677029865878799], [81, 1792, 2011.5537233241027, -71355.28650166595, -70.94544448333299, 266.8813216996032, 0.7139196624611215], [90, 1717, 2067.1792015753595, -69082.57819178466, -66.83753216860744, 243.135512211519, 0.6721886979974138], [81, 1636, 2104.682952595497, -67971.4506558445, -64.5906791538574, 227.44302335111297, 0.6443161830692872], [133, 1573, 2120.6448727175593, -66440.50064535323, -62.66065714266548, 212.70033353797547, 0.6207349281671649], [194, 1488, 2101.3805918458793, -63750.20300790121, -60.67459008165886, 197.99690795295922, 0.6016347557247406], [422, 1407, 2015.6070777639507, -59732.04808340092, -59.269535955058984, 186.69098337652855, 0.5965059852352873]], 'totalCount': 4961, 'weightedCount': 89487.2330131929, 'minAxis': 0, 'maxAxis': 1}}, 'bounds': [0.4265894850170295, 0.6757876453869698], 'bestTools': ['DecisionTool1', 'DecisionTool2', 'DecisionTool1'], '_className': 'ConfidenceDecisionNode', '_version': 1}, 'state': {}, 'minCasesPerPartition': 70, 'curvesPerTool': {}, '_className': 'DefaultDispatchConfiguration', '_version': 1}, 'isEditable': True, '_className': 'CompositeDecisionTool', '_version': 1}, 'CompositeDecisionTool2': {'id': 'CompositeDecisionTool2', 'name': 'initial scenario', 'comments': [], 'type': 'o', 'hasHuman': False, 'tools': ['DecisionTool1', 'DecisionTool2'], 'confidenceProvider': 'DecisionTool1', 'attributes': [], 'useDeciles': False, 'dispatchConfiguration': {'tools': ['DecisionTool1', 'DecisionTool2'], 'useDeciles': False, 'decisionTree': {'bestTool': 'DecisionTool1', 'basisCount': 4961, 'basisPerf': 259.1315964523127, 'basisStdDev': 1030.1811987507722, 'basisConfidence95': 28.667186867473713, 'secondTool': 'DecisionTool2', 'secondPerf': 152.032408788545, 'secondStdDev': 1129.5266064833627, 'secondConfidence95': 31.431703800367703, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[259.1315964523127, 259.18159645232817, -0.050000000000004464, 4961, 1030.1811987507722, 28.667186867473713]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[152.032408788545, 152.05240878855065, -0.019999999999999477, 4961, 1129.5266064833627, 31.431703800367703]], '_className': 'AnalysisResult', '_version': 1}}, 'slots': [{'bestTool': 'DecisionTool1', 'basisCount': 1109, 'basisPerf': 1518.6667718665442, 'basisStdDev': 1269.0681026377545, 'basisConfidence95': 74.69219575798107, 'secondTool': 'DecisionTool2', 'secondPerf': 956.928872858433, 'secondStdDev': 1612.5622458124085, 'secondConfidence95': 94.90886634515807, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[1518.6667718665442, 1518.7167718665467, -0.04999999999999909, 1109, 1269.0681026377545, 74.69219575798107]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[956.928872858433, 956.9488728584308, -0.019999999999999653, 1109, 1612.5622458124085, 94.90886634515807]], '_className': 'AnalysisResult', '_version': 1}}, 'param': 0, 'lowBound': 0, 'highBound': 0.4265894850170295, '_className': 'DecisionNode', '_version': 1}, {'bestTool': 'DecisionTool2', 'basisCount': 978, 'basisPerf': 124.60566462167593, 'basisStdDev': 1203.0887353893202, 'basisConfidence95': 75.40224983527078, 'secondTool': 'DecisionTool1', 'secondPerf': -315.8386503067458, 'secondStdDev': 1042.6191101052505, 'secondConfidence95': 65.34499435550252, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[-315.8386503067458, -315.78865030674825, -0.049999999999999344, 978, 1042.6191101052505, 65.34499435550252]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[124.60566462167593, 124.62566462167683, -0.019999999999999664, 978, 1203.0887353893202, 75.40224983527078]], '_className': 'AnalysisResult', '_version': 1}}, 'param': 1, 'lowBound': 0.4265894850170295, 'highBound': 0.6757876453869698, '_className': 'DecisionNode', '_version': 1}, {'bestTool': 'DecisionTool1', 'basisCount': 2874, 'basisPerf': -31.231524008352157, 'basisStdDev': 177.72526915852063, 'basisConfidence95': 6.49773312584119, 'secondTool': 'DecisionTool2', 'secondPerf': -149.22257480863553, 'secondStdDev': 604.4315866818647, 'secondConfidence95': 22.098349670155564, 'overlap': 0, 'statsPerTool': {'DecisionTool1': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[-31.231524008352157, -31.181524008350735, -0.049999999999999455, 2874, 177.72526915852063, 6.49773312584119]], '_className': 'AnalysisResult', '_version': 1}, 'DecisionTool2': {'name': 'rawPerformance', 'description': 'average performance', 'headerLabels': ['performance', 'impact cost', 'decision cost', 'count', 'standard deviation', 'confidence95'], 'rowLabels': None, 'content': [[-149.22257480863553, -149.20257480862907, -0.020000000000001177, 2874, 604.4315866818647, 22.098349670155564]], '_className': 'AnalysisResult', '_version': 1}}, 'param': 2, 'lowBound': 0.6757876453869698, 'highBound': 1, '_className': 'DecisionNode', '_version': 1}], 'confidenceProvider': 'DecisionTool1', 'curvesPerTool': {'DecisionTool1': {'name': 'ScalarAxisPerformanceCurve', 'description': 'average performance over ml', 'headerLabels': ['count', 'totalCount', 'weightedCount', 'weightedSum', 'weightedAverage', 'weightedStdDev', 'confidence95'], 'rowLabels': [0.005, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095, 0.105, 0.115, 0.125, 0.135, 0.145, 0.155, 0.165, 0.175, 0.185, 0.195, 0.205, 0.215, 0.225, 0.235, 0.245, 0.255, 0.265, 0.275, 0.285, 0.295, 0.305, 0.315, 0.325, 0.335, 0.345, 0.355, 0.365, 0.375, 0.385, 0.395, 0.405, 0.415, 0.425, 0.435, 0.445, 0.455, 0.465, 0.475, 0.485, 0.495, 0.505, 0.515, 0.525, 0.535, 0.545, 0.555, 0.565, 0.575, 0.585, 0.595, 0.605, 0.615, 0.625, 0.635, 0.645, 0.655, 0.665, 0.675, 0.685, 0.695, 0.705, 0.715, 0.725, 0.735, 0.745, 0.755, 0.765, 0.775, 0.785, 0.795, 0.805, 0.815, 0.825, 0.835, 0.845, 0.855, 0.865, 0.875, 0.885, 0.895, 0.905, 0.915, 0.925, 0.935, 0.945, 0.955, 0.965, 0.975, 0.985, 0.995], 'content': [[79, 362, 489.9275345206288, 528373.39815652, 2156.945102803861, 658.785048928632, 2.2728041542043456], [87, 388, 530.9121080279357, 569138.53995202, 2144.0028635477, 662.9110460432803, 2.190142557446295], [30, 422, 546.8421381860971, 580378.5895550667, 2122.6549639360696, 676.9707102782401, 2.180770805190957], [28, 443, 554.7489493340258, 582221.6918374808, 2099.0456765585077, 693.5734354500629, 2.1915634690318844], [26, 472, 557.8916530102493, 577639.1883111263, 2070.793442398086, 716.40539552667, 2.221061394804876], [24, 505, 556.0899019777773, 565974.4117626875, 2035.5500423573783, 738.2898064130303, 2.258379916499804], [18, 525, 549.6626098096364, 548535.25676137, 1995.898018063638, 757.9904402981722, 2.3016529379910637], [18, 553, 540.8611569106579, 529494.6160377273, 1957.9687292100803, 764.2543063427773, 2.329872322383725], [15, 581, 530.0420757234103, 508414.0736477782, 1918.3913765860423, 762.1691509069773, 2.3503177561564637], [29, 609, 518.6310228511693, 486293.60343946645, 1875.297010834685, 775.1727596720058, 2.396216625558494], [20, 587, 506.1339526101943, 463352.2768840791, 1830.9472205708196, 795.2519593711303, 2.456833410168547], [34, 535, 503.6596155658367, 449667.23729545856, 1785.5997320344202, 834.5962090928206, 2.523049147460784], [18, 504, 509.2044422701001, 445168.22728227155, 1748.4852461131456, 879.9830874470028, 2.576600829083228], [37, 502, 516.0882146239277, 441558.81601715524, 1711.1757389729125, 924.3813281791207, 2.62312920509842], [24, 507, 521.8740411460395, 436429.3628947749, 1672.5467391954294, 963.7891377552237, 2.663570753317751], [27, 492, 525.7403756707907, 429770.4956558691, 1634.9153138848278, 1009.1900205830974, 2.7155442212771774], [24, 496, 527.9288654267792, 420709.3165990254, 1593.8106216598812, 1061.5442928153236, 2.779312796560511], [30, 504, 528.647108361125, 408948.83627011464, 1547.1524569117928, 1118.4339229647235, 2.8508758587975382], [31, 505, 526.9675245165822, 393987.0387958554, 1495.2991236311289, 1182.9219147760334, 2.936582350720672], [25, 504, 520.1420621842151, 374597.21251441934, 1440.3650069805385, 1239.8114453354342, 3.0260277361461285], [27, 494, 510.1043381586664, 352216.7903412083, 1380.9597919226178, 1295.6937595631246, 3.1237606255920087], [24, 481, 497.3211818471546, 330068.3230498974, 1327.3849379346955, 1338.545945115802, 3.215542353206572], [25, 462, 483.73101902753115, 308791.20447355456, 1276.7062368434965, 1374.616018905163, 3.304036246512993], [25, 462, 469.59355347305564, 288081.2890982294, 1226.9388579447734, 1409.5919451327707, 3.3957969418055014], [19, 452, 453.5758971214299, 267861.55097135017, 1181.1101633543774, 1446.1565955575104, 3.499763911766855], [22, 435, 439.04738725721785, 249490.75053481138, 1136.5094419233892, 1477.6440068164616, 3.5957151603470576], [24, 433, 424.69398812651576, 232786.4749704444, 1096.2550988647272, 1507.0433961241615, 3.6921632382625518], [24, 413, 409.28343667089933, 216745.38649668783, 1059.145653484974, 1533.9323269484678, 3.7944347296061847], [18, 413, 392.48070008605765, 201976.85479687096, 1029.2320348622713, 1553.0004188528424, 3.8988156890348833], [17, 410, 380.1750897437334, 192209.40845049458, 1011.1625597566541, 1558.0836211661028, 3.967889999967082], [17, 407, 372.5680188298226, 185584.6177343058, 996.2455624462765, 1556.1893708339444, 4.005756144006946], [13, 410, 369.03065403029325, 180241.2597214069, 976.8362478994015, 1556.896190809806, 4.0258230113662945], [20, 419, 370.25391985848546, 178568.44827816729, 964.572898223024, 1551.1456145491998, 4.01173764224286], [13, 411, 374.1286192186176, 177950.9004535111, 951.2819458996138, 1545.475550758689, 3.9836087284778947], [20, 410, 381.0866052798922, 177665.24565131846, 932.4140139789524, 1547.5858945560117, 3.949768215474291], [12, 413, 389.28726483807003, 178129.79302731622, 915.1585942654046, 1555.6972229393011, 3.9181721319815876], [19, 409, 399.6015967302028, 180446.47889838862, 903.1319212681718, 1558.3295686577894, 3.8705449866746617], [13, 408, 412.4199087359011, 182449.2839041892, 884.774376985875, 1569.0372123391921, 3.82298753791692], [28, 422, 430.01258087716934, 184331.57416022065, 857.3310752174199, 1586.2322410261206, 3.764426975338503], [28, 437, 446.0132111277429, 181632.7339815276, 814.4724391560951, 1617.2026749655554, 3.7321959340386877], [28, 463, 461.919537695125, 176918.5191514876, 766.0144450017048, 1649.0650364280868, 3.7033247474222355], [26, 486, 478.0187630884355, 167497.3768053623, 700.7983357104105, 1674.341949812775, 3.668222569871048], [28, 501, 493.2074170149863, 150626.44979568172, 610.8036683929466, 1699.9449868221423, 3.638804351370104], [21, 515, 507.0566467501217, 128006.89097588649, 504.90173749351715, 1719.4491861987228, 3.609296033979238], [25, 531, 523.5474086277183, 103179.33207989857, 394.1546854384943, 1719.187709577509, 3.5517281145822626], [17, 569, 542.0604638542986, 74333.75186660636, 274.26369131612836, 1701.0571716300822, 3.472095486824801], [24, 586, 565.8530931267888, 41025.23590019717, 145.0031338469852, 1665.5795264397334, 3.362690609853125], [26, 617, 591.2417580429465, 6493.8706857647585, 21.96688781678732, 1610.4343161865759, 3.2347820879718974], [34, 637, 616.3925462942572, -29701.094528910784, -96.37071281109196, 1535.5368460401207, 3.093552588655884], [35, 656, 644.7653608489765, -66171.51819894271, -205.25767113733667, 1440.7250605726038, 2.929852641671837], [42, 680, 671.3958571318558, -103183.91411540253, -307.3713161001474, 1329.2123724918335, 2.757807385757026], [33, 693, 697.5982532478866, -135505.83556340562, -388.49247380570654, 1220.5099535208813, 2.5925314999430826], [31, 715, 724.8603003658354, -160969.04719347548, -444.1381245799632, 1113.676373256834, 2.429452527007756], [37, 740, 755.4656619962318, -179948.87133719213, -476.39192722988304, 1012.8245618647669, 2.269424918486337], [41, 776, 785.6776376809897, -194521.96007833135, -495.1699036579007, 920.283277837597, 2.1212636042219333], [50, 806, 815.1952161300952, -204278.8462404887, -501.1777355864372, 839.2730788884588, 1.988734951677985], [32, 841, 839.8752498216907, -208882.80163082902, -497.41387587067436, 767.3540609300209, 1.8734692861356317], [43, 866, 864.6347760818899, -210343.26264705067, -486.548236239642, 705.7028718532548, 1.77072314598558], [50, 881, 889.0974369395549, -208978.72114848054, -470.0918312571582, 659.757124817531, 1.6883925869793015], [37, 911, 909.818261859937, -203137.2544882157, -446.5446848097843, 629.1451982529911, 1.629874592147328], [49, 951, 936.4902138557305, -194582.39520166066, -415.55670806323394, 614.598196487518, 1.5878156946713144], [50, 984, 967.3467242900265, -185096.28764315546, -382.6886120465335, 596.5000158931228, 1.5391119794869674], [50, 1033, 1000.0107291210443, -175647.0345820629, -351.29030012797404, 577.7082159363395, 1.4897315433332088], [46, 1053, 1033.7052764605755, -165720.5644176394, -320.63406890031445, 558.1006932083352, 1.4401707488968587], [59, 1072, 1063.8112291172135, -155666.80120397452, -292.6586915860102, 538.5539416570883, 1.3945638097244366], [57, 1081, 1092.8242930419751, -145105.10281584447, -265.55980451703044, 517.0760588283993, 1.348211775473957], [50, 1108, 1120.1225627571314, -134336.0151403179, -239.85949324983838, 494.51239140178416, 1.3023025410254596], [53, 1128, 1149.296166934073, -123365.95810146579, -214.68088322362328, 470.6366891346173, 1.2542467440956997], [47, 1149, 1177.532691988352, -111631.02615723347, -189.60157440509974, 444.2661597035806, 1.2039021819417512], [74, 1170, 1209.2702154651274, -101506.24624048946, -167.8801725906176, 418.8546817970011, 1.153522527255387], [74, 1195, 1234.538417451084, -91179.83430510962, -147.71485928054958, 392.9544273593543, 1.1057956059582097], [70, 1213, 1251.2971808176467, -80619.355782661, -128.85724833165716, 367.3306790138121, 1.061950904387979], [79, 1219, 1260.773050757125, -71003.67284553041, -112.63513731181195, 343.46884907864427, 1.023013324212357], [52, 1241, 1262.8039138142037, -62341.073521201084, -98.73436855751355, 320.37053242844934, 0.9872210186599369], [57, 1243, 1269.3849869515752, -54510.90734107568, -85.88553969270345, 296.29117004924404, 0.9469318996938341], [51, 1270, 1275.8706639830027, -47744.1368813903, -74.84165633582802, 274.33005448412746, 0.908844200625387], [76, 1298, 1288.6035830840453, -42475.56594492333, -65.92495396181587, 255.3885890794162, 0.8725637135715629], [60, 1310, 1296.0470336526632, -37822.89467202044, -58.36654641371119, 237.29015825471149, 0.8386592726873059], [67, 1347, 1305.2835923705268, -33938.886410576844, -52.00231828386106, 220.66014498338728, 0.805871100079928], [68, 1366, 1313.2979664422573, -29678.760445998745, -45.19729902026564, 202.74988370570935, 0.7701135102188484], [69, 1353, 1323.3561954919246, -25913.674951351622, -39.16356766171916, 186.38405879237288, 0.7355667277360556], [50, 1353, 1335.6022728465473, -23207.87364784284, -34.7526716892751, 174.88151367038213, 0.7092338389058843], [65, 1349, 1354.0705972202122, -20491.329672716583, -30.266264867996515, 161.71154184916142, 0.6773387570091182], [75, 1353, 1376.0786884970978, -18235.018968537406, -26.502872431594763, 150.49194886044586, 0.6481731996173935], [64, 1373, 1393.1938667841257, -16384.895803266514, -23.521343574512507, 141.65716294629786, 0.6249849569516799], [88, 1397, 1412.0869614254686, -14656.141619441627, -20.758128953540645, 131.89973399813903, 0.599028257795916], [63, 1417, 1425.814054697005, -12682.216362065636, -17.78943940170472, 119.69409614259874, 0.5678857753811023], [79, 1464, 1444.0282679852098, -11012.190251958522, -15.252042492663048, 109.11707535089707, 0.5387838662926596], [75, 1544, 1469.1220184586969, -9571.557911037904, -13.030310336073695, 99.62451900000809, 0.5103995187945511], [82, 1777, 1517.2967131979808, -8531.488769990236, -11.245643249313524, 91.0081649198034, 0.48002183358919515], [64, 2041, 1628.5344078075004, -7685.332903712331, -9.438342680225114, 81.74875237837368, 0.4391351172251292], [79, 1980, 1748.1958761658364, -6991.652568614631, -7.998706167810903, 73.49692931859745, 0.40187928694254926], [61, 1923, 1849.7212775517257, -6472.956543712446, -6.998845309580903, 67.75010089333598, 0.37510935760394054], [68, 1858, 1936.6869325205805, -5843.628473241459, -6.034665051037474, 61.912162870862716, 0.3504405460001962], [81, 1792, 2011.5537233241027, -5021.949222769537, -4.9931047473797925, 54.95447793078099, 0.32396030308080165], [90, 1717, 2067.1792015753595, -4283.750630297966, -4.14453727769068, 49.30804544628199, 0.30270939616331516], [81, 1636, 2104.682952595497, -3714.6864940316605, -3.5299250078979405, 45.705346500872544, 0.28883261520344355], [133, 1573, 2120.6448727175593, -3102.9772614027374, -2.926446857107518, 41.48135961018739, 0.27412496499829425], [194, 1488, 2101.3805918458793, -2535.797035917185, -2.413458129152805, 37.681220684406995, 0.26246185538723726], [422, 1407, 2015.6070777639507, -1995.5167036367493, -1.9800651879537068, 34.063555527271824, 0.2547992367044842]], 'totalCount': 4961, 'weightedCount': 89487.2330131929, 'minAxis': 0, 'maxAxis': 1}, 'DecisionTool2': {'name': 'ScalarAxisPerformanceCurve', 'description': 'average performance over ml', 'headerLabels': ['count', 'totalCount', 'weightedCount', 'weightedSum', 'weightedAverage', 'weightedStdDev', 'confidence95'], 'rowLabels': [0.005, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095, 0.105, 0.115, 0.125, 0.135, 0.145, 0.155, 0.165, 0.175, 0.185, 0.195, 0.205, 0.215, 0.225, 0.235, 0.245, 0.255, 0.265, 0.275, 0.285, 0.295, 0.305, 0.315, 0.325, 0.335, 0.345, 0.355, 0.365, 0.375, 0.385, 0.395, 0.405, 0.415, 0.425, 0.435, 0.445, 0.455, 0.465, 0.475, 0.485, 0.495, 0.505, 0.515, 0.525, 0.535, 0.545, 0.555, 0.565, 0.575, 0.585, 0.595, 0.605, 0.615, 0.625, 0.635, 0.645, 0.655, 0.665, 0.675, 0.685, 0.695, 0.705, 0.715, 0.725, 0.735, 0.745, 0.755, 0.765, 0.775, 0.785, 0.795, 0.805, 0.815, 0.825, 0.835, 0.845, 0.855, 0.865, 0.875, 0.885, 0.895, 0.905, 0.915, 0.925, 0.935, 0.945, 0.955, 0.965, 0.975, 0.985, 0.995], 'content': [[79, 362, 489.9275345206288, 318753.40353122336, 1301.2267369014426, 1814.3532309799205, 3.7718209514318786], [87, 388, 530.9121080279357, 339352.7818411534, 1278.3765022865036, 1817.2180729767817, 3.626170997536704], [30, 422, 546.8421381860971, 345142.81425655755, 1262.312430426132, 1807.6975153524047, 3.5635919197663184], [28, 443, 554.7489493340258, 345240.7674118468, 1244.6738937543084, 1797.3592819860105, 3.52797319339893], [26, 472, 557.8916530102493, 341538.3506844893, 1224.3895345686942, 1785.2162400918642, 3.5061182067416397], [24, 505, 556.0899019777773, 334473.2211103046, 1202.946573641148, 1765.5537121066943, 3.4924004387631635], [18, 525, 549.6626098096364, 323175.3452755852, 1175.9044166657432, 1744.4735364407343, 3.491726077389597], [18, 553, 540.8611569106579, 309474.2259533937, 1144.3758606037748, 1725.3424119697556, 3.5006671891602865], [15, 581, 530.0420757234103, 294517.38826554193, 1111.2981469011856, 1703.2743207198682, 3.513526211144509], [29, 609, 518.6310228511693, 279125.3432295607, 1076.3927761014822, 1683.629302184823, 3.531425620821245], [20, 587, 506.1339526101943, 264205.48894858314, 1044.0140898908019, 1663.5335987889434, 3.5533593021075185], [34, 535, 503.6596155658367, 256204.39565661715, 1017.3712076112522, 1645.366229275817, 3.542572901061437], [18, 504, 509.2044422701001, 253036.55743951074, 993.850549737707, 1637.239873520991, 3.5145209457429263], [37, 502, 516.0882146239277, 249868.29287710573, 968.3162133790797, 1630.9989538398588, 3.484343330727361], [24, 507, 521.8740411460395, 247536.68952945876, 948.6453435617002, 1619.3104244734993, 3.4525365091796507], [27, 492, 525.7403756707907, 245081.13271632013, 932.327605250488, 1610.390556007328, 3.4303308832763726], [24, 496, 527.9288654267792, 242683.27251125607, 919.3786830165849, 1600.4906980023409, 3.4126751047537893], [30, 504, 528.647108361125, 241119.06306000534, 912.2117921253969, 1585.1536207436884, 3.393976409697896], [31, 505, 526.9675245165822, 239810.0452873643, 910.1511350527945, 1568.1687855304249, 3.3811197142840608], [25, 504, 520.1420621842151, 235219.0150748724, 904.4414292784749, 1547.3867206848652, 3.3806056777699744], [27, 494, 510.1043381586664, 229108.74673925876, 898.2818988220217, 1523.9173001371985, 3.3877180848304027], [24, 481, 497.3211818471546, 222415.21828926954, 894.4530271691748, 1500.124762280971, 3.4040919518500337], [25, 462, 483.73101902753115, 214690.32476572096, 887.6434064423809, 1479.2796495714824, 3.4275140530561217], [25, 462, 469.59355347305564, 207887.66302328324, 885.3940241971887, 1457.1865178578198, 3.452650197658323], [19, 452, 453.5758971214299, 198594.42118849012, 875.6833087862385, 1446.1087962751822, 3.4997060730839102], [22, 435, 439.04738725721785, 188788.7705042139, 859.9926840863361, 1439.0086397049074, 3.548395939039697], [24, 433, 424.69398812651576, 178861.42016222273, 842.307285541043, 1435.1956179203214, 3.6030771758653], [24, 413, 409.28343667089933, 168230.00443232918, 822.0709139891298, 1434.3750971951522, 3.669233542077721], [18, 413, 392.48070008605765, 156254.00782268998, 796.2379183915479, 1434.7037319806195, 3.7473825712395192], [17, 410, 380.1750897437334, 147454.3887442784, 775.7183083387907, 1434.9679687166113, 3.8078983468613017], [17, 407, 372.5680188298226, 139756.0542778182, 750.2310837992479, 1434.6897372248777, 3.8462036971302553], [13, 410, 369.03065403029325, 133642.43689333685, 724.2890824043377, 1431.6152645142809, 3.8604506949717177], [20, 419, 370.25391985848546, 130713.67615577232, 706.075853056369, 1425.9547331008505, 3.84644129324641], [13, 411, 374.1286192186176, 128417.2324999202, 686.487084404955, 1421.775751456342, 3.820860284129973], [20, 410, 381.0866052798922, 128366.9776545019, 673.689265778427, 1413.3693359323645, 3.774609787305215], [12, 413, 389.28726483807003, 129618.41477416016, 665.9268179660432, 1406.824357156498, 3.7259833734040955], [19, 409, 399.6015967302028, 132830.64633075582, 664.8153932199549, 1401.5002592853004, 3.6706168602479696], [13, 408, 412.4199087359011, 136379.54767642223, 661.3625811345339, 1399.2318234320837, 3.6101986330391407], [28, 422, 430.01258087716934, 141224.47172888304, 656.8387903479643, 1395.921573584683, 3.5313925787033194], [28, 437, 446.0132111277429, 143912.18384000094, 645.3270004093353, 1391.6687354115795, 3.462184001832032], [28, 463, 461.919537695125, 146206.216319214, 633.0375937279046, 1387.1515541685912, 3.396525361855946], [26, 486, 478.0187630884355, 147496.4247230944, 617.1156285587348, 1382.1041701974216, 3.3327596005225972], [28, 501, 493.2074170149863, 147245.13488017127, 597.0921352778325, 1374.7539722337312, 3.272304905838548], [21, 515, 507.0566467501217, 146400.99556683903, 577.4542016367086, 1362.8193491570314, 3.213268205690713], [25, 531, 523.5474086277183, 145190.52695144256, 554.6413736704559, 1348.0584944414793, 3.1450853298976598], [17, 569, 542.0604638542986, 143017.35379610592, 527.680446491843, 1329.6099087890952, 3.0696886374705854], [24, 586, 565.8530931267888, 140339.31037094447, 496.0273685011, 1303.84427359178, 2.9752063085559364], [26, 617, 591.2417580429465, 137402.1231021723, 464.7916735684685, 1274.998302013928, 2.8782487499123044], [34, 637, 616.3925462942572, 131671.17328572072, 427.23155585616945, 1252.3969617828777, 2.7938197610953766], [35, 656, 644.7653608489765, 124701.13900792014, 386.8109131784731, 1237.788156066643, 2.7156786874644894], [42, 680, 671.3958571318558, 116879.31532732684, 348.16811598041437, 1222.7947762668675, 2.6451087380431897], [33, 693, 697.5982532478866, 108449.17789308644, 310.921586710309, 1211.0681823039138, 2.5824842187312353], [31, 715, 724.8603003658354, 98246.69970028783, 271.0776121983862, 1199.7946676579872, 2.5216358937624905], [37, 740, 755.4656619962318, 86696.72212670733, 229.51863066183878, 1190.71992776003, 2.460670811804124], [41, 776, 785.6776376809897, 74717.72710443629, 190.19944954771407, 1182.2865292737313, 2.404336480360633], [50, 806, 815.1952161300952, 62516.20342157206, 153.3772578262904, 1173.8942229734967, 2.3520131353765548], [32, 841, 839.8752498216907, 46454.46299579605, 110.62229302663353, 1171.467674648572, 2.314801922205992], [43, 866, 864.6347760818899, 29277.37979147474, 67.72195752788429, 1170.9257641507115, 2.280890323741416], [50, 881, 889.0974369395549, 13799.560620240107, 31.041728492078125, 1166.4016639201586, 2.2449437180262968], [37, 911, 909.818261859937, -405.7415502734907, -0.89191779783368, 1154.998972298582, 2.208358338348539], [49, 951, 936.4902138557305, -16045.477488141098, -34.26726142087152, 1146.1498633956455, 2.1683288745280525], [50, 984, 967.3467242900265, -32902.56366077775, -68.02641252530466, 1137.6619177393218, 2.1255513388467775], [50, 1033, 1000.0107291210443, -49199.774247240995, -98.39849276514254, 1128.1710932292553, 2.081810647592656], [46, 1053, 1033.7052764605755, -64630.63670139185, -125.04654503203902, 1114.220600708587, 2.034901053935979], [59, 1072, 1063.8112291172135, -81225.10526322071, -152.70586179208513, 1101.468407297035, 1.9943887181253204], [57, 1081, 1092.8242930419751, -97520.28705854093, -178.47386387629507, 1088.832202324472, 1.9564167814227995], [50, 1108, 1120.1225627571314, -109892.82068875077, -196.2157077137247, 1073.0638632941902, 1.9183863850431708], [53, 1128, 1149.296166934073, -121562.13282411652, -211.54187462123448, 1053.639004543391, 1.8766617913566994], [47, 1149, 1177.532691988352, -133247.02939606033, -226.31563489088825, 1035.1344576326012, 1.8376718702312365], [74, 1170, 1209.2702154651274, -144108.50956168087, -238.3396328111029, 1019.4256489399991, 1.7995841995842998], [74, 1195, 1234.538417451084, -151700.88076638116, -245.7612960795398, 1000.2598566637229, 1.7642502456085107], [70, 1213, 1251.2971808176467, -155803.76422068168, -249.0275956969285, 973.802548675195, 1.7290648881748207], [79, 1219, 1260.773050757125, -157962.03119450936, -250.57964413127215, 949.3532460359693, 1.7007932854405359], [52, 1241, 1262.8039138142037, -159719.11820600333, -252.9594919033531, 929.0906455946342, 1.6811913834908785], [57, 1243, 1269.3849869515752, -159914.8959812941, -251.9564948776168, 906.4700902640192, 1.6562890548098426], [51, 1270, 1275.8706639830027, -160149.26933960884, -251.0431094005346, 883.9829626429846, 1.6314534731821777], [76, 1298, 1288.6035830840453, -161672.72154531284, -250.92700915572144, 863.9685285671559, 1.6048903492542628], [60, 1310, 1296.0470336526632, -161388.46506818145, -249.0472349808766, 843.9642153728931, 1.581640242689987], [67, 1347, 1305.2835923705268, -159623.2523235626, -244.58018664537207, 819.4361732662906, 1.5529633185904668], [68, 1366, 1313.2979664422573, -157364.70032600465, -239.6481291329611, 791.546278349176, 1.5216423645917616], [69, 1353, 1323.3561954919246, -155178.7869937261, -234.52308233013903, 764.0810484592214, 1.489317895180292], [50, 1353, 1335.6022728465473, -152332.02419575836, -228.10986068643865, 740.6757357151247, 1.4595922724681347], [65, 1349, 1354.0705972202122, -149124.53086920854, -220.26108708858766, 713.9117036082287, 1.4231729097611814], [75, 1353, 1376.0786884970978, -145142.6058435924, -210.95102635752872, 686.1516188506038, 1.3840267466573115], [64, 1373, 1393.1938667841257, -139093.02580742005, -199.6750475631722, 656.8122145328541, 1.3457701920108236], [88, 1397, 1412.0869614254686, -130929.11592940718, -185.4405847600736, 622.6904961762785, 1.3015518342393444], [63, 1417, 1425.814054697005, -120457.10843661502, -168.9660836766165, 581.6534802725506, 1.2518629640656904], [79, 1464, 1444.0282679852098, -110863.52072190335, -153.54757684430433, 540.9490190812529, 1.1996274442091504], [75, 1544, 1469.1220184586969, -102217.72985937215, -139.15485381753663, 502.19677116234135, 1.1459458483289577], [82, 1777, 1517.2967131979808, -94071.03950322726, -123.99821166811245, 460.6091441161893, 1.0799088257719134], [64, 2041, 1628.5344078075004, -87960.27279846894, -108.02384325043529, 412.88117866212633, 0.9868931734594801], [79, 1980, 1748.1958761658364, -83600.45850841807, -95.64198113974685, 370.44081509423626, 0.9022366011470442], [61, 1923, 1849.7212775517257, -79215.5161931164, -85.65130017638694, 333.14831312793893, 0.8318053981862149], [68, 1858, 1936.6869325205805, -74476.91519563725, -76.91167213970533, 297.1212680010481, 0.7677029865878799], [81, 1792, 2011.5537233241027, -71355.28650166595, -70.94544448333299, 266.8813216996032, 0.7139196624611215], [90, 1717, 2067.1792015753595, -69082.57819178466, -66.83753216860744, 243.135512211519, 0.6721886979974138], [81, 1636, 2104.682952595497, -67971.4506558445, -64.5906791538574, 227.44302335111297, 0.6443161830692872], [133, 1573, 2120.6448727175593, -66440.50064535323, -62.66065714266548, 212.70033353797547, 0.6207349281671649], [194, 1488, 2101.3805918458793, -63750.20300790121, -60.67459008165886, 197.99690795295922, 0.6016347557247406], [422, 1407, 2015.6070777639507, -59732.04808340092, -59.269535955058984, 186.69098337652855, 0.5965059852352873]], 'totalCount': 4961, 'weightedCount': 89487.2330131929, 'minAxis': 0, 'maxAxis': 1}}, 'bounds': [0.4265894850170295, 0.6757876453869698], 'bestTools': ['DecisionTool1', 'DecisionTool2', 'DecisionTool1'], '_className': 'ConfidenceDecisionNode', '_version': 1}, 'state': {}, 'minCasesPerPartition': 70, 'curvesPerTool': {'DecisionTool1': {'name': 'ScalarAxisPerformanceCurve', 'description': 'average performance over ml', 'headerLabels': ['count', 'totalCount', 'weightedCount', 'weightedSum', 'weightedAverage', 'weightedStdDev', 'confidence95'], 'rowLabels': [0.005, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095, 0.105, 0.115, 0.125, 0.135, 0.145, 0.155, 0.165, 0.175, 0.185, 0.195, 0.205, 0.215, 0.225, 0.235, 0.245, 0.255, 0.265, 0.275, 0.285, 0.295, 0.305, 0.315, 0.325, 0.335, 0.345, 0.355, 0.365, 0.375, 0.385, 0.395, 0.405, 0.415, 0.425, 0.435, 0.445, 0.455, 0.465, 0.475, 0.485, 0.495, 0.505, 0.515, 0.525, 0.535, 0.545, 0.555, 0.565, 0.575, 0.585, 0.595, 0.605, 0.615, 0.625, 0.635, 0.645, 0.655, 0.665, 0.675, 0.685, 0.695, 0.705, 0.715, 0.725, 0.735, 0.745, 0.755, 0.765, 0.775, 0.785, 0.795, 0.805, 0.815, 0.825, 0.835, 0.845, 0.855, 0.865, 0.875, 0.885, 0.895, 0.905, 0.915, 0.925, 0.935, 0.945, 0.955, 0.965, 0.975, 0.985, 0.995], 'content': [[79, 362, 489.9275345206288, 528373.39815652, 2156.945102803861, 658.785048928632, 2.2728041542043456], [87, 388, 530.9121080279357, 569138.53995202, 2144.0028635477, 662.9110460432803, 2.190142557446295], [30, 422, 546.8421381860971, 580378.5895550667, 2122.6549639360696, 676.9707102782401, 2.180770805190957], [28, 443, 554.7489493340258, 582221.6918374808, 2099.0456765585077, 693.5734354500629, 2.1915634690318844], [26, 472, 557.8916530102493, 577639.1883111263, 2070.793442398086, 716.40539552667, 2.221061394804876], [24, 505, 556.0899019777773, 565974.4117626875, 2035.5500423573783, 738.2898064130303, 2.258379916499804], [18, 525, 549.6626098096364, 548535.25676137, 1995.898018063638, 757.9904402981722, 2.3016529379910637], [18, 553, 540.8611569106579, 529494.6160377273, 1957.9687292100803, 764.2543063427773, 2.329872322383725], [15, 581, 530.0420757234103, 508414.0736477782, 1918.3913765860423, 762.1691509069773, 2.3503177561564637], [29, 609, 518.6310228511693, 486293.60343946645, 1875.297010834685, 775.1727596720058, 2.396216625558494], [20, 587, 506.1339526101943, 463352.2768840791, 1830.9472205708196, 795.2519593711303, 2.456833410168547], [34, 535, 503.6596155658367, 449667.23729545856, 1785.5997320344202, 834.5962090928206, 2.523049147460784], [18, 504, 509.2044422701001, 445168.22728227155, 1748.4852461131456, 879.9830874470028, 2.576600829083228], [37, 502, 516.0882146239277, 441558.81601715524, 1711.1757389729125, 924.3813281791207, 2.62312920509842], [24, 507, 521.8740411460395, 436429.3628947749, 1672.5467391954294, 963.7891377552237, 2.663570753317751], [27, 492, 525.7403756707907, 429770.4956558691, 1634.9153138848278, 1009.1900205830974, 2.7155442212771774], [24, 496, 527.9288654267792, 420709.3165990254, 1593.8106216598812, 1061.5442928153236, 2.779312796560511], [30, 504, 528.647108361125, 408948.83627011464, 1547.1524569117928, 1118.4339229647235, 2.8508758587975382], [31, 505, 526.9675245165822, 393987.0387958554, 1495.2991236311289, 1182.9219147760334, 2.936582350720672], [25, 504, 520.1420621842151, 374597.21251441934, 1440.3650069805385, 1239.8114453354342, 3.0260277361461285], [27, 494, 510.1043381586664, 352216.7903412083, 1380.9597919226178, 1295.6937595631246, 3.1237606255920087], [24, 481, 497.3211818471546, 330068.3230498974, 1327.3849379346955, 1338.545945115802, 3.215542353206572], [25, 462, 483.73101902753115, 308791.20447355456, 1276.7062368434965, 1374.616018905163, 3.304036246512993], [25, 462, 469.59355347305564, 288081.2890982294, 1226.9388579447734, 1409.5919451327707, 3.3957969418055014], [19, 452, 453.5758971214299, 267861.55097135017, 1181.1101633543774, 1446.1565955575104, 3.499763911766855], [22, 435, 439.04738725721785, 249490.75053481138, 1136.5094419233892, 1477.6440068164616, 3.5957151603470576], [24, 433, 424.69398812651576, 232786.4749704444, 1096.2550988647272, 1507.0433961241615, 3.6921632382625518], [24, 413, 409.28343667089933, 216745.38649668783, 1059.145653484974, 1533.9323269484678, 3.7944347296061847], [18, 413, 392.48070008605765, 201976.85479687096, 1029.2320348622713, 1553.0004188528424, 3.8988156890348833], [17, 410, 380.1750897437334, 192209.40845049458, 1011.1625597566541, 1558.0836211661028, 3.967889999967082], [17, 407, 372.5680188298226, 185584.6177343058, 996.2455624462765, 1556.1893708339444, 4.005756144006946], [13, 410, 369.03065403029325, 180241.2597214069, 976.8362478994015, 1556.896190809806, 4.0258230113662945], [20, 419, 370.25391985848546, 178568.44827816729, 964.572898223024, 1551.1456145491998, 4.01173764224286], [13, 411, 374.1286192186176, 177950.9004535111, 951.2819458996138, 1545.475550758689, 3.9836087284778947], [20, 410, 381.0866052798922, 177665.24565131846, 932.4140139789524, 1547.5858945560117, 3.949768215474291], [12, 413, 389.28726483807003, 178129.79302731622, 915.1585942654046, 1555.6972229393011, 3.9181721319815876], [19, 409, 399.6015967302028, 180446.47889838862, 903.1319212681718, 1558.3295686577894, 3.8705449866746617], [13, 408, 412.4199087359011, 182449.2839041892, 884.774376985875, 1569.0372123391921, 3.82298753791692], [28, 422, 430.01258087716934, 184331.57416022065, 857.3310752174199, 1586.2322410261206, 3.764426975338503], [28, 437, 446.0132111277429, 181632.7339815276, 814.4724391560951, 1617.2026749655554, 3.7321959340386877], [28, 463, 461.919537695125, 176918.5191514876, 766.0144450017048, 1649.0650364280868, 3.7033247474222355], [26, 486, 478.0187630884355, 167497.3768053623, 700.7983357104105, 1674.341949812775, 3.668222569871048], [28, 501, 493.2074170149863, 150626.44979568172, 610.8036683929466, 1699.9449868221423, 3.638804351370104], [21, 515, 507.0566467501217, 128006.89097588649, 504.90173749351715, 1719.4491861987228, 3.609296033979238], [25, 531, 523.5474086277183, 103179.33207989857, 394.1546854384943, 1719.187709577509, 3.5517281145822626], [17, 569, 542.0604638542986, 74333.75186660636, 274.26369131612836, 1701.0571716300822, 3.472095486824801], [24, 586, 565.8530931267888, 41025.23590019717, 145.0031338469852, 1665.5795264397334, 3.362690609853125], [26, 617, 591.2417580429465, 6493.8706857647585, 21.96688781678732, 1610.4343161865759, 3.2347820879718974], [34, 637, 616.3925462942572, -29701.094528910784, -96.37071281109196, 1535.5368460401207, 3.093552588655884], [35, 656, 644.7653608489765, -66171.51819894271, -205.25767113733667, 1440.7250605726038, 2.929852641671837], [42, 680, 671.3958571318558, -103183.91411540253, -307.3713161001474, 1329.2123724918335, 2.757807385757026], [33, 693, 697.5982532478866, -135505.83556340562, -388.49247380570654, 1220.5099535208813, 2.5925314999430826], [31, 715, 724.8603003658354, -160969.04719347548, -444.1381245799632, 1113.676373256834, 2.429452527007756], [37, 740, 755.4656619962318, -179948.87133719213, -476.39192722988304, 1012.8245618647669, 2.269424918486337], [41, 776, 785.6776376809897, -194521.96007833135, -495.1699036579007, 920.283277837597, 2.1212636042219333], [50, 806, 815.1952161300952, -204278.8462404887, -501.1777355864372, 839.2730788884588, 1.988734951677985], [32, 841, 839.8752498216907, -208882.80163082902, -497.41387587067436, 767.3540609300209, 1.8734692861356317], [43, 866, 864.6347760818899, -210343.26264705067, -486.548236239642, 705.7028718532548, 1.77072314598558], [50, 881, 889.0974369395549, -208978.72114848054, -470.0918312571582, 659.757124817531, 1.6883925869793015], [37, 911, 909.818261859937, -203137.2544882157, -446.5446848097843, 629.1451982529911, 1.629874592147328], [49, 951, 936.4902138557305, -194582.39520166066, -415.55670806323394, 614.598196487518, 1.5878156946713144], [50, 984, 967.3467242900265, -185096.28764315546, -382.6886120465335, 596.5000158931228, 1.5391119794869674], [50, 1033, 1000.0107291210443, -175647.0345820629, -351.29030012797404, 577.7082159363395, 1.4897315433332088], [46, 1053, 1033.7052764605755, -165720.5644176394, -320.63406890031445, 558.1006932083352, 1.4401707488968587], [59, 1072, 1063.8112291172135, -155666.80120397452, -292.6586915860102, 538.5539416570883, 1.3945638097244366], [57, 1081, 1092.8242930419751, -145105.10281584447, -265.55980451703044, 517.0760588283993, 1.348211775473957], [50, 1108, 1120.1225627571314, -134336.0151403179, -239.85949324983838, 494.51239140178416, 1.3023025410254596], [53, 1128, 1149.296166934073, -123365.95810146579, -214.68088322362328, 470.6366891346173, 1.2542467440956997], [47, 1149, 1177.532691988352, -111631.02615723347, -189.60157440509974, 444.2661597035806, 1.2039021819417512], [74, 1170, 1209.2702154651274, -101506.24624048946, -167.8801725906176, 418.8546817970011, 1.153522527255387], [74, 1195, 1234.538417451084, -91179.83430510962, -147.71485928054958, 392.9544273593543, 1.1057956059582097], [70, 1213, 1251.2971808176467, -80619.355782661, -128.85724833165716, 367.3306790138121, 1.061950904387979], [79, 1219, 1260.773050757125, -71003.67284553041, -112.63513731181195, 343.46884907864427, 1.023013324212357], [52, 1241, 1262.8039138142037, -62341.073521201084, -98.73436855751355, 320.37053242844934, 0.9872210186599369], [57, 1243, 1269.3849869515752, -54510.90734107568, -85.88553969270345, 296.29117004924404, 0.9469318996938341], [51, 1270, 1275.8706639830027, -47744.1368813903, -74.84165633582802, 274.33005448412746, 0.908844200625387], [76, 1298, 1288.6035830840453, -42475.56594492333, -65.92495396181587, 255.3885890794162, 0.8725637135715629], [60, 1310, 1296.0470336526632, -37822.89467202044, -58.36654641371119, 237.29015825471149, 0.8386592726873059], [67, 1347, 1305.2835923705268, -33938.886410576844, -52.00231828386106, 220.66014498338728, 0.805871100079928], [68, 1366, 1313.2979664422573, -29678.760445998745, -45.19729902026564, 202.74988370570935, 0.7701135102188484], [69, 1353, 1323.3561954919246, -25913.674951351622, -39.16356766171916, 186.38405879237288, 0.7355667277360556], [50, 1353, 1335.6022728465473, -23207.87364784284, -34.7526716892751, 174.88151367038213, 0.7092338389058843], [65, 1349, 1354.0705972202122, -20491.329672716583, -30.266264867996515, 161.71154184916142, 0.6773387570091182], [75, 1353, 1376.0786884970978, -18235.018968537406, -26.502872431594763, 150.49194886044586, 0.6481731996173935], [64, 1373, 1393.1938667841257, -16384.895803266514, -23.521343574512507, 141.65716294629786, 0.6249849569516799], [88, 1397, 1412.0869614254686, -14656.141619441627, -20.758128953540645, 131.89973399813903, 0.599028257795916], [63, 1417, 1425.814054697005, -12682.216362065636, -17.78943940170472, 119.69409614259874, 0.5678857753811023], [79, 1464, 1444.0282679852098, -11012.190251958522, -15.252042492663048, 109.11707535089707, 0.5387838662926596], [75, 1544, 1469.1220184586969, -9571.557911037904, -13.030310336073695, 99.62451900000809, 0.5103995187945511], [82, 1777, 1517.2967131979808, -8531.488769990236, -11.245643249313524, 91.0081649198034, 0.48002183358919515], [64, 2041, 1628.5344078075004, -7685.332903712331, -9.438342680225114, 81.74875237837368, 0.4391351172251292], [79, 1980, 1748.1958761658364, -6991.652568614631, -7.998706167810903, 73.49692931859745, 0.40187928694254926], [61, 1923, 1849.7212775517257, -6472.956543712446, -6.998845309580903, 67.75010089333598, 0.37510935760394054], [68, 1858, 1936.6869325205805, -5843.628473241459, -6.034665051037474, 61.912162870862716, 0.3504405460001962], [81, 1792, 2011.5537233241027, -5021.949222769537, -4.9931047473797925, 54.95447793078099, 0.32396030308080165], [90, 1717, 2067.1792015753595, -4283.750630297966, -4.14453727769068, 49.30804544628199, 0.30270939616331516], [81, 1636, 2104.682952595497, -3714.6864940316605, -3.5299250078979405, 45.705346500872544, 0.28883261520344355], [133, 1573, 2120.6448727175593, -3102.9772614027374, -2.926446857107518, 41.48135961018739, 0.27412496499829425], [194, 1488, 2101.3805918458793, -2535.797035917185, -2.413458129152805, 37.681220684406995, 0.26246185538723726], [422, 1407, 2015.6070777639507, -1995.5167036367493, -1.9800651879537068, 34.063555527271824, 0.2547992367044842]], 'totalCount': 4961, 'weightedCount': 89487.2330131929, 'minAxis': 0, 'maxAxis': 1}, 'DecisionTool2': {'name': 'ScalarAxisPerformanceCurve', 'description': 'average performance over ml', 'headerLabels': ['count', 'totalCount', 'weightedCount', 'weightedSum', 'weightedAverage', 'weightedStdDev', 'confidence95'], 'rowLabels': [0.005, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095, 0.105, 0.115, 0.125, 0.135, 0.145, 0.155, 0.165, 0.175, 0.185, 0.195, 0.205, 0.215, 0.225, 0.235, 0.245, 0.255, 0.265, 0.275, 0.285, 0.295, 0.305, 0.315, 0.325, 0.335, 0.345, 0.355, 0.365, 0.375, 0.385, 0.395, 0.405, 0.415, 0.425, 0.435, 0.445, 0.455, 0.465, 0.475, 0.485, 0.495, 0.505, 0.515, 0.525, 0.535, 0.545, 0.555, 0.565, 0.575, 0.585, 0.595, 0.605, 0.615, 0.625, 0.635, 0.645, 0.655, 0.665, 0.675, 0.685, 0.695, 0.705, 0.715, 0.725, 0.735, 0.745, 0.755, 0.765, 0.775, 0.785, 0.795, 0.805, 0.815, 0.825, 0.835, 0.845, 0.855, 0.865, 0.875, 0.885, 0.895, 0.905, 0.915, 0.925, 0.935, 0.945, 0.955, 0.965, 0.975, 0.985, 0.995], 'content': [[79, 362, 489.9275345206288, 318753.40353122336, 1301.2267369014426, 1814.3532309799205, 3.7718209514318786], [87, 388, 530.9121080279357, 339352.7818411534, 1278.3765022865036, 1817.2180729767817, 3.626170997536704], [30, 422, 546.8421381860971, 345142.81425655755, 1262.312430426132, 1807.6975153524047, 3.5635919197663184], [28, 443, 554.7489493340258, 345240.7674118468, 1244.6738937543084, 1797.3592819860105, 3.52797319339893], [26, 472, 557.8916530102493, 341538.3506844893, 1224.3895345686942, 1785.2162400918642, 3.5061182067416397], [24, 505, 556.0899019777773, 334473.2211103046, 1202.946573641148, 1765.5537121066943, 3.4924004387631635], [18, 525, 549.6626098096364, 323175.3452755852, 1175.9044166657432, 1744.4735364407343, 3.491726077389597], [18, 553, 540.8611569106579, 309474.2259533937, 1144.3758606037748, 1725.3424119697556, 3.5006671891602865], [15, 581, 530.0420757234103, 294517.38826554193, 1111.2981469011856, 1703.2743207198682, 3.513526211144509], [29, 609, 518.6310228511693, 279125.3432295607, 1076.3927761014822, 1683.629302184823, 3.531425620821245], [20, 587, 506.1339526101943, 264205.48894858314, 1044.0140898908019, 1663.5335987889434, 3.5533593021075185], [34, 535, 503.6596155658367, 256204.39565661715, 1017.3712076112522, 1645.366229275817, 3.542572901061437], [18, 504, 509.2044422701001, 253036.55743951074, 993.850549737707, 1637.239873520991, 3.5145209457429263], [37, 502, 516.0882146239277, 249868.29287710573, 968.3162133790797, 1630.9989538398588, 3.484343330727361], [24, 507, 521.8740411460395, 247536.68952945876, 948.6453435617002, 1619.3104244734993, 3.4525365091796507], [27, 492, 525.7403756707907, 245081.13271632013, 932.327605250488, 1610.390556007328, 3.4303308832763726], [24, 496, 527.9288654267792, 242683.27251125607, 919.3786830165849, 1600.4906980023409, 3.4126751047537893], [30, 504, 528.647108361125, 241119.06306000534, 912.2117921253969, 1585.1536207436884, 3.393976409697896], [31, 505, 526.9675245165822, 239810.0452873643, 910.1511350527945, 1568.1687855304249, 3.3811197142840608], [25, 504, 520.1420621842151, 235219.0150748724, 904.4414292784749, 1547.3867206848652, 3.3806056777699744], [27, 494, 510.1043381586664, 229108.74673925876, 898.2818988220217, 1523.9173001371985, 3.3877180848304027], [24, 481, 497.3211818471546, 222415.21828926954, 894.4530271691748, 1500.124762280971, 3.4040919518500337], [25, 462, 483.73101902753115, 214690.32476572096, 887.6434064423809, 1479.2796495714824, 3.4275140530561217], [25, 462, 469.59355347305564, 207887.66302328324, 885.3940241971887, 1457.1865178578198, 3.452650197658323], [19, 452, 453.5758971214299, 198594.42118849012, 875.6833087862385, 1446.1087962751822, 3.4997060730839102], [22, 435, 439.04738725721785, 188788.7705042139, 859.9926840863361, 1439.0086397049074, 3.548395939039697], [24, 433, 424.69398812651576, 178861.42016222273, 842.307285541043, 1435.1956179203214, 3.6030771758653], [24, 413, 409.28343667089933, 168230.00443232918, 822.0709139891298, 1434.3750971951522, 3.669233542077721], [18, 413, 392.48070008605765, 156254.00782268998, 796.2379183915479, 1434.7037319806195, 3.7473825712395192], [17, 410, 380.1750897437334, 147454.3887442784, 775.7183083387907, 1434.9679687166113, 3.8078983468613017], [17, 407, 372.5680188298226, 139756.0542778182, 750.2310837992479, 1434.6897372248777, 3.8462036971302553], [13, 410, 369.03065403029325, 133642.43689333685, 724.2890824043377, 1431.6152645142809, 3.8604506949717177], [20, 419, 370.25391985848546, 130713.67615577232, 706.075853056369, 1425.9547331008505, 3.84644129324641], [13, 411, 374.1286192186176, 128417.2324999202, 686.487084404955, 1421.775751456342, 3.820860284129973], [20, 410, 381.0866052798922, 128366.9776545019, 673.689265778427, 1413.3693359323645, 3.774609787305215], [12, 413, 389.28726483807003, 129618.41477416016, 665.9268179660432, 1406.824357156498, 3.7259833734040955], [19, 409, 399.6015967302028, 132830.64633075582, 664.8153932199549, 1401.5002592853004, 3.6706168602479696], [13, 408, 412.4199087359011, 136379.54767642223, 661.3625811345339, 1399.2318234320837, 3.6101986330391407], [28, 422, 430.01258087716934, 141224.47172888304, 656.8387903479643, 1395.921573584683, 3.5313925787033194], [28, 437, 446.0132111277429, 143912.18384000094, 645.3270004093353, 1391.6687354115795, 3.462184001832032], [28, 463, 461.919537695125, 146206.216319214, 633.0375937279046, 1387.1515541685912, 3.396525361855946], [26, 486, 478.0187630884355, 147496.4247230944, 617.1156285587348, 1382.1041701974216, 3.3327596005225972], [28, 501, 493.2074170149863, 147245.13488017127, 597.0921352778325, 1374.7539722337312, 3.272304905838548], [21, 515, 507.0566467501217, 146400.99556683903, 577.4542016367086, 1362.8193491570314, 3.213268205690713], [25, 531, 523.5474086277183, 145190.52695144256, 554.6413736704559, 1348.0584944414793, 3.1450853298976598], [17, 569, 542.0604638542986, 143017.35379610592, 527.680446491843, 1329.6099087890952, 3.0696886374705854], [24, 586, 565.8530931267888, 140339.31037094447, 496.0273685011, 1303.84427359178, 2.9752063085559364], [26, 617, 591.2417580429465, 137402.1231021723, 464.7916735684685, 1274.998302013928, 2.8782487499123044], [34, 637, 616.3925462942572, 131671.17328572072, 427.23155585616945, 1252.3969617828777, 2.7938197610953766], [35, 656, 644.7653608489765, 124701.13900792014, 386.8109131784731, 1237.788156066643, 2.7156786874644894], [42, 680, 671.3958571318558, 116879.31532732684, 348.16811598041437, 1222.7947762668675, 2.6451087380431897], [33, 693, 697.5982532478866, 108449.17789308644, 310.921586710309, 1211.0681823039138, 2.5824842187312353], [31, 715, 724.8603003658354, 98246.69970028783, 271.0776121983862, 1199.7946676579872, 2.5216358937624905], [37, 740, 755.4656619962318, 86696.72212670733, 229.51863066183878, 1190.71992776003, 2.460670811804124], [41, 776, 785.6776376809897, 74717.72710443629, 190.19944954771407, 1182.2865292737313, 2.404336480360633], [50, 806, 815.1952161300952, 62516.20342157206, 153.3772578262904, 1173.8942229734967, 2.3520131353765548], [32, 841, 839.8752498216907, 46454.46299579605, 110.62229302663353, 1171.467674648572, 2.314801922205992], [43, 866, 864.6347760818899, 29277.37979147474, 67.72195752788429, 1170.9257641507115, 2.280890323741416], [50, 881, 889.0974369395549, 13799.560620240107, 31.041728492078125, 1166.4016639201586, 2.2449437180262968], [37, 911, 909.818261859937, -405.7415502734907, -0.89191779783368, 1154.998972298582, 2.208358338348539], [49, 951, 936.4902138557305, -16045.477488141098, -34.26726142087152, 1146.1498633956455, 2.1683288745280525], [50, 984, 967.3467242900265, -32902.56366077775, -68.02641252530466, 1137.6619177393218, 2.1255513388467775], [50, 1033, 1000.0107291210443, -49199.774247240995, -98.39849276514254, 1128.1710932292553, 2.081810647592656], [46, 1053, 1033.7052764605755, -64630.63670139185, -125.04654503203902, 1114.220600708587, 2.034901053935979], [59, 1072, 1063.8112291172135, -81225.10526322071, -152.70586179208513, 1101.468407297035, 1.9943887181253204], [57, 1081, 1092.8242930419751, -97520.28705854093, -178.47386387629507, 1088.832202324472, 1.9564167814227995], [50, 1108, 1120.1225627571314, -109892.82068875077, -196.2157077137247, 1073.0638632941902, 1.9183863850431708], [53, 1128, 1149.296166934073, -121562.13282411652, -211.54187462123448, 1053.639004543391, 1.8766617913566994], [47, 1149, 1177.532691988352, -133247.02939606033, -226.31563489088825, 1035.1344576326012, 1.8376718702312365], [74, 1170, 1209.2702154651274, -144108.50956168087, -238.3396328111029, 1019.4256489399991, 1.7995841995842998], [74, 1195, 1234.538417451084, -151700.88076638116, -245.7612960795398, 1000.2598566637229, 1.7642502456085107], [70, 1213, 1251.2971808176467, -155803.76422068168, -249.0275956969285, 973.802548675195, 1.7290648881748207], [79, 1219, 1260.773050757125, -157962.03119450936, -250.57964413127215, 949.3532460359693, 1.7007932854405359], [52, 1241, 1262.8039138142037, -159719.11820600333, -252.9594919033531, 929.0906455946342, 1.6811913834908785], [57, 1243, 1269.3849869515752, -159914.8959812941, -251.9564948776168, 906.4700902640192, 1.6562890548098426], [51, 1270, 1275.8706639830027, -160149.26933960884, -251.0431094005346, 883.9829626429846, 1.6314534731821777], [76, 1298, 1288.6035830840453, -161672.72154531284, -250.92700915572144, 863.9685285671559, 1.6048903492542628], [60, 1310, 1296.0470336526632, -161388.46506818145, -249.0472349808766, 843.9642153728931, 1.581640242689987], [67, 1347, 1305.2835923705268, -159623.2523235626, -244.58018664537207, 819.4361732662906, 1.5529633185904668], [68, 1366, 1313.2979664422573, -157364.70032600465, -239.6481291329611, 791.546278349176, 1.5216423645917616], [69, 1353, 1323.3561954919246, -155178.7869937261, -234.52308233013903, 764.0810484592214, 1.489317895180292], [50, 1353, 1335.6022728465473, -152332.02419575836, -228.10986068643865, 740.6757357151247, 1.4595922724681347], [65, 1349, 1354.0705972202122, -149124.53086920854, -220.26108708858766, 713.9117036082287, 1.4231729097611814], [75, 1353, 1376.0786884970978, -145142.6058435924, -210.95102635752872, 686.1516188506038, 1.3840267466573115], [64, 1373, 1393.1938667841257, -139093.02580742005, -199.6750475631722, 656.8122145328541, 1.3457701920108236], [88, 1397, 1412.0869614254686, -130929.11592940718, -185.4405847600736, 622.6904961762785, 1.3015518342393444], [63, 1417, 1425.814054697005, -120457.10843661502, -168.9660836766165, 581.6534802725506, 1.2518629640656904], [79, 1464, 1444.0282679852098, -110863.52072190335, -153.54757684430433, 540.9490190812529, 1.1996274442091504], [75, 1544, 1469.1220184586969, -102217.72985937215, -139.15485381753663, 502.19677116234135, 1.1459458483289577], [82, 1777, 1517.2967131979808, -94071.03950322726, -123.99821166811245, 460.6091441161893, 1.0799088257719134], [64, 2041, 1628.5344078075004, -87960.27279846894, -108.02384325043529, 412.88117866212633, 0.9868931734594801], [79, 1980, 1748.1958761658364, -83600.45850841807, -95.64198113974685, 370.44081509423626, 0.9022366011470442], [61, 1923, 1849.7212775517257, -79215.5161931164, -85.65130017638694, 333.14831312793893, 0.8318053981862149], [68, 1858, 1936.6869325205805, -74476.91519563725, -76.91167213970533, 297.1212680010481, 0.7677029865878799], [81, 1792, 2011.5537233241027, -71355.28650166595, -70.94544448333299, 266.8813216996032, 0.7139196624611215], [90, 1717, 2067.1792015753595, -69082.57819178466, -66.83753216860744, 243.135512211519, 0.6721886979974138], [81, 1636, 2104.682952595497, -67971.4506558445, -64.5906791538574, 227.44302335111297, 0.6443161830692872], [133, 1573, 2120.6448727175593, -66440.50064535323, -62.66065714266548, 212.70033353797547, 0.6207349281671649], [194, 1488, 2101.3805918458793, -63750.20300790121, -60.67459008165886, 197.99690795295922, 0.6016347557247406], [422, 1407, 2015.6070777639507, -59732.04808340092, -59.269535955058984, 186.69098337652855, 0.5965059852352873]], 'totalCount': 4961, 'weightedCount': 89487.2330131929, 'minAxis': 0, 'maxAxis': 1}, 'CompositeDecisionTool2': {'name': 'ScalarAxisPerformanceCurve', 'description': 'average performance over ml', 'headerLabels': ['count', 'totalCount', 'weightedCount', 'weightedSum', 'weightedAverage', 'weightedStdDev', 'confidence95'], 'rowLabels': [0.005, 0.015, 0.025, 0.035, 0.045, 0.055, 0.065, 0.075, 0.085, 0.095, 0.105, 0.115, 0.125, 0.135, 0.145, 0.155, 0.165, 0.175, 0.185, 0.195, 0.205, 0.215, 0.225, 0.235, 0.245, 0.255, 0.265, 0.275, 0.285, 0.295, 0.305, 0.315, 0.325, 0.335, 0.345, 0.355, 0.365, 0.375, 0.385, 0.395, 0.405, 0.415, 0.425, 0.435, 0.445, 0.455, 0.465, 0.475, 0.485, 0.495, 0.505, 0.515, 0.525, 0.535, 0.545, 0.555, 0.565, 0.575, 0.585, 0.595, 0.605, 0.615, 0.625, 0.635, 0.645, 0.655, 0.665, 0.675, 0.685, 0.695, 0.705, 0.715, 0.725, 0.735, 0.745, 0.755, 0.765, 0.775, 0.785, 0.795, 0.805, 0.815, 0.825, 0.835, 0.845, 0.855, 0.865, 0.875, 0.885, 0.895, 0.905, 0.915, 0.925, 0.935, 0.945, 0.955, 0.965, 0.975, 0.985, 0.995], 'content': [[79, 362, 489.9275345206288, 528373.39815652, 2156.945102803861, 658.785048928632, 2.2728041542043456], [87, 388, 530.9121080279357, 569138.53995202, 2144.0028635477, 662.9110460432803, 2.190142557446295], [30, 422, 546.8421381860971, 580378.5895550667, 2122.6549639360696, 676.9707102782401, 2.180770805190957], [28, 443, 554.7489493340258, 582221.6918374808, 2099.0456765585077, 693.5734354500629, 2.1915634690318844], [26, 472, 557.8916530102493, 577639.1883111263, 2070.793442398086, 716.40539552667, 2.221061394804876], [24, 505, 556.0899019777773, 565974.4117626875, 2035.5500423573783, 738.2898064130303, 2.258379916499804], [18, 525, 549.6626098096364, 548535.25676137, 1995.898018063638, 757.9904402981722, 2.3016529379910637], [18, 553, 540.8611569106579, 529494.6160377273, 1957.9687292100803, 764.2543063427773, 2.329872322383725], [15, 581, 530.0420757234103, 508414.0736477782, 1918.3913765860423, 762.1691509069773, 2.3503177561564637], [29, 609, 518.6310228511693, 486293.60343946645, 1875.297010834685, 775.1727596720058, 2.396216625558494], [20, 587, 506.1339526101943, 463352.2768840791, 1830.9472205708196, 795.2519593711303, 2.456833410168547], [34, 535, 503.6596155658367, 449667.23729545856, 1785.5997320344202, 834.5962090928206, 2.523049147460784], [18, 504, 509.2044422701001, 445168.22728227155, 1748.4852461131456, 879.9830874470028, 2.576600829083228], [37, 502, 516.0882146239277, 441558.81601715524, 1711.1757389729125, 924.3813281791207, 2.62312920509842], [24, 507, 521.8740411460395, 436429.3628947749, 1672.5467391954294, 963.7891377552237, 2.663570753317751], [27, 492, 525.7403756707907, 429770.4956558691, 1634.9153138848278, 1009.1900205830974, 2.7155442212771774], [24, 496, 527.9288654267792, 420709.3165990254, 1593.8106216598812, 1061.5442928153236, 2.779312796560511], [30, 504, 528.647108361125, 408948.83627011464, 1547.1524569117928, 1118.4339229647235, 2.8508758587975382], [31, 505, 526.9675245165822, 393987.0387958554, 1495.2991236311289, 1182.9219147760334, 2.936582350720672], [25, 504, 520.1420621842151, 374597.21251441934, 1440.3650069805385, 1239.8114453354342, 3.0260277361461285], [27, 494, 510.1043381586664, 352216.7903412083, 1380.9597919226178, 1295.6937595631246, 3.1237606255920087], [24, 481, 497.3211818471546, 330068.3230498974, 1327.3849379346955, 1338.545945115802, 3.215542353206572], [25, 462, 483.73101902753115, 308791.20447355456, 1276.7062368434965, 1374.616018905163, 3.304036246512993], [25, 462, 469.59355347305564, 288081.2890982294, 1226.9388579447734, 1409.5919451327707, 3.3957969418055014], [19, 452, 453.5758971214299, 267861.55097135017, 1181.1101633543774, 1446.1565955575104, 3.499763911766855], [22, 435, 439.04738725721785, 249490.75053481138, 1136.5094419233892, 1477.6440068164616, 3.5957151603470576], [24, 433, 424.69398812651576, 232786.4749704444, 1096.2550988647272, 1507.0433961241615, 3.6921632382625518], [24, 413, 409.28343667089933, 216745.38649668783, 1059.145653484974, 1533.9323269484678, 3.7944347296061847], [18, 413, 392.48070008605765, 201976.85479687096, 1029.2320348622713, 1553.0004188528424, 3.8988156890348833], [17, 410, 380.1750897437334, 192209.40845049458, 1011.1625597566541, 1558.0836211661028, 3.967889999967082], [17, 407, 372.5680188298226, 185584.6177343058, 996.2455624462765, 1556.1893708339444, 4.005756144006946], [13, 410, 369.03065403029325, 180241.2597214069, 976.8362478994015, 1556.896190809806, 4.0258230113662945], [20, 419, 370.25391985848546, 178568.44827816729, 964.572898223024, 1551.1456145491998, 4.01173764224286], [13, 411, 374.1286192186176, 178019.02351665584, 951.6461151165372, 1544.7905058849346, 3.982725746792513], [20, 410, 381.0866052798922, 178180.43282484976, 935.1177939932246, 1538.265023149094, 3.937855829057041], [12, 413, 389.28726483807003, 178607.09414137423, 917.6107737080408, 1534.8234721525055, 3.891797092533971], [19, 409, 399.6015967302028, 181275.72181853207, 907.2822696498041, 1521.9160853629528, 3.825056173959052], [13, 408, 412.4199087359011, 184215.22949081488, 893.3382001632694, 1510.3637977474611, 3.750827091239024], [28, 422, 430.01258087716934, 188125.70031310664, 874.977657301816, 1496.6485375680359, 3.6565827254545913], [28, 437, 446.0132111277429, 188372.40953467754, 844.6943042712952, 1489.6163678216742, 3.581949244836751], [28, 463, 461.919537695125, 187766.4692791677, 812.9834482259847, 1477.6028787612854, 3.505514514948268], [26, 486, 478.0187630884355, 185799.382803565, 777.3727608645828, 1459.515582635572, 3.4248217076073066], [28, 501, 493.2074170149863, 180718.54877243153, 732.8298096820403, 1442.0644461856248, 3.3514566522993063], [21, 515, 507.0566467501217, 174257.51241223497, 687.3295657560302, 1420.007465567701, 3.279994724249153], [25, 531, 523.5474086277183, 168269.42091889345, 642.804904182214, 1393.4322201110504, 3.197576830682296], [17, 569, 542.0604638542986, 162037.06522257885, 597.8560549146896, 1362.8195026160395, 3.1077879261886654], [24, 586, 565.8530931267888, 155294.29680842478, 548.8855630365124, 1329.0058037342908, 3.0037768300817875], [26, 617, 591.2417580429465, 148777.49415603158, 503.27126638854463, 1292.640568224291, 2.8980936303567453], [34, 637, 616.3925462942572, 139696.13419888844, 453.2700307255158, 1263.4490561102266, 2.806120069101703], [35, 656, 644.7653608489765, 130065.45545554823, 403.4505057290554, 1242.4256297518202, 2.7207611866096544], [42, 680, 671.3958571318558, 119740.91137676695, 356.69243444036016, 1223.8211421904869, 2.6462186054061045], [33, 693, 697.5982532478866, 109390.04886632507, 313.6190446493394, 1211.3669401275818, 2.582802734979718], [31, 715, 724.8603003658354, 98303.60272796861, 271.2346163208414, 1199.7591789722733, 2.521598599795492], [37, 740, 755.4656619962318, 86696.72212670733, 229.51863066183878, 1190.71992776003, 2.460670811804124], [41, 776, 785.6776376809897, 74717.72710443629, 190.19944954771407, 1182.2865292737313, 2.404336480360633], [50, 806, 815.1952161300952, 62516.20342157206, 153.3772578262904, 1173.8942229734967, 2.3520131353765548], [32, 841, 839.8752498216907, 46454.46299579605, 110.62229302663353, 1171.467674648572, 2.314801922205992], [43, 866, 864.6347760818899, 29277.37979147474, 67.72195752788429, 1170.9257641507115, 2.280890323741416], [50, 881, 889.0974369395549, 13389.733984110855, 30.119834852298993, 1165.6058622593746, 2.244177757713186], [37, 911, 909.818261859937, -209.48340979945743, -0.46049506496212006, 1148.843304410066, 2.202465657159962], [49, 951, 936.4902138557305, -14279.697522191305, -30.496202332747796, 1127.963375326275, 2.1510571492078165], [50, 984, 967.3467242900265, -28022.421450376827, -57.93666478985304, 1096.747340200402, 2.0869799829758806], [50, 1033, 1000.0107291210443, -39621.52430835303, -79.24219841756742, 1060.7702680112993, 2.0186657326814363], [46, 1053, 1033.7052764605755, -50113.54686021638, -96.95906173915648, 1018.5204926634369, 1.9455508691646353], [59, 1072, 1063.8112291172135, -60479.17573377262, -113.70283388333903, 973.3909663874034, 1.8748539493806582], [57, 1081, 1092.8242930419751, -69640.8776172667, -127.4511887421811, 924.303360849239, 1.8025535150710006], [50, 1108, 1120.1225627571314, -73985.77575197704, -132.10300053212816, 867.9319343772065, 1.725306000006724], [53, 1128, 1149.296166934073, -76297.51868897212, -132.7725974976627, 801.0376486556369, 1.6363136618300187], [47, 1149, 1177.532691988352, -76436.29617051016, -129.82449946496476, 727.880546009649, 1.540989362505013], [74, 1170, 1209.2702154651274, -76656.1265265465, -126.78080638422388, 660.3789623149348, 1.4484083104430767], [74, 1195, 1234.538417451084, -74500.76760442801, -120.69412591994927, 593.4931172066586, 1.3589754465065695], [70, 1213, 1251.2971808176467, -70136.57240337285, -112.102182404891, 531.1040939203452, 1.2769258028984019], [79, 1219, 1260.773050757125, -65639.39201743425, -104.1256266986611, 472.00749228005793, 1.1992564390667395], [52, 1241, 1262.8039138142037, -60157.88904201765, -95.27669083684623, 416.6372298078085, 1.1258153070815393], [57, 1243, 1269.3849869515752, -53788.95600657187, -84.7480576176435, 361.7081045568467, 1.0462573271653066], [51, 1270, 1275.8706639830027, -47429.486212058866, -74.34842347420056, 310.0725276315668, 0.9662386160042501], [76, 1298, 1288.6035830840453, -42506.833788260345, -65.97348377152225, 267.52852740282077, 0.8930616774158263], [60, 1310, 1296.0470336526632, -37807.00557993193, -58.342027099711096, 237.35129180097877, 0.8387672984774172], [67, 1347, 1305.2835923705268, -33938.886410576844, -52.00231828386106, 220.66014498338728, 0.805871100079928], [68, 1366, 1313.2979664422573, -29678.760445998745, -45.19729902026564, 202.74988370570935, 0.7701135102188484], [69, 1353, 1323.3561954919246, -25913.674951351622, -39.16356766171916, 186.38405879237288, 0.7355667277360556], [50, 1353, 1335.6022728465473, -23207.87364784284, -34.7526716892751, 174.88151367038213, 0.7092338389058843], [65, 1349, 1354.0705972202122, -20491.329672716583, -30.266264867996515, 161.71154184916142, 0.6773387570091182], [75, 1353, 1376.0786884970978, -18235.018968537406, -26.502872431594763, 150.49194886044586, 0.6481731996173935], [64, 1373, 1393.1938667841257, -16384.895803266514, -23.521343574512507, 141.65716294629786, 0.6249849569516799], [88, 1397, 1412.0869614254686, -14656.141619441627, -20.758128953540645, 131.89973399813903, 0.599028257795916], [63, 1417, 1425.814054697005, -12682.216362065636, -17.78943940170472, 119.69409614259874, 0.5678857753811023], [79, 1464, 1444.0282679852098, -11012.190251958522, -15.252042492663048, 109.11707535089707, 0.5387838662926596], [75, 1544, 1469.1220184586969, -9571.557911037904, -13.030310336073695, 99.62451900000809, 0.5103995187945511], [82, 1777, 1517.2967131979808, -8531.488769990236, -11.245643249313524, 91.0081649198034, 0.48002183358919515], [64, 2041, 1628.5344078075004, -7685.332903712331, -9.438342680225114, 81.74875237837368, 0.4391351172251292], [79, 1980, 1748.1958761658364, -6991.652568614631, -7.998706167810903, 73.49692931859745, 0.40187928694254926], [61, 1923, 1849.7212775517257, -6472.956543712446, -6.998845309580903, 67.75010089333598, 0.37510935760394054], [68, 1858, 1936.6869325205805, -5843.628473241459, -6.034665051037474, 61.912162870862716, 0.3504405460001962], [81, 1792, 2011.5537233241027, -5021.949222769537, -4.9931047473797925, 54.95447793078099, 0.32396030308080165], [90, 1717, 2067.1792015753595, -4283.750630297966, -4.14453727769068, 49.30804544628199, 0.30270939616331516], [81, 1636, 2104.682952595497, -3714.6864940316605, -3.5299250078979405, 45.705346500872544, 0.28883261520344355], [133, 1573, 2120.6448727175593, -3102.9772614027374, -2.926446857107518, 41.48135961018739, 0.27412496499829425], [194, 1488, 2101.3805918458793, -2535.797035917185, -2.413458129152805, 37.681220684406995, 0.26246185538723726], [422, 1407, 2015.6070777639507, -1995.5167036367493, -1.9800651879537068, 34.063555527271824, 0.2547992367044842]], 'totalCount': 4961, 'weightedCount': 89487.2330131929, 'minAxis': 0, 'maxAxis': 1}}, '_className': 'DefaultDispatchConfiguration', '_version': 1}, 'isEditable': True, '_className': 'CompositeDecisionTool', '_version': 1}}, 'selectedConfiguration': 'CompositeDecisionTool2', 'indicatorsShown': ['Performance', 'Impact', 'Decision Cost', 'ml volume', 'human volume'], 'status': 'Active', '_className': 'Scenario', '_version': 1}}, 'selectedScenario': 'Scenario0', '_className': 'DefaultTaskModel', '_version': 1}
    
    # Function used to prcoess the input values from the user and preparing it as a payload for the model_decision.    
    def get_payload(input_values):
        result = {"fields": [], "values": [[]]}        
        for attribute in input_values[0]:
            result["fields"].append(attribute)            
        for value in input_values[1]:
            result["values"][0].append(value)            
        return result
    
    # Function for creating a case source for the dispatch    
    def create_case(input_values):        
        case = {
            "case": {
                "caseId": "CaseDataModel1"
            }
        }                
        for attribute in range(len(input_values[0])):
            case["case"][input_values[0][attribute]] = input_values[1][attribute]                        
        return case
    
    def model_decision(input_values):        
        result = get_payload(input_values)        
        header = {"Content-Type": "application/json", "Authorization": "Bearer " + token}        
        payload_scoring= {"input_data": [
                {
                        "fields": result["fields"],
                        "values": result["values"]
                        }
                  ]
        }                
        response_scoring = requests.post(input_values[2], json=payload_scoring, headers=header,verify=False)        
        case = create_case(input_values)        
        case["decisionTools"] = {
              "ml": {
                "outcome": json.loads(response_scoring.text)['predictions'][0]['values'][0][0],
                "confidence": json.loads(response_scoring.text)['predictions'][0]['values'][0][1][0]
              }
            }
        return case
    
    # Function for loading the model from user and passing the case source. pass json -> deserialize -> dispatch.            
    def dispatch(model, case):        
        load_model = aidc.load_task_model(model)        
        dispatch_result = aidc.dispatch(load_model, case)        
        return dispatch_result
    
    # Case -> dispatch using default model
    def dispatch_with_default(case):    
        load_model = aidc.load_task_model(default_model) 
        dispatch_result = aidc.dispatch(load_model, case)        
        return dispatch_result
        
    def score( payload ):                
        if len(payload["input_data"][0]["values"][0]) == 2: #json uploaded
            ml_results = model_decision(payload["input_data"][0]["values"][0][1])
            model_json=payload["input_data"][0]["values"][0][0]
            dispatch_result = dispatch(model_json, ml_results)            
            data = dispatch_result.valueOf()
        elif len(payload["input_data"][0]["values"][0]) == 1:
            ml_results = model_decision(payload["input_data"][0]["values"][0][0])            
            dispatch_result = dispatch_with_default(ml_results)            
            data = dispatch_result.valueOf()
        else:
            data = "Missing data"        
        # Score using the pre-defined model
        response = {
            'predictions': [{'fields': ['AIDC Decision'], 
                             'values': [[data]]}]
        } 
        return response
    
    return score

### Test the function offline

In [7]:
#Test custom function
test_values_for_ml = [["CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose", "LoanAmount", "ExistingSavings", "EmploymentDuration", "InstallmentPercent", "Sex", "OthersOnLoan", "CurrentResidenceDuration", "OwnsProperty", "Age", "InstallmentPlans", "Housing", "ExistingCreditsCount", "Job", "Dependents", "Telephone", "ForeignWorker"], 
                  ["no_checking", 8, "prior_payments_delayed", "repairs", 3094, "500_to_1000", "greater_7", 3, "male", "none", 3, "savings_insurance", 39, "none", "own", 1, "skilled", 1, "yes", "yes"],
                   model_serving_url]

test_values_for_human = [["CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose", "LoanAmount", "ExistingSavings", "EmploymentDuration", "InstallmentPercent", "Sex", "OthersOnLoan", "CurrentResidenceDuration", "OwnsProperty", "Age", "InstallmentPlans", "Housing", "ExistingCreditsCount", "Job", "Dependents", "Telephone", "ForeignWorker"], 
                  ["no_checking", 20, "prior_payments_delayed", "repairs", 3094, "500_to_1000", "greater_7", 39, "male", "none", 3, "savings_insurance", 29, "none", "own", 1, "skilled", 1, "yes", "yes"],
                   model_serving_url]

input_data = { "input_data": [{ "fields": [ "message" ],
                                "values": [[model_json, test_values_for_ml]]}]}    
function_result = my_deployable_function()( input_data )
print( function_result )

input_data = { "input_data": [{ "fields": [ "message" ],
                                "values": [[model_json,test_values_for_human]]}]}    
function_result = my_deployable_function()( input_data )
print( function_result )

input_data = { "input_data": [{ "fields": [ "message" ],
                                "values": [[test_values_for_human]]}]}    
function_result = my_deployable_function()( input_data )
print( function_result )

{'predictions': [{'fields': ['AIDC Decision'], 'values': [[{'toolToUse': 'ml', 'type': 'ml', 'confidence': 0.8580362993842072, 'outcome': 'No Risk'}]]}]}
{'predictions': [{'fields': ['AIDC Decision'], 'values': [[{'toolToUse': 'human', 'type': 'h'}]]}]}
{'predictions': [{'fields': ['AIDC Decision'], 'values': [[{'toolToUse': 'human', 'type': 'h'}]]}]}


### Prepare software specification for the deployment

In [8]:
#Prepare and deploy the custom function

#Find the AIDC custom software definition
file_path_AIDC = '/project_data/data_asset/aidc-1.4.zip'
specs_list = client.software_specifications.get_details()
for spec in specs_list["resources"]:
    name=(spec['metadata']['name'])
    if("aidc-spec" in name):
        print(name)
        base_software_spec_id =  client.software_specifications.get_id_by_name(name)


# Package extension metadata
meta_prop_pkg_ext = {
     client.package_extensions.ConfigurationMetaNames.NAME: "AIDC_custom_package",
     client.package_extensions.ConfigurationMetaNames.DESCRIPTION: "AIDC custom package",
     client.package_extensions.ConfigurationMetaNames.TYPE: "pip_zip"
}
package_ext_details = client.package_extensions.store(meta_props=meta_prop_pkg_ext, file_path = file_path_AIDC)
package_ext_uid = client.package_extensions.get_uid(package_ext_details)

# Prepare software specificaion
meta_prop_sw_spec = {
     client.software_specifications.ConfigurationMetaNames.NAME: "AIDC_software_spec",
     client.software_specifications.ConfigurationMetaNames.DESCRIPTION: "AIDC software specification",
     client.software_specifications.ConfigurationMetaNames.PACKAGE_EXTENSIONS : [{
         "guid": package_ext_uid
     }],
     client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {
         "guid": base_software_spec_id
     }
}
software_spec_details = client.software_specifications.store(meta_props=meta_prop_sw_spec)
software_spec_uid = client.software_specifications.get_uid(software_spec_details)

#Function meta details
meta_props = {
         client.repository.FunctionMetaNames.NAME: 'AIDC',
         client.repository.FunctionMetaNames.SOFTWARE_SPEC_ID: software_spec_uid
}

#Prepare the function
function_artifact = client.repository.store_function(meta_props=meta_props, function=my_deployable_function)
function_uid = client.repository.get_function_id(function_artifact)

#Prepare hardware specs
hardware_spec_id = client.hardware_specifications.get_id_by_name('S')

#Prepare the deployment metadata
deploy_meta = {
     client.deployments.ConfigurationMetaNames.NAME: "AIDC_function",
     client.deployments.ConfigurationMetaNames.ONLINE: {},
     client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { "id": hardware_spec_id}
}

#Deploy the function
deployment_details = client.deployments.create(function_uid, meta_props=deploy_meta)

aidc-spec
Creating package extensions
SUCCESS


#######################################################################################

Synchronous deployment creation for uid: 'e6b3caa2-7fa2-4988-957e-a7f2c876c9e3' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
........
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='a04a3b93-2fbe-42d5-a52e-388761d1ccab'
------------------------------------------------------------------------------------------------




### Test the function online

In [9]:
#Test the deployed function
deployment_uid = deployment_details['metadata']['id']
   
job_payload = {
     client.deployments.ScoringMetaNames.INPUT_DATA: [{
         "fields": [ "message" ],
         "values": [[model_json, test_values_for_ml]]
     }]
 }

function_result = client.deployments.score(deployment_uid, job_payload)
print( function_result )

job_payload = {
     client.deployments.ScoringMetaNames.INPUT_DATA: [{
         "fields": [ "message" ],
         "values": [[model_json,test_values_for_human]]
     }]
 }

function_result = client.deployments.score(deployment_uid, job_payload)
print( function_result )

job_payload = {
     client.deployments.ScoringMetaNames.INPUT_DATA: [{
         "fields": [ "message" ],
         "values": [[test_values_for_human]]
     }]
 }

function_result = client.deployments.score(deployment_uid, job_payload)
print( function_result )

{'predictions': [{'fields': ['AIDC Decision'], 'values': [[{'toolToUse': 'ml', 'type': 'ml', 'confidence': 0.8580362993842072, 'outcome': 'No Risk'}]]}]}
{'predictions': [{'fields': ['AIDC Decision'], 'values': [[{'toolToUse': 'human', 'type': 'h'}]]}]}
{'predictions': [{'fields': ['AIDC Decision'], 'values': [[{'toolToUse': 'human', 'type': 'h'}]]}]}
